In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import torch
import json
import re


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the model
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/Model')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Model')

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [ ]:
data = [
    {
        "title": "Data item 1",
        "context_type": "Introduction to University",
        "paragraphs":  [
            {
                "context": "Jamia Hamdard, nestled in Jamia Hamdard Nagar, New Delhi, stands as a beacon of educational excellence and innovation since its inception. Declared as a deemed-to-be University in the year 1989, it has consistently strived towards academic brilliance under the leadership of visionaries and educators. Inaugurated by the then Prime Minister of India, Shri Rajiv Gandhi, on August 1, 1989, Jamia Hamdard embarked on a journey dedicated to fostering knowledge and empowerment. During the inauguration, Shri Rajiv Gandhi envisioned Jamia Hamdard as not just a center of learning but as a catalyst for societal progress. He remarked,This will enable minority to go forward and help India to march forward. These words echoed the institutions commitment to inclusivity and national development. The sprawling campus of Jamia Hamdard hosts a myriad of facilities aimed at holistic growth and development. With ten Schools, including a prestigious medical college, two hospitals, a state-of-the-art library, archive center, and modern hostels for both girls and boys, it provides an enriching environment for academic pursuits. Additionally, facilities like a gymnasium, expansive playground, and residential quarters for staff underscore its dedication to the overall well-being of its community. Furthermore, its reach extends beyond Delhi, with an off-campus center in Kannur, Kerala, enriching its educational outreach. In the realm of pharmaceutical sciences, Jamia Hamdard reigns supreme, holding the coveted first position in the country according to the Government of Indias NIRF-2022 ranking. Such accolades are a testament to its commitment to excellence and innovation in academia. Financial support in the form of grants has been instrumental in advancing the institutions research endeavors. Grants from various sources including the PURSE grant, a substantial Rs. 10 Cr allocation from the AYUSH Ministry, and funding under STUITI have fueled groundbreaking research initiatives. The academic prowess of Jamia Hamdard is further evidenced by its scholarly impact, with an impressive h-index surpassing 100. This metric reflects the institutions significant contributions to the advancement of knowledge and research. Collaborations with esteemed organizations like the Defense Research and Development Organization (DRDO) have been pivotal in fostering innovation and technological advancements. The close relationship between Jamia Hamdard and DRDO has led to joint research endeavors yielding pharmaceutical products utilized by defense forces, showcasing the institutions multifaceted contributions to national development. In essence, Jamia Hamdard stands as a testament to the transformative power of education, fostering excellence, inclusivity, and innovation in the pursuit of societal progress and national advancement.",
                "qas": [
                    {
                        "id": "1",
                        "question": "Where is Jamia Hamdard located?",
                        "answers": [{"text": "Jamia Hamdard is located in Jamia Hamdard Nagar, New Delhi.", "answer_start": 1}]
                    },
                    {
                        "id": "2",
                        "question": "When was Jamia Hamdard declared as a deemed-to-be University?",
                        "answers": [{"text": "Jamia Hamdard was declared as a deemed-to-be University in 1989.", "answer_start": 68}]
                    },
                    {
                        "id": "3",
                        "question": "Who inaugurated Jamia Hamdard?",
                        "answers": [{"text": "Jamia Hamdard was inaugurated by the then Prime Minister of India, Shri Rajiv Gandhi on August 1, 1989.", "answer_start": 275}]
                    },
                    {
                        "id": "4",
                        "question": "What did Shri Rajiv Gandhi say about Jamia Hamdard during its inauguration?",
                        "answers": [{"text": "Shri Rajiv Gandhi said, This will enable minority to go forward and help India to march forward. ", "answer_start": 534}]
                    },
                    {
                        "id": "5",
                        "question": "What facilities are available on the campus of Jamia Hamdard?",
                        "answers": [{"text": "Jamia Hamdard has ten Schools including a medical college, two hospitals, a big library, archive centre, separate hostels for girls and boys, gymnasium, a large playground, residential quarters for the staff, and also an off-campus in Kannur, Kerala.", "answer_start": 776}]
                    },
                    {
                        "id": "6",
                        "question": "What is the ranking of Jamia Hamdard in the field of Pharmaceutical sciences in the country?",
                        "answers": [{"text": "Jamia Hamdard is ranked 1st in the field of Pharmaceutical sciences in the country by the Govt. of India (NIRF-2022 ranking).", "answer_start": 1384}]
                    },
                    {
                        "id": "7",
                        "question": "What grants has Jamia Hamdard received?",
                        "answers": [{"text": "Jamia Hamdard has received grants such as the PURSE grant, Rs. 10 Cr from AYUSH Ministry, and funding under STUITI.", "answer_start": 1161}]
                    },
                    {
                        "id": "8",
                        "question": "What is the h-index of Jamia Hamdard?",
                        "answers": [{"text": "The h-index of Jamia Hamdard is more than 100.", "answer_start": 1696}]
                    },
                    {
                        "id": "9",
                        "question": "What is the relationship of Jamia Hamdard with DRDO?",
                        "answers": [{"text": "Jamia Hamdard has a close relationship with DRDO.", "answer_start": 1948}]
                    },
                    {
                        "id": "10",
                        "question": "What development has joint research between Jamia Hamdard and INMAS-DRDO led to?",
                        "answers": [{"text": "Joint research between Jamia Hamdard and INMAS-DRDO has led to the development of pharmaceutical products which are being used by defense force.", "answer_start": 2002}]
                    }]
            }]
        },

     {
            "title": "Data item 2",
        "context_type": "Introduction to University",
        "paragraphs": [
     {          "context": "Hakeem Abdul Hameed, a luminary in the realm of education and healthcare, left an indelible mark as the Founder-Chancellor of Jamia Hamdard. His visionary leadership and unwavering commitment to excellence laid the foundation for the institutions journey towards academic distinction. Throughout his illustrious life, Hakeem Abdul Hameed garnered numerous accolades, reflecting his significant contributions to society. In 1965, he was honored with the Padma Shri, followed by the prestigious Padma Bhushan in 1992. Additionally, he received the esteemed Avicenna Award in 1983, among various other recognitions, underscoring his profound impact and dedication to service. Jamia Hamdard, inaugurated as a Deemed-to-be University in May 1989, stands as a testament to Hakeem Abdul Hameeds vision and foresight. His relentless pursuit of excellence in education and healthcare culminated in the establishment of this esteemed institution, which continues to uphold his legacy of innovation and social responsibility. Beyond Jamia Hamdard, Hakeem Abdul Hameeds influence extended far and wide through the establishment of several other institutions. These include the Hamdard National Foundation, Hamdard Education Society, Hamdard Study Circle, Business and Employment Bureau, Hamdard Public School, Ghalib Academy, Centre for South Asian Studies, and India Islamic Cultural Centre. Each of these institutions reflects his multifaceted approach towards societal development and his enduring commitment to holistic education and cultural enrichment. Hakeem Abdul Hameeds visionary leadership, coupled with his profound dedication to education, healthcare, and cultural preservation, continues to inspire generations. His legacy remains etched in the annals of history, serving as a guiding light for institutions and individuals alike, striving for excellence and societal advancement.",
                "qas": [
                    {
                        "id": "11",
                        "question": "Who was Hakeem Abdul Hameed?",
                        "answers": [{"text": "Hakeem Abdul Hameed was the Founder-Chancellor of Jamia Hamdard.", "answer_start": 23}]
                    },
                    {
                        "id": "12",
                        "question": "What awards were conferred upon Hakeem Abdul Hameed?",
                        "answers": [{"text": "Hakeem Abdul Hameed was conferred with Padma Shri in 1965, Padma Bhushan in 1992, and the Avicenna Award in 1983 among other awards.", "answer_start": 207}]
                    },
                    {
                        "id": "13",
                        "question": "When was Jamia Hamdard inaugurated?",
                        "answers": [{"text": "Jamia Hamdard was inaugurated as a Deemed to be University in May 1989.", "answer_start": 712}]
                    },
                    {
                        "id": "14",
                        "question": "What other institutions did Hakeem Abdul Hameed establish?",
                        "answers": [{"text": "Hakeem Abdul Hameed established several other institutions including Hamdard National Foundation, Hamdard Education Society, Hamdard Study Circle, Business and Employment Bureau, Hamdard Public School, Ghalib Academy, Centre for South Asian Studies, and India Islamic Cultural Centre.", "answer_start": 1562}]
                    }
                ]
            }
        ]
    },
    {
    "title": "Data item 3",
    "context_type": "Introduction to University",
    "paragraphs": [
        {
            "context": "Hammad Ahmed has been a pivotal figure in the Hamdard National Foundation, demonstrating a profound commitment to its mission and objectives. He has been involved with the foundation for more than 50 years, dating back to its inception in 1964. His extensive experience and dedication have significantly contributed to the foundations growth and success.Currently, Hammad Ahmed holds the prestigious position of President of Hamdard National Foundation (India) - HECA, as well as Co-chairman of Hamdard National Foundation (India). His leadership roles within the foundation reflect his deep-rooted involvement and influence in guiding its strategic direction and initiatives.In addition to his work with the Hamdard National Foundation, Hammad Ahmed has played a crucial role at Jamia Hamdard. He has served on the Executive Council, Finance Committee, and various other advisory bodies, providing valuable insights and governance to the university. His contributions have been instrumental in steering Jamia Hamdard towards academic excellence and institutional integrity.Hammad Ahmeds educational background is as impressive as his professional achievements. He received his early education at Sherwood College in Nainital, a renowned institution known for its rigorous academic environment. He furthered his studies in Cornwall, UK, and New York, USA, acquiring a diverse and international educational experience that has undoubtedly enriched his professional and leadership skills.Through his longstanding service and dedication, Hammad Ahmed continues to uphold the values and vision of the Hamdard National Foundation and Jamia Hamdard, fostering growth, excellence, and societal contribution.",
            "qas": [
                {
                    "id": "15",
                    "question": "What is Hammad Ahmeds current role in Hamdard National Foundation?",
                    "answers": [{"text": "Hammad Ahmed is currently President, Hamdard National Foundation (India) - HECA and Co-chairman, Hamdard National Foundation (India).", "answer_start": 222}]
                },
                {
                    "id": "16",
                    "question": "How long has Hammad Ahmed been involved with Hamdard National Foundation?",
                    "answers": [{"text": "Hammad Ahmed has been involved with Hamdard National Foundation for more than 50 years since its inception in 1964.", "answer_start": 77}]
                },
                {
                    "id": "17",
                    "question": "What positions has Hammad Ahmed held at Jamia Hamdard?",
                    "answers": [{"text": "Hammad Ahmed has served in the Executive Council, Finance Committee, and various other advisory bodies of Jamia Hamdard.", "answer_start": 454}]
                },
                {
                    "id": "18",
                    "question": "What is Hammad Ahmeds educational background?",
                    "answers": [{"text": "Hammad Ahmed was educated in Sherwood College, Nainital, and then pursued further studies in Cornwall, UK and New York, USA.", "answer_start": 1286}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 4",
    "context_type": "Introduction to University",
    "paragraphs": [
        {
            "context": "Prof. Mohammad Afshar Alam has been a distinguished figure in the academic and administrative landscape of Jamia Hamdard. He currently serves as the Vice Chancellor of Jamia Hamdard, New Delhi, a role that underscores his leadership and vision for the institutions future.Prof. Alams academic journey is marked by notable achievements. He completed his postgraduate studies (MCA) at Aligarh Muslim University, followed by a Ph.D. from Jamia Milia Islamia. His strong educational foundation has been pivotal in his ascent to various influential positions within Jamia Hamdard.Throughout his tenure at Jamia Hamdard, Prof. Alam has held several critical roles. These include serving as the Head of the Computer Centre, Dean of Management and Information Technology, Dean of Engineering Sciences and Technology, Dean of Students Welfare, Foreign Students Advisor, and Finance Officer. Each of these roles highlights his versatility and dedication to fostering a robust academic environment.Prof. Mohammad Afshar Alams expertise spans multiple domains, reflecting his diverse academic interests and research contributions. His areas of specialization include Software Re-engineering, Data Mining, Bio-Informatics, Fuzzy Databases, and Sustainable Development. This broad spectrum of knowledge underscores his ability to drive interdisciplinary research and innovation, contributing significantly to the academic communitys advancement.In his capacity as Vice Chancellor, Prof. Alam continues to lead Jamia Hamdard with a commitment to excellence, ensuring the institution remains at the forefront of higher education and research. His leadership and expertise are pivotal in navigating the university through the evolving educational landscape, fostering a culture of continuous growth and development.",
            "qas": [
                {
                    "id": "19",
                    "question": "What is Prof. Mohammad Afshar Alams current role at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Mohammad Afshar Alam has been appointed as Vice Chancellor of Jamia Hamdard, New Delhi.", "answer_start": 0}]
                },
                {
                    "id": "20",
                    "question": "What are some of the important roles Prof. Mohammad Afshar Alam has held at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Afshar Alams important roles at Jamia Hamdard include Head of Computer Centre, Dean of Management and Information Technology, Dean of Engineering Sciences and Technology, Dean, Students Welfare, Foreign Students Advisor, and Finance Officer.", "answer_start": 355}]
                },
                {
                    "id": "21",
                    "question": "What is the educational background of Prof. Mohammad Afshar Alam?",
                    "answers": [{"text": "Prof. Mohammad Afshar Alam did his Post graduation (MCA) from the Aligarh Muslim University and (Ph.D.) from Jamia Milia Islamia.", "answer_start": 146}]
                },
                {
                    "id": "22",
                    "question": "What are some of the domains in which Prof. Mohammad Afshar Alams expertise lies?",
                    "answers": [{"text": "Prof. Mohammad Afshar Alams expertise extends across diverse domains, including Software Re-engineering, Data Mining, Bio-Informatics, Fuzzy Databases, and Sustainable Development.", "answer_start": 477}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 5",
    "context_type": "Faculty Information",
    "paragraphs": [
        {
            "context": "Jamia Hamdard, a premier institution in New Delhi, is led by a distinguished team of administrators and academic leaders who ensure the smooth functioning and progressive growth of the university. At the helm of this team is Mr. Hammad Ahmad, who serves as the Chancellor of Jamia Hamdard, bringing his extensive experience and vision to the institution. Assisting him is Prof. Mohammad Afshar Alam, the Vice Chancellor, who is instrumental in steering the university towards academic excellence and innovation.The administrative framework is further strengthened by Dr. M.A. Sikandar, the Registrar, who oversees the universitys administrative operations. The financial matters are managed by Mr. Munish Malik, the Finance Officer, ensuring the financial health and stability of the institution. Mr. Syed Saud Akhtar, responsible for examinations, plays a crucial role in maintaining the academic integrity of the universitys assessment processes.The academic leadership includes several deans who oversee various schools within Jamia Hamdard. Prof. Farhan Jalees Ahmed is the Dean of the School of Pharmaceutical Education and Research, renowned for its top-ranking programs. The School of Nursing Sciences and Allied Health is led by Prof. Shakir Ali, who ensures the provision of high-quality education and training in nursing and allied health sciences. The School of Engineering Sciences & Technology is headed by Prof. Farheen Siddiqui, who drives innovation and research in engineering disciplines. Prof. Reshma Nasreen serves as the Dean of the School of Management & Business Studies, as well as the Director of the Centre for Media and Mass Communication Studies and the Dean of Students Welfare, exemplifying her versatile leadership. Prof. S. Raisuddin is the Dean of the School of Chemical & Life Sciences and also serves as the Director of the IQAC/R & D cell, playing a pivotal role in advancing research and quality assurance.Further, Prof. Asim Ali Khan holds dual roles as the Dean of the School of Unani Medical Education and Research and the Chief Medical Superintendent at Majeedia Unani Hospital, showcasing his expertise in Unani medicine. The Rufaida College of Nursing is led by Principal Prof. Veena Sharma, who upholds the highest standards in nursing education.Other key roles include Prof. M.S. Alam, the Chief Proctor; Prof. Zeenat Iqbal, the Foreign Students Advisor; Prof. Md. Shahnawaz Abdin, the Provost for Boys; and Prof. Syeedun Nisa, the Provost for Girls, each ensuring a supportive and well-regulated campus environment. The universitys library services are managed by Mr. M. Shoaib, the University Librarian, who ensures access to a vast repository of knowledge and resources.Mr. Mirza Rahil Beg serves as the Public Information Officer, handling the dissemination of information, while Mr. Md. Tauhid Alam, the Media Coordinator, manages the universitys media relations. Additionally, Prof. Mahmooduzzafar, as the Director of the Center for Distance and Online Education, facilitates flexible learning opportunities for students.Together, this team of dedicated leaders and administrators at Jamia Hamdard works tirelessly to maintain the universitys esteemed reputation and foster an environment of academic rigor, research innovation, and holistic development.",
            "qas": [
                {
                    "id": "23",
                    "question": "Who is the Chancellor of Jamia Hamdard?",
                    "answers": [{"text": "Mr. Hammad Ahmad", "answer_start": 0}]
                },
                {
                    "id": "24",
                    "question": "Who holds the position of Vice Chancellor at Jamia Hamdard?",
                    "answers": [{"text": "Prof. M. Afshar Alam", "answer_start": 29}]
                },
                {
                    "id": "25",
                    "question": "Who is the Registrar of Jamia Hamdard?",
                    "answers": [{"text": "Dr. M.A. Sikandar", "answer_start": 57}]
                },
                {
                    "id": "26",
                    "question": "Who serves as the Finance Officer of Jamia Hamdard?",
                    "answers": [{"text": "Mr. Munish Malik", "answer_start": 78}]
                },
                {
                    "id": "27",
                    "question": "Who is responsible for examinations at Jamia Hamdard?",
                    "answers": [{"text": "Mr. Syed Saud Akhtar", "answer_start": 101}]
                },
                {
                    "id": "28",
                    "question": "Who is the Dean of School of Pharmaceutical Education and Research?",
                    "answers": [{"text": "Prof. Farhan Jalees Ahmed", "answer_start": 155}]
                },
                {
                    "id": "29",
                    "question": "Who holds the position of Dean at the School of Nursing Sciences and Allied Health?",
                    "answers": [{"text": "Prof. Shakir Ali", "answer_start": 204}]
                },
                {
                    "id": "30",
                    "question": "Who is the Dean of School of Engineering Sciences & Technology?",
                    "answers": [{"text": "Prof. Farheen Siddiqui", "answer_start": 259}]
                },
                {
                    "id": "31",
                    "question": "Who serves as the Dean of School of Management & Business Studies?",
                    "answers": [{"text": "Prof. Reshma Nasreen", "answer_start": 311}]
                },
                {
                    "id": "32",
                    "question": "Who is the Dean of School of Chemical & Life Sciences at Jamia Hamdard?",
                    "answers": [{"text": "Prof. S. Raisuddin", "answer_start": 368}]
                },
                {
                    "id": "33",
                    "question": "Who holds the position of Dean at the School of Unani Medical Education and Research?",
                    "answers": [{"text": "Prof. Asim Ali Khan", "answer_start": 418}]
                },
                {
                    "id": "34",
                    "question": "Who is the Director of the Centre for Media and Mass Communication Studies at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Reshma Nasreen", "answer_start": 471}]
                },
                {
                    "id": "35",
                    "question": "Who is the Director of the Center for Distance and Online Education at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Mahmooduzzafar", "answer_start": 529}]
                },
                {
                    "id": "36",
                    "question": "Who serves as the Dean of Students Welfare at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Reshma Nasreen", "answer_start": 592}]
                },
                {
                    "id": "37",
                    "question": "Who is the Principal of Rufaida College of Nursing at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Veena Sharma", "answer_start": 645}]
                },
                {
                    "id": "38",
                    "question": "Who is the Chief Proctor at Jamia Hamdard?",
                    "answers": [{"text": "Prof. M.S. Alam", "answer_start": 696}]
                },
                {
                    "id": "39",
                    "question": "Who serves as the Foreign Students Advisor at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Zeenat Iqbal", "answer_start": 750}]
                },
                {
                    "id": "40",
                    "question": "Who is the Provost for Boys at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Md. Shahnawaz Abdin", "answer_start": 802}]
                },
                {
                    "id": "41",
                    "question": "Who is the Provost for Girls at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Syeedun Nisa", "answer_start": 860}]
                },
                {
                    "id": "42",
                    "question": "Who serves as the Director of IQAC/R & D cell at Jamia Hamdard?",
                    "answers": [{"text": "Prof. S. Raisuddin", "answer_start": 915}]
                },
                {
                    "id": "43",
                    "question": "Who is the University Librarian at Jamia Hamdard?",
                    "answers": [{"text": "Mr. M. Shoaib", "answer_start": 966}]
                },
                {
                    "id": "44",
                    "question": "Who holds the position of Chief Medical Superintendent at Majeedia Unani Hospital, Jamia Hamdard?",
                    "answers": [{"text": "Prof. Asim Ali Khan", "answer_start": 1017}]
                },
                {
                    "id": "45",
                    "question": "Who is the Public Information Officer at Jamia Hamdard?",
                    "answers": [{"text": "Mr. Mirza Rahil Beg", "answer_start": 1074}]
                },
                {
                    "id": "46",
                    "question": "Who serves as the Media Coordinator at Jamia Hamdard?",
                    "answers": [{"text": "Mr. Md. Tauhid Alam", "answer_start": 1127}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 6",
    "context_type": "Academics",
    "paragraphs": [
        {
            "context": "The School of Pharmaceutical Education & Research (SPER) at Jamia Hamdard stands as a beacon of excellence in pharmaceutical education and research. Under the adept leadership of Prof. (Dr.) Farhan Jalees Ahmed, the school has achieved remarkable milestones and accolades.The Ministry of Education, Government of India, has consistently recognized SPERs outstanding contributions to the field. The school was ranked No. 1 among national institutions in the National Institutional Ranking Framework (NIRF) for the years 2019, 2020, 2021, and 2022, and secured the No. 2 position in 2023. These rankings underscore the schools commitment to maintaining high standards in education, research, and innovation.SPER offers a comprehensive range of academic programs designed to cater to various levels of pharmaceutical education. These include undergraduate programs such as the Diploma in Pharmacy (D. Pharm.) and Bachelor of Pharmacy (B. Pharm.), postgraduate programs (M. Pharm.) in specialties like Pharmaceutics, Pharmaceutical Chemistry, Pharmacology, Pharmacognosy, Pharmaceutical Quality Assurance, Pharmaceutical Analysis, Pharmacy Practice, and Pharmaceutical Biotechnology, as well as rigorous doctoral programs. These diverse programs equip students with the knowledge and skills required to excel in the dynamic field of pharmaceuticals.The school has also established robust collaborations with prominent industries, enhancing the practical and research experience for its students and faculty. Notable industry partners include Sun Pharma, Dabur Research Foundation, Jubilant Life Sciences, Hamdard Labs India Ltd (Food Division), AIMIL Pharmaceuticals International, Fermish Clinical Technology, GBC in Beijing, China, Multani Dolphin Pharmaceuticals, Avitech Nutrition, and Tulips Lab. These collaborations facilitate cutting-edge research, internships, and job placements, thereby bridging the gap between academic learning and industry requirements.The continuous efforts and strategic initiatives led by Prof. (Dr.) Farhan Jalees Ahmed and his team have propelled the School of Pharmaceutical Education & Research at Jamia Hamdard to the forefront of pharmaceutical education, making it a preferred destination for aspiring pharmaceutical professionals.",
            "qas": [
                {
                    "id": "47",
                    "question": "Who is the Dean of the School of Pharmaceutical Education & Research (SPER) at Jamia Hamdard?",
                    "answers": [{"text": "Prof. (Dr.) Farhan Jalees Ahmed", "answer_start": 8}]
                },
                {
                    "id": "48",
                    "question": "How has the School of Pharmaceutical Education & Research (SPER) been ranked by the Ministry of Education, Govt. of India?",
                    "answers": [{"text": "No. 1 amongst National Institutions Ranking Framework (NIRF) -2019, 2020, 2021, 2022 and at No. 2 in 2023", "answer_start": 213}]
                },
                {
                    "id": "49",
                    "question": "What academic programs are offered by the School of Pharmaceutical Education & Research (SPER)?",
                    "answers": [{"text": "Undergraduate (D. Pharm. & B. Pharm.), Postgraduate (M. Pharm. in Pharmaceutics, Pharmaceutical Chemistry, Pharmacology, Pharmacognosy, Pharmaceutical Quality Assurance, Pharmaceutical Analysis, Pharmacy Practice and Pharmaceutical Biotechnology) and Doctoral programs", "answer_start": 432}]
                },
                {
                    "id": "50",
                    "question": "What are some examples of industries collaborating with the School of Pharmaceutical Education & Research (SPER)?",
                    "answers": [{"text": "Sun Pharma, Dabur Research foundation, Jubilant Life Sciences, Hamdard Labs India Ltd (Food Div.), AIMIL Pharmaceuticals International, Fermish Clinical Technology, GBC, Beijing, China and Multani Dolphin Pharmaceuticals, Avitech Nutrition and Tulips Lab", "answer_start": 878}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 7",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The School of Pharmaceutical Education & Research (SPER) at Jamia Hamdard comprises several specialized departments, each dedicated to advancing education and research in their respective fields. These departments include the Department of Pharmaceutics, the Department of Pharmaceutical Chemistry, the Department of Pharmacology, and the Department of Pharmacognosy and Phytochemistry.The Department of Pharmaceutics offers two masters degree programs in Pharmaceutics and Pharmaceutical Quality Assurance, as well as Ph.D. programs in Pharmaceutics and Pharmaceutics with a specialization in Pharmaceutical Quality Assurance. The departments research focuses on areas such as oral bioavailability enhancement, transdermal drug delivery systems, ocular drug delivery systems, targeted nanoparticles, intranasal drug delivery systems, biodegradable targeted retention devices, herbal drug delivery systems, and transmucosal drug delivery systems.The Department of Pharmaceutical Chemistry provides academic programs including M. Pharm. in Pharmaceutical Chemistry and Pharmaceutical Analysis, along with a Ph.D. program. The departments research encompasses Drug Design and Development, Synthetic Chemistry, Analytical Pharmacy, and Natural Product Chemistry, contributing significantly to the field of pharmaceutical sciences.The Department of Pharmacology offers a masters program in Pharmacology and Pharmacy Practice, and Ph.D. programs in Pharmacology, Pharmacology with a specialization in Pharmacy Practice, and Pharmaceutical Medicine. The research conducted by this department covers Neuropharmacology, Cardiovascular Pharmacology, Endocrinology, and Clinical Pharmacology, aiming to advance understanding and treatment of various medical conditions.The Department of Pharmacognosy & Phytochemistry focuses on the discovery and development of herbal drugs, phytopharmaceuticals, and microbial products. Their academic offerings include programs that cater to the study and research of these natural compounds. The departments research areas include quality control of herbal/microbial products and botanicals, enhanced production of secondary metabolites, studies on herbal cosmetics, immunomodulatory, anti-obesity, and anti-diabetic medicinal plants, screening of new drug molecules, and the isolation, identification, and characterization of natural compounds.Additionally, the Centre of Excellence in Unani Medicine at Jamia Hamdard is dedicated to conducting scientific investigations related to Traditional Unani Medicine, contributing to the integration of traditional knowledge with modern scientific research.Each of these departments and research centers at SPER plays a crucial role in advancing pharmaceutical education and research, making Jamia Hamdard a leading institution in the field.",
            "qas": [
                {
                    "id": "51",
                    "question": "What departments are part of the School of Pharmaceutical Education & Research (SPER) at Jamia Hamdard?",
                    "answers": [{"text": "1. Department of Pharmaceutics 2. Department of Pharmaceutical Chemistry 3. Department of Pharmacology 4. Department of Pharmacognosy and Phytochemistry", "answer_start": 8}]
                },
                {
                    "id": "52",
                    "question": "What academic programs are offered by the Department of Pharmaceutics?",
                    "answers": [{"text": "Two masters degree programs in Pharmaceutics and Pharmaceutical Quality Assurance, and Ph.D. in Pharmaceutics and Pharmaceutics with specialization in Pharmaceutical Quality Assurance", "answer_start": 288}]
                },
                {
                    "id": "53",
                    "question": "What are the research thrust areas of the Department of Pharmaceutics?",
                    "answers": [{"text": "Oral bioavailability enhancement, transdermal drug delivery system, ocular drug delivery systems, targeted nanoparticles, intranasal drug delivery systems, biodegradable targeted retention devices, herbal drug delivery systems, and transmucosal drug delivery systems", "answer_start": 477}]
                },
                {
                    "id": "54",
                    "question": "What academic programs are offered by the Department of Pharmaceutical Chemistry?",
                    "answers": [{"text": "M. Pharm. (Pharmaceutical Chemistry, Pharmaceutical Analysis) and Ph.D. Programme", "answer_start": 1015}]
                },
                {
                    "id": "55",
                    "question": "What research areas are covered by the Department of Pharmaceutical Chemistry?",
                    "answers": [{"text": "Drug Design and Development, Synthetic Chemistry, Analytical Pharmacy, and Natural Product Chemistry", "answer_start": 1155}]
                },
                {
                    "id": "56",
                    "question": "What academic programs are offered by the Department of Pharmacology?",
                    "answers": [{"text": "Masters program in Pharmacology, Pharmacy Practice, and Ph.D. program in Pharmacology, Pharmacology with specialization in Pharmacy Practice, and Ph.D. in Pharmaceutical Medicine", "answer_start": 1362}]
                },
                {
                    "id": "57",
                    "question": "What research areas are covered by the Department of Pharmacology?",
                    "answers": [{"text": "Neuropharmacology, Cardiovascular Pharmacology, Endocrinology, and Clinical pharmacology", "answer_start": 1686}]
                },
                {
                    "id": "58",
                    "question": "What academic programs are offered by the Department of Pharmacognosy & Phytochemistry?",
                    "answers": [{"text": "Mention the academic programs offered by the Department of Pharmacognosy & Phytochemistry.", "answer_start": 2239}]
                },
                {
                    "id": "59",
                    "question": "What are the research areas covered by the Department of Pharmacognosy & Phytochemistry?",
                    "answers": [{"text": "Discovery and development of herbal drugs, phytopharmaceuticals, and microbial products; quality control of herbal/microbial products and botanicals; enhanced production of secondary metabolites; studies on herbal cosmetics, immunomodulatory, anti-obesity, and anti-diabetic medicinal plants; screening of new drug molecules; isolation, identification, and characterization of natural compounds", "answer_start": 2411}]
                },
                {
                    "id": "60",
                    "question": "What is the Centre of Excellence in Unani Medicine dedicated to?",
                    "answers": [{"text": "The Centre of Excellence in Unani Medicine is dedicated to conducting scientific investigations related to Traditional Unani Medicine.", "answer_start": 3001}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 8",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "The School of Pharmaceutical Education & Research (SPER) at Jamia Hamdard offers a range of academic programs across different levels, each with specific eligibility criteria and selection processes. Here is an overview of the programs and their requirements:The Diploma in Pharmacy (D. Pharm.) program requires candidates to have passed the Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry, and Biology or Mathematics, securing at least 45% marks in the aggregate and in these subjects. Candidates must be at least 17 years old by December 31, 2024, as per PCI norms. Admission is based on merit from the qualifying examination (10+2 PCB/PCM stream).For the Bachelor of Pharmacy (B. Pharm.) program, candidates must have passed the Senior Secondary examination with Physics, Chemistry, and Biology/Mathematics, securing at least 50% marks in aggregate. Candidates must be at least 17 years old by December 31, 2024. Selection is based on NEET 2024 or Paper-1 of JEE (Main) 2024 scores. Non-NEET/non-JEE candidates may apply but will be considered only if NEET/JEE candidates are unavailable.The B. Pharm. II Year (Lateral entry) program requires candidates to have passed the Diploma in Pharmacy examination with 50% marks in aggregate and the 10+2 examination with PCB. Admission is based on merit determined by D. Pharm. marks. Preference is given to those who have completed 500 hours of practical training over at least three months.The Master of Pharmacy (M. Pharm.) program requires candidates to have passed the B. Pharm. examination with at least 55% marks. Preference is given to GPAT qualified candidates. For the Sponsored category, candidates need two years of full-time work experience. Admission is based on GPAT merit; if insufficient GPAT candidates are available, B. Pharm. percentage is considered.The Ph.D. programs at Jamia Hamdard, including Ph.D. in Pharmaceutical Medicine, require an M. Pharm. in relevant specializations or an MBBS degree with at least 55% marks. Preference is given to candidates with MD/DNB degrees in related fields. Two seats in the Ph.D. (Pharmaceutical Medicine) program are reserved for Pharmaceutical Industry nominated candidates, one under General and one under SFS category. Notifications for Ph.D. admissions for the 2024-25 session will be released in July 2024.SPER at Jamia Hamdard also offers scholarships for GPAT qualified M. Pharm. students through UGC, supporting the academic and research pursuits of its students.This comprehensive range of programs and the robust selection criteria ensure that SPER at Jamia Hamdard continues to produce highly qualified professionals in the field of pharmaceutical education and research.",
            "qas": [
                {
                    "id": "61",
                    "question": "What is the eligibility criteria for the Diploma in Pharmacy (D. Pharm.) program?",
                    "answers": [{"text": "Passed Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry and Biology or Mathematics from Central Board of Secondary Education or any other examination recognized by Jamia Hamdard as equivalent thereto, securing at least 45% marks in the aggregate and also in the aggregate of Physics, Chemistry and Biology or Mathematics. Completed the age of 17 years on or before 31st December 2024 as per PCI norms.", "answer_start": 239}]
                },
                {
                    "id": "62",
                    "question": "What is the selection process for the Diploma in Pharmacy (D. Pharm.) program?",
                    "answers": [{"text": "Admission to D. Pharm. Programme shall be made through merit of qualifying examination (10+2 PCB/PCM stream). Allocation of seats for PCB/PCM stream candidates will depend upon the number of applications received under a category (PCB/PCM stream).", "answer_start": 413}]
                },
                {
                    "id": "63",
                    "question": "What is the eligibility criteria for the Bachelor of Pharmacy (B. Pharm.) program?",
                    "answers": [{"text": "Passed Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry and Biology/Mathematics from Central Board of Secondary Education or any other examination recognized by Jamia Hamdard as equivalent thereto securing at least 50% marks in the aggregate and also in the aggregate of Physics, Chemistry and Biology/Mathematics. Completed the age of 17 years on or before 31st December 2024 as per PCI norms.", "answer_start": 643}]
                },
                {
                    "id": "64",
                    "question": "What is the selection process for the Bachelor of Pharmacy (B. Pharm.) program?",
                    "answers": [{"text": "Appeared in NEET 2024/Paper-1 of JEE (Main) 2024. Selection will be based on merit in NEET 2024/Paper-1 of JEE (Main) 2024. Non-NEET/non-JEE candidates may also apply, however NEET-2024/JEE-2024 candidates will be preferred. Non-NEET/non-JEE applicants may be considered only if suitable number of NEET-2024/JEE-2024 candidates are not available. In case of delay in conduct of NEET-2024/JEE-2024 the University may take admissions on the basis of merit of aggregate percentage of first five subjects of the qualifying examination.", "answer_start": 811}]
                },
                {
                    "id": "65",
                    "question": "What is the eligibility criteria for the B. Pharm. II Year (Lateral entry) program?",
                    "answers": [{"text": "Passed Diploma in Pharmacy examination of Jamia Hamdard or from any other recognized institution with 50% marks in the aggregate. Passed 10+2 examination from CBSE or any other board recognized by Jamia Hamdard as equivalent thereto with PCB.", "answer_start": 1102}]
                },
                {
                    "id": "66",
                    "question": "What is the selection process for the B. Pharm. II Year (Lateral entry) program?",
                    "answers": [{"text": "Admission to B. Pharm. II Year will be made on the basis of merit determined by marks obtained in D. Pharm. (all semesters/years). Candidates have to submit the result of the D. Pharm., preferably on or before the last date of submission of application form or as notified by the University. Preference will be given to candidates who have completed 500 hours practical training spread over a period of not less than 3 months. Candidate with result awaited may be offered admission only after submission of final result, if seats remain vacant.", "answer_start": 1207}]
                },
                {
                    "id": "67",
                    "question": "What is the eligibility criteria for the Master of Pharmacy (M. Pharm.) program?",
                    "answers": [{"text": "Passed B. Pharm. examination from an institution recognized by Jamia Hamdard and approved by PCI and securing at least 55% marks in the aggregate of theory papers and practical. Preference will be given to GPAT qualified candidates. For consideration under the Sponsored category, candidates should have a two-year full time work experience in a registered firm/industry/educational institution. If experienced candidates are not available, consideration to fresh candidates may be given.", "answer_start": 1512}]
                },
                {
                    "id": "68",
                    "question": "What is the selection process for the Master of Pharmacy (M. Pharm.) program?",
                    "answers": [{"text": "The admission to the M. Pharm. Programme will be offered on the basis of merit of GPAT qualified candidates. If sufficient numbers of GPAT qualified candidates are not available, admission will be subsequently offered on the basis of B. Pharm. percentage. Admission under NRI/Sponsored category candidates will also be made on the basis of B. Pharm. percentage.", "answer_start": 1794}]
                },
                {
                    "id": "69",
                    "question": "What programs offer scholarships to students through UGC?",
                    "answers": [{"text": "M. Pharm. students (GPAT qualified) through UGC", "answer_start": 2191}]
                },
                {
                    "id": "70",
                    "question": "When will the notification for admission in Ph.D. programs for the session 2024-25 be released?",
                    "answers": [{"text": "The notification for admission in Ph.D. program from session 2024-25 will be released in the month of July 2024.", "answer_start": 2928}]
                },
                {
                    "id": "71",
                    "question": "What is the eligibility criteria for the Ph.D. (Pharmaceutical Medicine) program?",
                    "answers": [{"text": "M. Pharm. in Pharmacology/Hospital Pharmacy/Clinical Pharmacy/Pharmaceutics or MBBS degree securing at least 55% marks in the aggregate or equivalent grade from Jamia Hamdard or from any other institution, recognized by PCI/AIU and Jamia Hamdard. Preference will be given to MD/DNB degree holders in Pharmacology/Medicine/Clinical Pharmacology.", "answer_start": 3227}]
                },
                {
                    "id": "72",
                    "question": "What seats are reserved for Pharmaceutical Industry nominated candidates in the Ph.D. (Pharmaceutical Medicine) program?",
                    "answers": [{"text": "Two seats are reserved for Pharmaceutical Industry nominated candidates, one under General and one under SFS category.", "answer_start": 3292}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 9",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The School of Nursing Sciences and Allied Health (SNSAH) at Jamia Hamdard encompasses a variety of departments dedicated to healthcare education and training. These include the College of Nursing (Rufaida College of Nursing, RCN), the Department of Occupational Therapy (DOT), the Department of Physiotherapy (DPT), the Department of Paramedical Sciences (DPS), and the Rehabilitation Centre. Each of these departments offers specialized programs designed to equip students with the necessary skills and knowledge for their respective fields.Rufaida College of Nursing (RCN) offers a comprehensive range of programs including the Diploma in General Nursing and Midwifery, B.Sc. (Hons.) Nursing, M.Sc. Nursing, Post Basic B.Sc. Nursing, and Ph.D. in Nursing. The colleges motto, “Par excellence in preparing competent nurse professionals,” reflects its commitment to producing highly skilled nursing graduates. All courses offered by Rufaida College of Nursing are recognized by the Indian Nursing Council (INC) and the State Nursing Council–Delhi Nursing Council (DNC), ensuring that the education provided meets national standards.Students at Rufaida College of Nursing receive extensive clinical training, gaining hands-on experience in the parent hospital, Hakeem Abdul Hameed Centenary Hospital (HAHC), as well as other affiliated hospitals such as Safdarjung Hospital, VIMHANS, IHBAS, and various rural and urban health centers operated by M.C.D. and Jamia Hamdard. This diverse clinical exposure is designed to prepare students for real-world healthcare environments.The college is well-equipped with state-of-the-art infrastructure and learning resources. It boasts a dedicated building with an upgraded simulation lab and a well-maintained library, providing students with excellent facilities for both practical and theoretical learning. This robust educational environment supports the colleges mission to deliver high-quality nursing education and research opportunities.",
            "qas": [
                {
                    "id": "73",
                    "question": "What departments are part of the School of Nursing Sciences and Allied Health (SNSAH)?",
                    "answers": [{"text": "The departments include the College of Nursing (Rufaida College of Nursing, RCN), Department of Occupational Therapy (DOT), Department of Physiotherapy (DPT), Department of Paramedical Sciences (DPS), and Rehabilitation Centre.", "answer_start": 181}]
                },
                {
                    "id": "74",
                    "question": "What programs does Rufaida College of Nursing offer?",
                    "answers": [{"text": "Rufaida College of Nursing offers Diploma in General Nursing and Midwifery Programme, B.Sc. (Hons.) Nursing, M.Sc. Nursing, Post Basic B.Sc. Nursing, and Ph.D. in Nursing.", "answer_start": 841}]
                },
                {
                    "id": "75",
                    "question": "What is the motto of Rufaida College of Nursing?",
                    "answers": [{"text": "The motto is “Par excellence in preparing competent nurse professionals”.", "answer_start": 1191}]
                },
                {
                    "id": "76",
                    "question": "How are the courses offered by Rufaida College of Nursing recognized?",
                    "answers": [{"text": "The courses offered are recognized by the Indian Nursing Council (INC) and the State Nursing Council–Delhi Nursing Council (DNC).", "answer_start": 1385}]
                },
                {
                    "id": "77",
                    "question": "Where do students receive clinical training from Rufaida College of Nursing?",
                    "answers": [{"text": "The students receive rich clinical, hands-on training in the parent hospital i.e. Hakeem Abdul Hameed Centenary Hospital (HAHC) and other affiliated hospitals, such as Safdarjung Hospital, VIMHANS, IHBAS, rural and urban Health Centers, run by M.C.D. and Jamia Hamdard.", "answer_start": 1449}]
                },
                {
                    "id": "78",
                    "question": "What facilities does Rufaida College of Nursing have?",
                    "answers": [{"text": "The college has state-of-the-art infrastructure and excellent learning resources and research facilities. It has its own building with upgraded simulation lab and well-maintained library.", "answer_start": 1696}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 10",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "The School of Nursing Sciences and Allied Health (SNSAH) at Jamia Hamdard offers a range of programs aimed at preparing competent healthcare professionals.Diploma in General Nursing and Midwifery (DGNM)Eligibility criteria for the DGNM program include passing the 10+2 examination with English, securing a minimum of 40% marks, and passing English individually. Candidates must also be at least 17 years old by 31st December 2024. Subjects required for admission include Physics, Chemistry, Biology, Mathematics, Commerce, Accountancy, Economics, Business Studies, Computer Science, and others.B.Sc. (Hons.) NursingFor the B.Sc. (Hons.) Nursing program, candidates must be at least 17 years old by 31st December 2024 and have passed the Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry, Biology, and English from a recognized board, securing at least 45% marks in aggregate. Selection is based on merit from NEET 2024.Post Basic B.Sc. NursingThe Post Basic B.Sc. Nursing program requires candidates to have a DGNM qualification from a recognized institution and be registered as RN (Registered Nurse) and RM (Registered Midwife) under the State Nursing Council. Candidates must be at least 17 years old by 31st December 2024. Selection may involve an entrance test conducted by Jamia Hamdard or be based on the merit of the qualifying examination.M.Sc. NursingEligibility for the M.Sc. Nursing program includes having a B.Sc. Nursing, B.Sc. (Hons.) Nursing, or Post Basic B.Sc. Nursing with at least 55% marks in aggregate from a recognized institution, being a Registered Nurse and Registered Midwife or equivalent, and having a minimum of one year of work experience in the nursing field. Candidates are selected based on the merit of the qualifying examination.Ph.D. in NursingThe Ph.D. in Nursing program requires candidates to have an M.Sc. Nursing or a postgraduate degree in Nursing recognized by INC with a minimum of 60% marks. Additionally, candidates must have three years of teaching or clinical experience after their M.Sc. Nursing. Selection is based on the merit of the entrance test and an interview conducted by Jamia Hamdard.These programs are designed to equip students with both theoretical knowledge and practical skills, preparing them for various professional roles in the healthcare sector.",
            "qas": [
                {
                    "id": "79",
                    "question": "What are the eligibility criteria for the Diploma in General Nursing and Midwifery (DGNM) program?",
                    "answers": [{"text": "Eligibility criteria for the DGNM program include passing 10+2 with English and obtaining a minimum of 40% at the qualifying examination, passing English individually, and completing the age of 17 years on or before 31st December 2024.", "answer_start": 239}]
                },
                {
                    "id": "80",
                    "question": "What subjects are required for admission to the DGNM program?",
                    "answers": [{"text": "Subjects required for admission to the DGNM program include Physics, Chemistry, Biology, Mathematics, Commerce, Accountancy, Economics, Business Studies, Computer Science, Informatics Practice, Word processing, Multimedia, Geography, History, Civics, Sociology, Psychology, Political Science, and Home Science.", "answer_start": 733}]
                },
                {
                    "id": "81",
                    "question": "What is the eligibility criteria for the B.Sc. (Hons.) Nursing program?",
                    "answers": [{"text": "Eligibility criteria for the B.Sc. (Hons.) Nursing program include completing the age of 17 years on or before 31st December 2024, passing Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry, Biology, and English from a recognized board, and securing at least 45% marks in the aggregate.", "answer_start": 1538}]
                },
                {
                    "id": "82",
                    "question": "How are candidates selected for the B.Sc. (Hons.) Nursing program?",
                    "answers": [{"text": "Candidates for the B.Sc. (Hons.) Nursing program are selected based on merit based on NEET 2024.", "answer_start": 1877}]
                },
                {
                    "id": "83",
                    "question": "What is the eligibility criteria for the Post Basic B.Sc. Nursing program?",
                    "answers": [{"text": "Eligibility criteria for the Post Basic B.Sc. Nursing program include having a DGNM qualification from a recognized institution, being registered as RN (Registered Nurse) and RM (Registered Midwife) under State Nursing Council, and completing the age of 17 years on or before 31st December 2024.", "answer_start": 2149}]
                },
                {
                    "id": "84",
                    "question": "How are candidates selected for the Post Basic B.Sc. Nursing program?",
                    "answers": [{"text": "Candidates for the Post Basic B.Sc. Nursing program may appear in the Entrance Test conducted by Jamia Hamdard or be selected based on merit of the qualifying examination.", "answer_start": 2512}]
                },
                {
                    "id": "85",
                    "question": "What is the eligibility criteria for the M.Sc. Nursing program?",
                    "answers": [{"text": "Eligibility criteria for the M.Sc. Nursing program include having passed B.Sc. Nursing/ B.Sc. (Hons.) Nursing/ Post Basic B.Sc. Nursing with a minimum of 55% marks in aggregate from a recognized institution, being a Registered Nurse and Registered Midwife or equivalent, and having a minimum of one year of work experience in nursing field.", "answer_start": 2823}]
                },
                {
                    "id": "86",
                    "question": "How are candidates selected for the M.Sc. Nursing program?",
                    "answers": [{"text": "Candidates for the M.Sc. Nursing program are selected based on merit of qualifying examination.", "answer_start": 3111}]
                },
                {
                    "id": "87",
                    "question": "What is the eligibility criteria for the Ph.D. in Nursing program?",
                    "answers": [{"text": "Eligibility criteria for the Ph.D. in Nursing program include having an M.Sc. Nursing or a Post Graduate degree in Nursing recognized by INC with a minimum of 60% marks, and having three years teaching or clinical experience after M.Sc. Nursing.", "answer_start": 3362}]
                },
                {
                    "id": "88",
                    "question": "How are candidates selected for the Ph.D. in Nursing program?",
                    "answers": [{"text": "Candidates for the Ph.D. in Nursing program are selected based on merit of entrance test and interview conducted by Jamia Hamdard.", "answer_start": 3688}]
                }
            ]
}
    ]
},
{
    "title": "Data item 11",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Department of Occupational Therapy at Jamia Hamdard has been a cornerstone of academic excellence since its inception in 2000 and 2001. Offering a diverse range of academic programs, it has played a pivotal role in shaping the future of healthcare professionals in the field of occupational therapy.With a focus on both undergraduate and postgraduate education, the Department offers comprehensive programs in Occupational Therapy. Additionally, it provides avenues for advanced research through Ph.D. programs in Rehabilitation Science and Occupational Therapy.Occupational therapy, as envisioned by the department, aims to enable individuals to participate fully in the activities of everyday life. This holistic approach is reflected in the multifaceted roles played by occupational therapists, ranging from clinical practitioners to researchers, academicians, counselors, diplomats, and rehabilitation directors.In India, occupational therapists find employment opportunities across various sectors, including government organizations, institute hospitals, projects, non-governmental organizations, and the private sector. They contribute significantly to healthcare delivery in settings such as acute care hospitals, nursing homes, and special schools.Students enrolled in the Department of Occupational Therapy receive a well-rounded education encompassing both theoretical knowledge and practical skills. Clinical exposure and internships form an integral part of their training, with opportunities available in government and private hospitals, NGOs, and special schools across Delhi and the National Capital Region (NCR).The department takes pride in its alumni, boasting a remarkable 100% placement rate. Many alumni have gone on to serve in prestigious institutions such as LN Hospital, GB Pant Hospital, Kalawati Saran Childrens Hospital, ESI Hospital, PDDUNIPPD, and Ganga Ram Hospital.Beyond national borders, the impact of the Department of Occupational Therapy extends globally, with alumni settled in countries including the USA, England, Ireland, UAE, Singapore, Republic of Congo, Australia, and Canada. Their contributions resonate across continents, showcasing the departments commitment to excellence and its enduring legacy in the field of occupational therapy.",
            "qas": [
                {
"id": "89",
                    "question": "When did the Department of Occupational Therapy start offering academic programs?",
                    "answers": [{"text": "The Department of Occupational Therapy started offering academic programs since 2000 and 2001.", "answer_start": 0}]
                },
                {
                    "id": "90",
                    "question": "What programs are offered by the Department of Occupational Therapy?",
                    "answers": [{"text": "The Department of Occupational Therapy offers Postgraduate and Undergraduate programs in Occupational Therapy, as well as Ph.D. programs in Rehabilitation Science and Occupational Therapy.", "answer_start": 124}]
                },
                {
                    "id": "91",
                    "question": "What is the primary goal of occupational therapy?",
                    "answers": [{"text": "The primary goal of occupational therapy is to enable people to participate in the activities of everyday life.", "answer_start": 506}]
                },
                {
                    "id": "92",
                    "question": "What are some of the roles played by Occupational Therapists?",
                    "answers": [{"text": "Some of the roles played by Occupational Therapists include Clinical, Researcher, Academician, Counsellor, Diplomat, and Rehabilitation Director.", "answer_start": 1546}]
                },
                {
                    "id": "93",
                    "question": "Where do Occupational Therapists in India generally practice?",
                    "answers": [{"text": "Occupational Therapists in India generally practice in Government organizations/institute hospitals/projects, Non-Government organizations, and the private sector such as acute care hospitals & nursing homes.", "answer_start": 1821}]
                },
                {
                    "id": "94",
                    "question": "What kind of exposure do students receive in the Department of Occupational Therapy?",
                    "answers": [{"text": "Students in the Department of Occupational Therapy receive exposure to theoretical and practical aspects of the field, including clinical exposure and internships in government and private hospitals, NGOs, and special schools in Delhi & NCR.", "answer_start": 2121}]
                },
                {
                    "id": "95",
                    "question": "What is the placement rate for alumni of the Department of Occupational Therapy?",
                    "answers": [{"text": "The alumni of the Department of Occupational Therapy have a 100% placement rate, with many serving in hospitals like LN Hospital, GB pant Hospital, Kalawati Saran Childrens Hospital, ESI hospital, PDDUNIPPD, and Ganga Ram Hospital.", "answer_start": 2512}]
                },
                {
                    "id": "96",
                    "question": "Where are some of the alumni of the Department of Occupational Therapy settled internationally?",
                    "answers": [{"text": "Some alumni of the Department of Occupational Therapy are settled in countries like USA, England, Ireland, UAE, Singapore, Republic of Congo, Australia, and Canada.", "answer_start": 2783}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 12",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "At Jamia Hamdard, the Bachelor of Occupational Therapy (BOT) program offers a gateway into the dynamic field of occupational therapy. With a commitment to excellence, Jamia Hamdard provides comprehensive education and training in occupational therapy to meet the evolving healthcare needs of society.Bachelor of Occupational Therapy Program:Available Seats: The BOT program at Jamia Hamdard has 50 seats available, offering ample opportunities for aspiring occupational therapists to pursue their education and career goals.Program Duration: The duration of the BOT program spans four years, ensuring a comprehensive academic curriculum and practical training. Additionally, students undergo a compulsory internship of six months to gain hands-on experience in real-world healthcare settings.Eligibility Criteria: To be eligible for admission to the BOT program, candidates must have passed the Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry, and Biology from a recognized board. They should secure at least 50% marks in the aggregate and also in the aggregate of Physics, Chemistry, and Biology, demonstrating their academic proficiency and readiness for the program.Selection Process: The selection process for the BOT program is based on 10+2 merit among the candidates applying through the Jamia Hamdard Online Admission Portal. This ensures a fair and transparent method of selecting candidates who exhibit academic excellence and potential for success in the field of occupational therapy.Jamia Hamdards commitment to providing quality education and training in occupational therapy sets the stage for aspiring healthcare professionals to make a meaningful impact in improving the lives of individuals through rehabilitation, therapy, and holistic care.",
            "qas": [
                {
                    "id": "97",
                    "question": "How many seats are available in the Bachelor of Occupational Therapy program at Jamia Hamdard?",
                    "answers": [{"text": "The Bachelor of Occupational Therapy (BOT) program at Jamia Hamdard has 50 seats available.", "answer_start": 0}]
                },
                {
                    "id": "98",
                    "question": "What is the duration of the Bachelor of Occupational Therapy program?",
                    "answers": [{"text": "The Bachelor of Occupational Therapy (BOT) program is four years long, with a compulsory internship of six months.", "answer_start": 96}]
                },
                {
                    "id": "99",
                    "question": "What are the eligibility criteria for the Bachelor of Occupational Therapy program?",
                    "answers": [{"text": "The eligibility criteria include passing the Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry, and Biology from a recognized board, securing at least 50% marks in the aggregate and also in the aggregate of Physics, Chemistry, and Biology.", "answer_start": 298}]
                },
                {
                    "id": "100",
                    "question": "How is selection done for the Bachelor of Occupational Therapy program?",
                    "answers": [{"text": "Selection is based on 10+2 merit among the candidates applying through the Jamia Hamdard Online Admission Portal.", "answer_start": 505}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 13",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "Specializations in the Master of Occupational Therapy program:The Master of Occupational Therapy (MOT) program at Jamia Hamdard offers specializations in Musculoskeletal and Pediatrics. This enables students to focus their studies on specific areas of occupational therapy practice.Duration of the Master of Occupational Therapy program:The Master of Occupational Therapy (MOT) program spans over 2 years, providing students with a comprehensive and in-depth understanding of occupational therapy principles and practices.Eligibility Criteria for the Master of Occupational Therapy program:To be eligible for the Master of Occupational Therapy (MOT) program, candidates must have completed a Bachelor of Occupational Therapy (BOT) from an institution recognized by Jamia Hamdard, with a minimum aggregate score of 50%. Candidates who obtained their BOT degree through distance education are not eligible for admission to the MOT program.Selection Process for the Master of Occupational Therapy program:Selection for the MOT program is based on an entrance test conducted by Jamia Hamdard. The test assesses candidates understanding of basic sciences and occupational therapy subjects taught at the BOT level. This ensures that selected candidates have the necessary foundational knowledge to excel in the Masters program.",
            "qas": [
                {
                    "id": "101",
                    "question": "What specializations are offered in the Master of Occupational Therapy program?",
                    "answers": [{"text": "The Master of Occupational Therapy (MOT) program offers specializations in Musculoskeletal and Paediatrics.", "answer_start": 77}]
                },
                {
                    "id": "102",
                    "question": "What is the duration of the Master of Occupational Therapy program?",
                    "answers": [{"text": "The duration of the Master of Occupational Therapy (MOT) program is 2 years.", "answer_start": 132}]
                },
                {
                    "id": "103",
                    "question": "What are the eligibility criteria for the Master of Occupational Therapy program?",
                    "answers": [{"text": "The eligibility criteria include having a Bachelor of Occupational Therapy (BOT) from an institution recognized by Jamia Hamdard, securing at least 50% marks in aggregate. Candidates who obtained their BOT degree from distant mode are not eligible for Post-Graduate courses in Occupational Therapy.", "answer_start": 227}]
                },
                {
                    "id": "104",
                    "question": "How is selection done for the Master of Occupational Therapy program?",
                    "answers": [{"text": "Selection for the MOT program is based on an entrance test conducted by Jamia Hamdard, covering questions on basic sciences and occupational therapy subjects taught at the BOT level.", "answer_start": 451}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 14",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "Notification for Ph.D. Program in Occupational Therapy:The notification for admission to the Ph.D. program in Occupational Therapy for the session 2024-25 is scheduled to be released in July 2024. Prospective candidates interested in pursuing doctoral studies in this field should stay updated with the official announcements from Jamia Hamdard regarding the application process and important deadlines.Eligibility Criteria for Ph.D. Program in Occupational Therapy:Candidates aspiring to enroll in the Ph.D. program in Occupational Therapy must meet certain eligibility criteria. Specifically, they should hold a postgraduate degree in Occupational Therapy with a minimum of 55% marks. This degree should be obtained from either Jamia Hamdard or any other institution recognized by Jamia Hamdard. Fulfilling these criteria is essential for consideration for admission to the doctoral program.",
            "qas": [
                {
                    "id": "105",
                    "question": "When will the notification for admission in the Ph.D. program in Occupational Therapy be released?",
                    "answers": [{"text": "The notification for admission in the Ph.D. program in Occupational Therapy for the session 2024-25 will be released in July 2024.", "answer_start": 0}]
                },
                {
                    "id": "106",
                    "question": "What are the eligibility criteria for the Ph.D. program in Occupational Therapy?",
                    "answers": [{"text": "The eligibility criteria include holding a postgraduate degree in Occupational therapy with 55% marks from Jamia Hamdard or from any institution recognized by Jamia Hamdard.", "answer_start": 188}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 15",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Department of Physiotherapy at Jamia Hamdard, formerly known as the Department of Rehabilitation Sciences, has a rich history dating back to its establishment in 1998. Initially offering a Postgraduate program, it expanded its offerings to include an Undergraduate program in 2001. Since then, it has been a hub for education and research in the field of Physiotherapy, catering to a diverse range of disorders across various specialties. The scope of Physiotherapy encompasses Orthopedics, Neurology, Cardiopulmonary, Sports, Hand Rehabilitation, Community Based Rehabilitation, Obstetrics, Gynecology, Geriatrics, Pediatrics, and Womens health, reflecting the comprehensive nature of the discipline.Physiotherapists, besides their clinical practice, also play a vital role in academic and research activities. They contribute significantly to preventive healthcare initiatives,emphasizing the importance of proactive approaches to maintaining physical well-being. Thus, the Department of Physiotherapy at Jamia Hamdard serves as a cornerstone for both professional development and the advancement of knowledge in the field.",
            "qas": [
                {
                    "id": "107",
                    "question": "When was the Department of Physiotherapy established at Jamia Hamdard?",
                    "answers": [{"text": "The Department of Physiotherapy at Jamia Hamdard, formerly known as the Department of Rehabilitation Sciences, was established in 1998 with a Postgraduate program and expanded to include an Undergraduate program in 2001.", "answer_start": 0}]
                },
                {
                    "id": "108",
                    "question": "What is the scope of Physiotherapy?",
                    "answers": [{"text": "The scope of Physiotherapy covers a wide range of disorders across various specialties, including Orthopedics, Neurology, Cardiopulmonary, Sports, Hand Rehabilitation, Community Based Rehabilitation, Obstetrics, Gynecology, Geriatrics, Pediatrics, and Womens health.", "answer_start": 332}]
                },
                {
                    "id": "109",
                    "question": "What role do Physiotherapists play besides clinical practice?",
                    "answers": [{"text": "Besides clinical practice, Physiotherapists play a vital role in academic and research activities, contributing significantly to preventive healthcare.", "answer_start": 651}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 16",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "Bachelor of Physiotherapy (B.P.T) Seats: 50 (Inclusive of seats for NRI/Sponsored candidates) Duration: Four years plus compulsory internship of six months Eligibility: Passed Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry and Biology from Central Board of Secondary Education or any other examination recognized by Jamia Hamdard as equivalent thereto securing at least 50% marks in the aggregate and also in the aggregate of Physics, Chemistry and Biology. Selection process: On the basis of merit through NEET 2024, among the candidates, who apply for admission to BPT Degree Course through online Jamia Hamdard Application Form Portal. Non-NEET candidates may also apply. Although NEET candidates will be preferred, non-NEET applicants may be considered on the basis of merit of first 5 subjects in qualifying examination. Master of Physiotherapy (M.P.T) Seats: 21 (Inclusive of seats for NRI/Sponsored candidates) Specialization Cardiopulmonary 5 Musculoskeletal 6 Neurology 5 Sports 5 Duration: 2 Years Eligibility: A candidate must have: Bachelor of Physiotherapy (B.P.T.) from an institution recognized by Jamia Hamdard or University/Institute approved by the UGC, securing at least 50% marks in the aggregate. The BPT course should be recognized by Delhi Council of Physiotherapy and Occupational therapy. Candidates who have obtained degree in B.P.T. from distant mode are not eligible for Post Graduate courses in Physiotherapy. Selection process: Selection will be based on the Entrance Test conducted by Jamia Hamdard. The question paper will comprise of questions on basic sciences and physiotherapy subjects taught at BPT level. Ph.D. (Physiotherapy) The notification for admission in Ph.D. programme from session 2024-25 will be released in the month of July 2024. Eligibility: Master of Physiotherapy Programme with at least 55% marks (or equivalent CGPA) in aggregate from any University/Institution recognized by UGC. NET-JRF/Individual Fellowship holders/Project Fellows will be preferred for admissions in Ph.D. Programme.",
            "qas": [
                {
                    "id": "110",
                    "question": "What is the duration of the Bachelor of Physiotherapy (B.P.T) program?",
                    "answers": [{"text": "Four years plus compulsory internship of six months", "answer_start": 63}]
                },
                {
                    "id": "111",
                    "question": "What are the eligibility criteria for the Bachelor of Physiotherapy (B.P.T) program?",
                    "answers": [{"text": "Passed Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry and Biology from Central Board of Secondary Education or any other examination recognized by Jamia Hamdard as equivalent thereto securing at least 50% marks in the aggregate and also in the aggregate of Physics, Chemistry and Biology.", "answer_start": 161}]
                },
                {
                    "id": "112",
                    "question": "How are candidates selected for the Bachelor of Physiotherapy (B.P.T) program?",
                    "answers": [{"text": "On the basis of merit through NEET 2024, among the candidates, who apply for admission to BPT Degree Course through online Jamia Hamdard Application Form Portal. Non-NEET candidates may also apply. Although NEET candidates will be preferred, non-NEET applicants may be considered on the basis of merit of first 5 subjects in qualifying examination.", "answer_start": 291}]
                },
                {
                    "id": "113",
                    "question": "What is the duration of the Master of Physiotherapy (M.P.T) program?",
                    "answers": [{"text": "2 Years", "answer_start": 404}]
                },
                {
                    "id": "114",
                    "question": "What are the eligibility criteria for the Master of Physiotherapy (M.P.T) program?",
                    "answers": [{"text": "A candidate must have: Bachelor of Physiotherapy (B.P.T.) from an institution recognized by Jamia Hamdard or University/Institute approved by the UGC, securing at least 50% marks in the aggregate. The BPT course should be recognized by Delhi Council of Physiotherapy and Occupational therapy. Candidates who have obtained degree in B.P.T. from distant mode are not eligible for Post Graduate courses in Physiotherapy.", "answer_start": 504}]
                },
                {
                    "id": "115",
                    "question": "How are candidates selected for the Master of Physiotherapy (M.P.T) program?",
                    "answers": [{"text": "Selection will be based on the Entrance Test conducted by Jamia Hamdard. The question paper will comprise of questions on basic sciences and physiotherapy subjects taught at BPT level.", "answer_start": 656}]
                },
                {
                    "id": "116",
                    "question": "When will the notification for admission in the Ph.D. (Physiotherapy) program for the session 2024-25 be released?",
                    "answers": [{"text": "in the month of July 2024", "answer_start": 890}]
                },
                {
                    "id": "117",
                    "question": "What are the eligibility criteria for the Ph.D. (Physiotherapy) program?",
                    "answers": [{"text": "Master of Physiotherapy Programme with at least 55% marks (or equivalent CGPA) in aggregate from any University/Institution recognized by UGC. NET-JRF/Individual Fellowship holders/Project Fellows will be preferred for admissions in Ph.D. Programme.", "answer_start": 958}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 17",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "The Department of Paramedical Sciences at Jamia Hamdard was established in 2012 with a primary focus on training and providing skilled paramedical healthcare professionals. The department offers a range of diplomas, degrees, and postgraduate degrees to fulfill its mission.The vision of the Paramedical Department is to provide high-quality skilled students and competent professionals, thereby enhancing medical services and meeting international standards. Notably, 100% of successful graduate and postgraduate candidates are employed immediately after passing from the university, reflecting the departments commitment to producing job-ready professionals.For admission to diploma courses in paramedical sciences, candidates are selected based on the merit of the qualifying examination. Similarly, eligibility criteria for programs such as B.Sc. (Medical Laboratory Techniques) and M.Sc. (Medical Lab Science) require candidates to have passed the Senior Secondary examination with specific subjects and secured minimum aggregate marks.Lateral entry into B.Sc. (Medical Laboratory Techniques) is available for candidates who have completed a two-year diploma course (after 10+2) with a minimum of 50% aggregate marks or a three-year diploma after the 10th examination from a science stream.For Ph.D. (Medical Lab Science), candidates must have completed a Masters degree in Medical Lab Science or a relevant field with at least 55% marks from a recognized university or institution.The notification for admission to the Ph.D. program in Medical Lab Science for the session 2024-25 will be released in the month of July 2024, providing aspiring candidates with an opportunity to pursue advanced research and academic endeavors in the field.",
            "qas": [
                {
                    "id": "118",
                    "question": "When was the Department of Paramedical Sciences established?",
                    "answers": [{"text": "2012", "answer_start": 39}]
                },
                {
                    "id": "119",
                    "question": "What is the main focus of the Department of Paramedical Sciences?",
                    "answers": [{"text": "to train & provide skilled paramedical healthcare professionals", "answer_start": 71}]
                },
                {
                    "id": "120",
                    "question": "What degrees does the Department of Paramedical Sciences offer?",
                    "answers": [{"text": "Diplomas, Degrees, and Postgraduate degrees", "answer_start": 177}]
                },
                {
                    "id": "121",
                    "question": "What is the Vision of the Paramedical Department?",
                    "answers": [{"text": "to provide high quality skilled students and competent professionals to provide better medical services and enhance their employability & to meet the international standard", "answer_start": 413}]
                },
                {
                    "id": "122",
                    "question": "What percentage of successful graduate & postgraduate candidates are employed immediately after passing from the university?",
                    "answers": [{"text": "100%", "answer_start": 667}]
                },
                {
                    "id": "123",
                    "question": "What is the eligibility criteria for Diploma in X-ray & ECG Techniques (DXE)?",
                    "answers": [{"text": "Passed Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry and Biology from Central Board of Secondary Education or any other examination recognized by Jamia Hamdard as equivalent thereto and secured at least 45% marks in the aggregate.", "answer_start": 1867}]
                },
                {
                    "id": "124",
                    "question": "How are candidates selected for the Diploma courses of paramedical sciences?",
                    "answers": [{"text": "Admission to all Diploma courses of paramedical sciences shall be made through merit of the qualifying examination.", "answer_start": 2029}]
                },
                {
                    "id": "125",
                    "question": "What is the eligibility criteria for B.Sc. (Medical Laboratory Techniques)?",
                    "answers": [{"text": "Passed Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry and Biology from Central Board of Secondary Education or any other examination recognized by Jamia Hamdard as equivalent thereto and secured at least 50% marks in the aggregate.", "answer_start": 2655}]
                },
                {
                    "id": "126",
                    "question": "How are candidates selected for B.Sc. (Medical Laboratory Techniques)?",
                    "answers": [{"text": "On the basis of merit of aggregate percentage of the qualifying examination.", "answer_start": 2721}]
                },
                {
                    "id": "127",
                    "question": "What is the eligibility criteria for the lateral entry in B.Sc. (Medical Laboratory Techniques)?",
                    "answers": [{"text": "A candidate must have passed a two-year Diploma in respective course (after 10+2) with 50% marks in the aggregate or 3 years Diploma after 10th examination from science stream, except for DXE where the student is eligible to apply for BMIT.", "answer_start": 3186}]
                },
                {
                    "id": "128",
                    "question": "What is the eligibility criteria for M.Sc. (Medical Lab Science)?",
                    "answers": [{"text": "Passed B.Sc. Medical Lab Technology with at least 55% marks.", "answer_start": 4021}]
                },
                {
                    "id": "129",
                    "question": "How are candidates selected for M.Sc. (Medical Lab Science)?",
                    "answers": [{"text": "On the basis of merit of qualifying examination.", "answer_start": 4071}]
                },
                {
                    "id": "130",
                    "question": "When will the notification for admission in Ph.D. (Medical Lab Science) be released for the session 2024-25?",
                    "answers": [{"text": "in the month of July 2024", "answer_start": 4482}]
                },
                {
                    "id": "131",
                    "question": "What is the eligibility criteria for Ph.D. (Medical Lab Science)?",
                    "answers": [{"text": "Candidate must have completed Masters degree in Medical Lab Science / Relevant Degree in the Field with 55% of marks from any University/ Institution recognized by UGC or its equivalent Grade B in the UGC 7-point scale.", "answer_start": 4527}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 18",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Department of Computer Science & Engineering (CSE) at the School of Engineering Sciences & Technology, established in 1997, stands as a pioneering institution in the realm of computer education. It offers a diverse range of Under Graduate (UG) and Post Graduate (PG) Programmes catering to the dynamic landscape of technology and innovation. The academic spectrum includes programs in Computer Science & Engineering, Computer Application, Information Technology, Electronics & Communication Engineering, Data Science, Cyber Forensics & Information Security. Moreover, the department is a hub for cutting-edge research, with Ph.D. programs delving into various thrust areas of computer science and engineering. These include Big Data, Soft Computing, IoT, Artificial Intelligence, Data Mining, Information Security, and much more. Infrastructure The department boasts state-of-the-art infrastructure, featuring smart cluster rooms and well-equipped computer labs furnished with the latest technology. The computer labs house advanced PCs with i5 processors, ample RAM, and spacious storage, running on both Ubuntu and Windows operating systems. Additionally, theres a dedicated Research Lab for Ph.D. students, equipped with modern amenities like air conditioning and CCTV surveillance. A spacious Conference Room facilitates departmental meetings and student seminars. In the Electronics & Communication Engineering (ECE) domain, the department offers a range of labs covering Analog & Digital Electronics, Embedded Systems, FPGA Development, and more. These labs are equipped with the latest software and advanced instruments, providing students with hands-on experience in cutting-edge technologies. Internship and Training Opportunities Students at the Department of CSE benefit from numerous internship and training opportunities facilitated through strategic partnerships with industry leaders. MoUs signed with organizations like AICTEs Internshala Scheme, Co-Cubes, and RnF enable students to gain practical exposure and industry insights. Furthermore, collaborations with ETrain India Pvt. Ltd. ensure the conduct of certified seminars and workshops by Microsoft and Adobe. Industry Recruiters The department takes pride in its robust placement record, with students securing positions in renowned industry giants such as TCS, Wipro, HCL, Fiserv, RnF, and Ericsson. These partnerships not only provide valuable career opportunities but also validate the departments commitment to producing skilled professionals poised to make significant contributions to the technology landscape.",
            "qas": [
                {
                    "id": "132",
                    "question": "When did the Department of Computer Science & Engineering start at the School of Engineering Sciences & Technology?",
                    "answers": [{"text": "The Department of Computer Science & Engineering in the School of Engineering Sciences & Technology started in 1997.", "answer_start": 96}]
                },
                {
                    "id": "133",
                    "question": "What programs does the Department of Computer Science & Engineering offer?",
                    "answers": [{"text": "The Department not only offers Under Graduate (UG) and Post Graduate (PG) Programmes in Computer Science & Engineering/Computer Application/IT/ECE/Data Science/Cyber Forensics & Information Security but is also engaged in high-quality research activities leading to Ph.D. degree in different thrust areas of computer science and engineering and interdisciplinary areas viz. Big Data, Soft Computing, IoT, Neural Network, Artificial Intelligence, Fuzzy Logic, Software Engineering, Data Mining, Information Security, Cyber laws, Cyber Forensics, Computer Architecture, Graph Theory, Computer Networks, Distributed System, Discrete Structure, Robotics, Machine Intelligence, Telecommunication, Mobile Computing, DBMS, Algorithms, VLSI Design etc.", "answer_start": 299}]
                },
                {
                    "id": "134",
                    "question": "What infrastructure facilities does the department provide?",
                    "answers": [{"text": "The department has several smart cluster rooms fully equipped with state of art of technology. It also has 2 computer labs with the latest PCs having an i5 processor with 8 GB RAM and 1 TB storage. The computer lab is fully equipped with the latest Open-Source Software as per our curriculum requirement running on Ubuntu OS. Several PCs in Computer Lab at 1st floor runs on Windows OS having required software for academics. The Department also has a Research Lab (2nd Floor, Department of CSE) for our Ph.D. students. These labs are fully equipped with Air Conditioning and CCTV cameras have been installed too. The Department has 1 Conference Room with a seating capacity of more than 100 for conducting departmental meetings and student seminars. In ECE domain, we have basic and advanced Electronics & Communication Labs according to the required curriculum. These labs include Analog & Digital Labs, Embedded Systems Lab, FPGA Development kits, DSP Lab, Satellite & Microwave Lab, Digital Design Lab etc. The ECE labs have software as per the academic curriculum and these labs are equipped with updated and advanced instruments. The department also has high-end computing and Artificial Intelligence Lab.", "answer_start": 400}]
                },
                {
                    "id": "135",
                    "question": "What opportunities are provided to students regarding internships and trainings?",
                    "answers": [{"text": "The students are provided internships and trainings via the MoUs signed with several setups like the AICTEs Internshala Scheme, Co-Cubes and RnF (for Campus placements), and ETrain India Pvt. Ltd for conducting seminars/workshops which are Microsoft/Adobe certified.", "answer_start": 819}]
                },
                {
                    "id": "136",
                    "question": "Which big giants in the industry recruit students from the Department of Computer Science & Engineering?",
                    "answers": [{"text": "The campus placements (On-site or Off-site) have big giants in the industry like TCS, Wipro, HCL, Fiserv, RnF, and Ericcson to name a few which recruit students and they prove to be assets to the industry.", "answer_start": 922}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 19",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "The School of Engineering Sciences & Technology at Jamia Hamdard offers a diverse range of undergraduate programs designed to cater to the dynamic needs of the industry and academia. Here are some key details regarding the admission criteria and selection processes for various undergraduate programs: B.Tech. Computer Science & Engineering Program Duration: The Bachelor of Technology (B.Tech.) program in Computer Science & Engineering spans over a duration of 4 years, providing students with comprehensive knowledge and practical skills in this rapidly evolving field. Eligibility Criteria for B.Tech. Electronics and Communication Engineering: Candidates aspiring to join the B.Tech. Electronics and Communication Engineering program must have passed the Senior Secondary (12th Standard/Intermediate) examination with Mathematics and Physics as compulsory subjects, along with another subject from a specified list. A minimum aggregate of 50% marks or equivalent CGPA is required in the qualifying examination. Selection Process for Bachelor of Computer Application (BCA) Program: The selection process for admission to the Bachelor of Computer Application (BCA) program is based on the percentage of marks obtained in the qualifying examination. Selection Criteria for Bachelor of Science in Computer Science with Research Program: Admission to the Bachelor of Science (B.Sc.) program in Computer Science with Research is determined based on the merit of the Central University Entrance Test (CUET)-2024. The CUET subjects considered for evaluation include Mathematics, Computer Science, and Informatics Practices. In case of insufficient CUET-2024 qualified candidates, selection may be offered based on the merit of the qualifying examination. Eligibility Requirement for Lateral Entry to 2nd Year in B.Tech. (ECE): Candidates seeking lateral entry to the second year of the B.Tech. program in Electronics and Communication Engineering (ECE) must hold a Diploma in Engineering from a recognized institution/university, with a minimum aggregate of 50% marks or equivalent CGPA.",
            "qas": [
                {
                    "id": "137",
                    "question": "What is the duration of the B.Tech. Computer Science & Engineering program?",
                    "answers": [{"text": "Duration: 4 Years", "answer_start": 330}]
                },
                {
                    "id": "138",
                    "question": "What is the eligibility criteria for admission to the B.Tech. Electronics and Communication Engineering program?",
                    "answers": [{"text": "Eligibility: A candidate seeking admission to this program must have passed Senior Secondary (12th Standard/Intermediate) examination with at least 5 subjects (all passed) having Mathematics and Physics (both compulsory) subjects, and one subject out of the following: Chemistry/Computer Science/Electronics/Information Technology/Biology/Informatics Practices/Biotechnology/Technical Vocational subject/Agriculture/Engineering Graphics/Business Studies/Entrepreneurship from CBSE or any other Board recognized by Jamia Hamdard as equivalent thereto, securing at least 50% marks or equivalent CGPA in aggregate.", "answer_start": 893}]
                },
                {
                    "id": "139",
                    "question": "What is the selection process for admission to the Bachelor of Computer Application (BCA) program?",
                    "answers": [{"text": "Selection Process: On the basis of percentage of qualifying examination.", "answer_start": 1971}]
                },
                {
                    "id": "140",
                    "question": "How are students selected for the Bachelor of Science in Computer Science with Research program?",
                    "answers": [{"text": "Selection Process: On the basis of merit of CUET-2024. CUET Subjects: Mathematics / Computer Science/ Informatics Practices In the event of non-availability of CUET-2024 qualified candidates, selection may be offered on the basis of merit of qualifying examination.", "answer_start": 2637}]
                },
                {
                    "id": "141",
                    "question": "What is the eligibility requirement for lateral entry to 2nd year in B.Tech. (ECE)?",
                    "answers": [{"text": "Eligibility: A candidate seeking admission to B.Tech. (ECE) lateral entry must have passed Diploma Engineering in any branch from a recognized institution/university securing at least 50% marks or equivalent CGPA in aggregate.", "answer_start": 3337}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 20",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "Master of Computer Applications (MCA) Seats: 60, inclusive of seats reserved for NRI/sponsored candidates; additional seats are available for Foreign Nationals. Duration: 2 Years Eligibility:  Passed  B.E./B.Tech./BCA/B.Sc./B.Com./B.A.with  Mathematics  at  10+2  level  or  at Graduation Level and Obtained at least 50% marks in aggregate in the qualifying Examination. Selection process: Merit of the qualifying examination. M.Sc. (Computational Biology & System Biology and Bioinformatics)Seats: 10 candidates; inclusive of seats reserved for NRI/sponsored candidates; additional seats are available for Foreign Nationals.Duration: 2 years (4 Semesters)Eligibility:  A  minimum  of  55%  marks  in  Bachelors  degree  program  in  bioinformatics,  or  any branch  of  basic/applied  Sciences  or  Technology,  including  Life  Sciences,  Medicine,  Pharmacy, Engineering and Computer Science and allied disciplines.Selection process: Merit of the qualifying examination.Regular M.Tech. CoursesSeats: 30 (28 seats General and 02 seats reserved for Industry/NRI sponsored candidates)M.Tech. (Computer Science & Engineering) - M.Tech. (CSE)Duration: 2 years (4 Semesters)Eligibility:  Passed  B.Tech./B.E.  or  equivalent  degree  in  Computer  Science/Computer  Science  & Engineering/Computer Engineering/Information Technology/Software Engineering/ICT with at least 55% marks (or equivalent CGPA) in aggregate.(OR)MCA or M.Sc. in IT/Computer Science/Information Science & Technology/Electronics/Software Engineering or equivalent degree with at least 55% marks (or equivalent CGPA) in aggregate.(OR)B.Tech./B.E.  or in  Electronics  &  Communication/Electronics Engineering/Electrical Engineering with at least 55% marks (or equivalent CGPA) in aggregate.Selection process: Merit of the qualifying examination.M.Tech. (CSE) with Specialization in Data ScienceDuration: 2 years (4 Semesters)Eligibility: Passed  B.Tech./B.E.  or  equivalent  degree  in  Computer  Science/Computer  Science  and Engineering /Computer Engineering/Information Technology/Software Engineering/ICT with at least 55% marks (or equivalent CGPA) in aggregate.(OR)Passed  B.Tech./B.E.  or  equivalent  degree  in  Electronics  &  Communication/Electronics Engineering/Electrical Engineering with at least 55% marks (or equivalent CGPA) in aggregate.(OR)Passed MCA or M.Sc. (CS), M.Sc. (IT), M.Sc. (Software Engineering), M.Sc. (Electronics) with at least 55% marks (or equivalent CGPA) in aggregate.Selection process: Merit of the qualifying examination.M. Tech. (CSE) with Specialization in Cyber Forensics & Information SecurityDuration: 2 years (4 Semesters)Eligibility: Passed  B.Tech./B.E.  or  equivalent  degree  in  Computer  Science/Computer  Science  & Engineering/Computer Engineering/Information Technology/Software Engineering/ICT with at least 55% marks (or equivalent CGPA) in aggregate.(OR)MCA or  in  M.Sc. Science/Information IT/Computer  Science Technology/Electronics/Software Engineering or equivalent degree with at least 55% marks (or equivalent CGPA) in aggregate.(OR)B.Tech./B.E.  or  in  Electronics  &  Communication/Electronics Engineering/Electrical Engineering with at least 55% marks (or equivalent CGPA) in aggregate.Selection process: Merit of the qualifying examination.",
            "qas": [
                {
                    "id": "142",
                    "question": "What is the duration of the MCA program?",
                    "answers": [{"text": "Duration: 2 Years", "answer_start": 278}]
                },
                {
                    "id": "143",
                    "question": "What is the eligibility requirement for the M.Tech. (CSE) program?",
                    "answers": [{"text": "Eligibility:  Passed  B.Tech./B.E.  or  equivalent  degree  in  Computer  Science/Computer  Science  & Engineering/Computer Engineering/Information Technology/Software Engineering/ICT with at least 55% marks (or equivalent CGPA) in aggregate.", "answer_start": 974}]
                },
                {
                    "id": "144",
                    "question": "What is the eligibility criterion for the M.Tech. (CSE) program with specialization in Data Science?",
                    "answers": [{"text": "Eligibility: Passed  B.Tech./B.E.  or  equivalent  degree  in  Computer  Science/Computer  Science  and Engineering /Computer Engineering/Information Technology/Software Engineering/ICT with at least 55% marks (or equivalent CGPA) in aggregate.", "answer_start": 1596}]
                },
                {
                    "id": "145",
                    "question": "What is the duration of the M.Tech. (CSE) program with specialization in Cyber Forensics & Information Security?",
                    "answers": [{"text": "Duration: 2 years (4 Semesters)", "answer_start": 2302}]
                },
                {
                    "id": "146",
                    "question": "What is the eligibility requirement for admission to the Ph.D. (Computer Science & Engineering) program?",
                    "answers": [{"text": "Eligibility:  M.Tech. /M.E. in Computer Science/IT /Computer Engineering (or in allied field), with at least 60% marks (or equivalent CGPA) in aggregate from any University/Institution recognized by the Association of Indian Universities (AIU)/UGC/AICTE.", "answer_start": 3205}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 21",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "Management education was introduced in Jamia Hamdard in the year 1998. Since then, it has become one of the important units of the University. The School of Management & Business Studies (SMBS) has three departments namely,  Department  of  Management,  Department  of  Healthcare  & Pharmaceutical Management and Department of Hotel Management and Catering Technology. Today more than 1200 students are enrolled in the school for various PG and UG courses. Owing to the core competencies of the faculty, the school has made a name for itself, both among industry erudite and intelligentsia. The teaching pedagogy employed in SMBS is a combination of theoretical concept with practical application. The school is well endowed with intellectual capital and field experience with both senior and  young  faculty  members  who  have  plenty  of  research  experience  behind  them  and  extensive industry experience in the domain of Management specializations like Marketing, Finance, Human Resources,  Operations  Management,  International  Business  and  Entrepreneurship,  Public  Health, Healthcare  Management,  Pharmaceutical  Management,  Hotel  management,  Commerce,  Supply Chain Management etc.  Placement/Internships activities are  coordinated  by  a  placement  cell  headed  by,  Training  and Placement Officer, comprising of selected student representatives and faculty members. This includes both summer internship as well as final placement.",
            "qas": [
                {
                    "id": "147",
                    "question": "When was management education introduced in Jamia Hamdard?",
                    "answers": [{"text": "1998", "answer_start": 33}]
                },
                {
                    "id": "148",
                    "question": "What are the three departments under the School of Management & Business Studies (SMBS)?",
                    "answers": [{"text": "Department  of  Management,  Department  of  Healthcare  & Pharmaceutical Management and Department of Hotel Management and Catering Technology", "answer_start": 213}]
                },
                {
                    "id": "149",
                    "question": "How many students are enrolled in the School of Management & Business Studies (SMBS)?",
                    "answers": [{"text": "more than 1200", "answer_start": 380}]
                },
                {
                    "id": "150",
                    "question": "What teaching pedagogy is employed in SMBS?",
                    "answers": [{"text": "combination of theoretical concept with practical application", "answer_start": 432}]
                },
                {
                    "id": "151",
                    "question": "What types of specializations are offered in the Department of Management?",
                    "answers": [{"text": "Marketing, Finance, Human Resources,  Operations  Management,  International  Business  and  Entrepreneurship", "answer_start": 545}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 22",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "BBA (Bachelor of Business Administration)Seats: 180Duration: 4 years/8 semestersEligibility: 10+2 from any recognized board with 50% aggregate marks.Selection Process: Candidate must appear in the CUET in the General test. If seats are vacant, the admission may be done on the basis of merit of the qualifying examination.Bachelor of Commerce (B.Com)Seats: 60Duration: 4 years/8 semestersEligibility: 10+2 from any recognized board with 50% aggregate marks.Selection  Process:  Candidate  must  appear  in  the  CUET  in  the  General  test.  If  seats  are  vacant,  the admission may be done on the basis of the qualifying examination.Master of Business Administration (MBA)The Department of Management offers MBA degree with various specialization in Management viz., Marketing,  Finance,  Human  Resource,  Operations  Management,  Information  Technology, International Business and Entrepreneurship.Seats: 120Duration: 2 years/4 semestersEligibility:i.  Bachelors degree in any discipline with minimum 50% aggregate marks/CGPA 5 on a 10-point scale. Candidates in their final year of graduation are also eligible to apply. Their final admission is subject to the availability of final mark sheet/provisional degree. Candidates from final year Graduate Programme need to  submit the aggregate marks up to  4th  semester (for 3-Year UG Programme) and should be minimum of 50% or CGPA 5 on a 10-Point Scale. * ii.  Candidates  who have appeared in any national level aptitude test like CAT/CMAT/MAT/GMAT/XAT/GPAT/ATMA and possess a valid score card. * Admission shall be cancelled if in any case candidates do not meet the eligibility criteria. Exemptions from Aptitude Test\u2022  Mid-career professionals with 2 years of work experience (after graduation).  \u2022  Candidates who have  completed undergraduate  programme of four-year or above duration (Example:  B.E./B.Tech./MBBS/B.A.  LLB,  etc.)  with  a  minimum  of  50%  aggregate marks/CGPA  of  5  on  a  10-point  scale.  Candidates  in  their  final  year  of  under  graduate programme with four-year or above duration (Example: B.E./B.Tech./MBBS/B.A. LLB, etc.) need to submit the aggregate marks up to 6th semester. * \u2022  Candidates who have obtained first division in at least two examinations out of three i.e. 10th, 12th & Undergraduate with second division in the third examination. However, no candidates will be called for GD & PI if they failed to secure minimum 50% or 5 CGPA on a 10-Point Scale in their Graduation.  Selection  Process:  On  the  basis  of  eligibility  criteria/exemptions  mentioned  above,  eligible candidates  will  be  called  for  GD/PI  at  Jamia  Hamdard  campus/online  mode  as  per  the  prevailing condition during the time of GD/PI. For  NRI/Sponsored  Seats:  CMAT/MAT/GMAT/XAT/CAT/ATMA  scores  are  not  necessary; admission will be determined solely on the basis of their performance in GD/PI.Master of Business Administration-Executive (MBA Executive)The Department of Management offers 2 years MBA Executive program for working professionals with various specialisation in Management viz., Marketing, Finance, Human Resource, Operations Management,  Information  Technology,  International  Business,  and  Entrepreneurship.  It  will  be offered in hybrid (online+offline) mode and classes will be conducted in evenings and on weekends.  Seats: 30Duration: 2 years/4 semesters.Eligibility: Educational  Qualification:  Bachelors  degree  in  any  discipline  with  minimum  50%  aggregate marks/CGPA 5 on a 10- point scale. The student should hold a degree from any of the universities incorporated  by  an  act  of  the  central  or  state  legislature  in  India,  or  other  educational  institutions established by an act of Parliament or declared in India, or other educational institutions established by an act of Parliament or declared to be deemed as a university under Section 3 of UGC Act, 1956, or possess an equivalent qualification recognized by the Ministry of Education, Government of India. Experience: Applicant must have three years of professional experience at Executive/ Managerial Level after graduation. Note: The last date of application shall be treated as the cut-off date in determining the experience of the candidate. Selection Process: The applicants will be shortlisted based on academic performance and/or work experience. Shortlisted candidates will be called for Group Discussion/Interview at Jamia Hamdard campus. Ph.D. (Management)The notification for admission in Ph.D. programme session 2024-25 will be released in the month of July 2024.Duration: 3 years minimumEligibility: Masters Degree (preferably MBA/PGDM) or a Professional degree declared equivalent to Masters degree in allied subjects with at least 55%aggregate marks (or equivalent CGPA) from any University/Institution recognized by the Association of Indian Universities (AIU/UGC/AICTE)/ corresponding statutory body. NET-JRF/Individual  Fellowship holders/Project Fellows will be preferred for admissions in Ph.D. programsPh.D. (Management) Part timeThe notification for admission in Ph.D. programme session 2024-25 will be released in the month of July, 2024.The part time Ph.D. Programme is a professional doctoral degree, designed to enhance executive and professional  practice  through  the  application  of  sound  theory  and  rigorous  research  into  real  and complex  issues  in  business  and  management.  Key  to  this  is  a  programme  to  hone  personal, consultancy and research skills amongst working professionals at different management levels. Corporate executives working with reputed organizations in India applying for admission to the part time Ph.D. Program shall be eligible as per the following conditions: Duration: 4 years minimum. The overall duration will be as per UGC Regulations for Ph.D. Eligibility: Masters Degree (preferably MBA/PGDM) or a Professional degree declared equivalent to  Masters  degree  with  at  least  55%  aggregate  marks  (or  equivalent  CGPA)  from  any  University/Institution  recognized  by  the  Association  of  Indian  Universities  (AIU/UGC/AICTE)/ corresponding statutory body. NET-JRF/Individual  Fellowship holders/Project Fellows will be preferred for admissions in Ph.D. programs Selection Process for Ph.D. (Management) and Part time Ph.D. (Management) Programmes: The admission process would be a two-tier process   \u2022  First stage would include qualifying Ph.D. entrance test, conducted by Jamia Hamdard \u2022  NET/JRF qualified candidates are  exempted from  the Ph.D. entrance test  and will directly qualify for second stage. \u2022  Second stage will include an interview organized by School  of Management  and Business Studies,  Jamia  Hamdard,  where  the  candidate  is  required  to  discuss  his/her  research interest/area  through  a  presentation  before  a  duly  constituted  Department  Research  Committee. The interview shall also consider the following aspects, viz. whether:  ●  The candidate possesses the competence for the proposed research; ●  The research work can be suitably undertaken at the Institution ●  The proposed area of research can contribute to new/additional knowledge.",
            "qas": [
                {
                    "id": "152",
                    "question": "What is the eligibility criteria for BBA (Bachelor of Business Administration) program?",
                    "answers": [{"text": "10+2 from any recognized board with 50% aggregate marks", "answer_start": 178}]
                },
                {
                    "id": "153",
                    "question": "How are candidates selected for admission to the BBA program?",
                    "answers": [{"text": "CUET in the General test. If seats are vacant, the admission may be done on the basis of merit of the qualifying examination", "answer_start": 269}]
                },
                {
                    "id": "154",
                    "question": "What is the eligibility criteria for the Master of Business Administration (MBA) program?",
                    "answers": [{"text": "Bachelors degree in any discipline with minimum 50% aggregate marks/CGPA 5 on a 10-point scale. Candidates in their final year of graduation are also eligible to apply", "answer_start": 732}]
                },
                {
                    "id": "155",
                    "question": "How are candidates exempted from aptitude tests for the MBA program?",
                    "answers": [{"text": "Mid-career professionals with 2 years of work experience (after graduation).Candidates who have  completed undergraduate  programme of four-year or above duration (Example:  B.E./B.Tech./MBBS/B.A.  LLB,  etc.)  with  a  minimum  of  50%  aggregate marks/CGPA  of  5  on  a  10-point  scale.  Candidates  in  their  final  year  of  under  graduate programme with four-year or above duration (Example: B.E./B.Tech./MBBS/B.A. LLB, etc.) need to submit the aggregate marks up to 6th semester.Candidates who have obtained first division in at least two examinations out of three i.e. 10th, 12th & Undergraduate with second division in the third examination", "answer_start": 922}]
                },
                {
                    "id": "156",
                    "question": "What is the eligibility criteria for the Master of Business Administration-Executive (MBA Executive) program?",
                    "answers": [{"text": "Bachelors  degree  in  any  discipline  with  minimum  50%  aggregate marks/CGPA 5 on a 10- point scale. The student should hold a degree from any of the universities  incorporated  by  an  act  of  the  central  or  state  legislature  in  India,  or  other  educational  institutions established by an act of Parliament or declared in India, or other educational institutions established by an act of Parliament or declared to be deemed as a university under Section 3 of UGC Act, 1956, or possess an equivalent qualification recognized by the Ministry of Education, Government of India", "answer_start": 1286}]
                },
                {
                    "id": "157",
                    "question": "What experience is required for applicants to the MBA Executive program?",
                    "answers": [{"text": "Applicant must have three years of professional experience at Executive/ Managerial Level after graduation", "answer_start": 1361}]
                },
                {
                    "id": "158",
                    "question": "What is the selection process for the MBA Executive program?",
                    "answers": [{"text": "The applicants will be shortlisted based on academic performance and/or work experience. Shortlisted candidates will be called for Group Discussion/Interview at Jamia Hamdard campus", "answer_start": 1407}]
                },
                {
                    "id": "159",
                    "question": "What is the eligibility criteria for the Ph.D. (Management) program?",
                    "answers": [{"text": "Masters Degree (preferably MBA/PGDM) or a Professional degree declared equivalent to Masters degree in allied subjects with at least 55%aggregate marks (or equivalent CGPA) from any University/Institution recognized by the Association of Indian Universities (AIU/UGC/AICTE)/ corresponding statutory body", "answer_start": 1714}]
                },
                {
                    "id": "160",
                    "question": "What is the eligibility criteria for the part time Ph.D. (Management) program?",
                    "answers": [{"text": "Masters Degree (preferably MBA/PGDM) or a Professional degree declared equivalent to  Masters  degree  with  at  least  55%  aggregate  marks  (or  equivalent  CGPA)  from  any  University/Institution  recognized  by  the  Association  of  Indian  Universities  (AIU/UGC/AICTE)/ corresponding statutory body", "answer_start": 1853}]
                },
                {
                    "id": "161",
                    "question": "What is the selection process for the Ph.D. (Management) and part time Ph.D. (Management) programs?",
                    "answers": [{"text": "The admission process would be a two-tier process   \u2022  First stage would include qualifying Ph.D. entrance test, conducted by Jamia Hamdard \u2022  NET/JRF qualified candidates are  exempted from  the Ph.D. entrance test  and will directly qualify for second stage. \u2022  Second stage will include an interview organized by School  of Management  and Business Studies,  Jamia  Hamdard,  where  the  candidate  is  required  to  discuss  his/her  research interest/area  through  a  presentation  before  a  duly  constituted  Department  Research  Committee", "answer_start": 1957}]
                }
            ]
        }
    ]
},
{
    "title": "Data item 23",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "The School of Management and Business Studies at Jamia Hamdard offers several programs catering to the dynamic needs of the healthcare and pharmaceutical management sectors. For undergraduates interested in healthcare management, the Bachelor of Management Studies (BMS) - Healthcare Management program provides a solid foundation. Eligibility requires passing the 12th class board exams with a minimum of 50% marks, and selection is based on the merit of the qualifying exam. Moving on to postgraduate studies, the MBA program offers specializations in Healthcare and Hospital Management as well as Pharmaceutical Management. A bachelors degree in any discipline with a minimum of 50% aggregate marks is necessary for eligibility, and candidates in their final year of graduation are also eligible. Selection involves a comprehensive process including group discussions, personal interviews, and evaluation of academic performance. For those aspiring for doctoral studies, the Ph.D. programs in Health Management and Pharmaceutical Management are highly regarded. Eligibility criteria include holding an MBA, MHA, or M.Pharm. degree with at least 55% marks. The admission process comprises a two-tier system involving a qualifying Ph.D. entrance test followed by an interview where candidates present their research interests before a departmental research committee. Additionally, for working professionals seeking to pursue doctoral studies part-time, similar eligibility criteria apply. The admission process remains the same, emphasizing both academic excellence and research potential. Overall, these programs reflect Jamia Hamdards commitment to excellence in healthcare and pharmaceutical management education, providing students with the knowledge and skills needed to thrive in these critical sectors.",
            "qas": [
                {
                    "id": "162",
                    "question": "What is the eligibility criteria for the Bachelor of Management Studies (BMS) - Healthcare Management program?",
                    "answers": [{"text": "The student must have passed intermediate/12th class board exams from CBSE/ICSE or any other recognized central or state board with a minimum of 50%.", "answer_start": 905}]
                },
                {
                    "id": "163",
                    "question": "How are candidates selected for admission to the Bachelor of Management Studies (BMS) - Healthcare Management program?",
                    "answers": [{"text": "Selection will be made on the basis of the merit of Qualifying Exam", "answer_start": 1003}]
                },
                {
                    "id": "164",
                    "question": "What is the eligibility criteria for the MBA (Healthcare and Hospital Management) program?",
                    "answers": [{"text": "Bachelors degree in discipline* with minimum 50% aggregate marks or core CGPA 5 on a 10 point scale. Candidates in their final year of graduation are also eligible to apply.", "answer_start": 1141}]
                },
                {
                    "id": "165",
                    "question": "How are candidates selected for admission to the MBA (Healthcare and Hospital Management) program?",
                    "answers": [{"text": "Shortlisted candidates based on the eligibility will be called for Group Discussion (GD) and Personal Interview (Pl) to be conducted at Jamia Hamdard campus. Selection will be made on the basis of overall merit determined by the University on the basis of performance in GD/PI and marks in the Graduation and Industry Work Experience.", "answer_start": 1278}]
                },
                {
                    "id": "166",
                    "question": "What is the eligibility criteria for the MBA (Pharmaceutical Management) program?",
                    "answers": [{"text": "Bachelors degree in Pharmacy/discipline* with minimum 50% aggregate marks or core CGPA 5 on a 10 point scale. Candidates in their final year of graduation are also eligible to apply.", "answer_start": 1945}]
                },
                {
                    "id": "167",
                    "question": "How are candidates selected for admission to the MBA (Pharmaceutical Management) program?",
                    "answers": [{"text": "Shortlisted candidates based on the eligibility will be called for Group Discussion (GD) and Personal Interview (Pl) to be conducted at Jamia Hamdard campus. Selection will be made on the basis of overall merit determined by the University on the basis of performance in GD/PI and marks in the Graduation and Industry Work Experience.", "answer_start": 2082}]
                },
                {
                    "id": "168",
                    "question": "What is the eligibility criteria for the Ph.D. (Health Management and Pharmaceutical Management) program?",
                    "answers": [{"text": "MBA/MHA/M. Pharm. and other Masters degree in the subject concerned with at least 55% marks; or Masters degree in allied subjects with at least 55% marks (or Equivalent CGPA) in aggregate from any University/Institution recognized by the Association of Indian Universities (AIU)/UGC/AICTE", "answer_start": 2487}]
                },
                {
                    "id": "169",
                    "question": "How are candidates selected for admission to the Ph.D. (Health Management and Pharmaceutical Management) program?",
                    "answers": [{"text": "The admission process would be a two-tier process \u2022  First stage would include qualifying Ph.D. entrance test, conducted by Jamia Hamdard. 2022  NET/JRF qualified candidates are  exempted from  the Ph.D. entrance test  and will directly qualify for second stage.\u2022  Second stage will include an interview organized by School  of Management  and Business Studies,  Jamia Hamdard,  where  the  candidate  is  required  to  discuss  his/her  research interest/area  through  a  presentation  before  a  duly constituted Department Research Committee", "answer_start": 2643}]
                },
                {
                    "id": "170",
                    "question": "What is the eligibility criteria for the part-time Ph.D. (Health Management and Pharmaceutical Management) program?",
                    "answers": [{"text": "MBA/MHA/M. Pharm. and other Masters degree in the subject concerned with at least 55% marks; or Masters degree in allied subjects with at least 55% marks (or Equivalent CGPA) in aggregate from any University/Institution recognized by the Association of Indian Universities (AIU)/UGC/AICTE", "answer_start": 2886}]
                },
                {
                    "id": "171",
                    "question": "How are candidates selected for admission to the part-time Ph.D. (Health Management and Pharmaceutical Management) program?",
                    "answers": [{"text": "The admission process would be a two-tier process \u2022  First stage would include qualifying Ph.D. entrance test, conducted by Jamia Hamdard.\u2022  NET/JRF qualified candidates are  exempted from  the Ph.D. entrance test  and will directly qualify for second stage.\u2022  Second stage will include an interview organized by School  of Management  and Business Studies,  Jamia  Hamdard,  where  the  candidate  is  required  to  discuss  his/her  research interest/area  through  a  presentation  before  a  duly  constituted  Department  Research Committee", "answer_start": 3032}]
                }
            ]
}
    ]
}, {
    "title": "Data item 24",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The School of Chemical and Life Sciences at Jamia Hamdard has been a cornerstone of academic excellence since its inception. Teaching activities commenced in the academic year 1994-95, with a strong focus on postgraduate programs, marking the beginning of a journey towards advanced education and cutting-edge research in the fields of chemistry and life sciences. Comprising five departments and a Center for Translational and Clinical Research, the School offers a diverse range of academic avenues for students and researchers alike. These departments serve as hubs of innovation and exploration, fostering interdisciplinary collaboration and pushing the boundaries of scientific knowledge. In addition to its robust academic programs, the School of Chemical and Life Sciences boasts state-of-the-art research facilities designed to support groundbreaking research endeavors. The Central Instrumentation Facility (CIF) provides researchers with access to advanced equipment and instrumentation, empowering them to conduct high-impact experiments and analyses. Furthermore, the Center for Transgenic Plant Development stands as a testament to the Schools commitment to pioneering research in biotechnology and genetic engineering. Through its unwavering dedication to excellence in education and research, the School of Chemical and Life Sciences continues to inspire the next generation of scientists and contribute to the advancement of knowledge in chemical and life sciences disciplines.",
            "qas": [
                {
                    "id": "172",
                    "question": "When did the teaching activities at the School of Chemical and Life Sciences begin?",
                    "answers": [{"text": "The teaching activities at the School of Chemical and Life Sciences began in the academic year 1994-95 with postgraduate programs.", "answer_start": 153}]
                },
                {
                    "id": "173",
                    "question": "How many departments are there in the School of Chemical and Life Sciences?",
                    "answers": [{"text": "The School comprises five departments and a Center for Translational and Clinical Research.", "answer_start": 114}]
                },
                {
                    "id": "174",
                    "question": "What kind of research facilities are available in the School of Chemical and Life Sciences?",
                    "answers": [{"text": "The School houses specialized research facilities like the Central Instrumentation Facility (CIF) and the Center for Transgenic Plant Development.", "answer_start": 459}]
                }
            ]
}
    ]
},
{
    "title": "Data item 25",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Department of Biotechnology at Jamia Hamdard was established in 1997 with a vision to be recognized internationally for its strong interdisciplinary research and teaching base in Plant and Animal Biotechnology, fostering collaboration with industries and healthcare institutions. Supported by FIST (DST, Govt. of India) and SAP (UGC), the department offers a range of programs including 4-Year B.Sc. (Hons.) Biotechnology, 2-Year M.Sc. Biotechnology, 2-Year M.Tech. Biotechnology, and doctoral research leading to a Ph.D. degree in Biotechnology. Faculty members have secured numerous extramurally funded research projects from agencies like DBT, DST, CSIR, and others. Students in the M.Sc. program engage in intensive project work mentored by department faculty, covering literature survey, benchwork, data generation, thesis writing, and presentation skills.",
            "qas": [
                {
                    "id": "175",
                    "question": "When was the Department of Biotechnology established at Jamia Hamdard?",
                    "answers": [{"text": "The Department of Biotechnology was established in the year 1997.", "answer_start": 4}]
                },
                {
                    "id": "176",
                    "question": "What programs does the Department of Biotechnology offer?",
                    "answers": [{"text": "The department offers 4-Year B.Sc. (Hons.) Biotechnology, 2-Year M.Sc. Biotechnology, 2-Year M.Tech. Biotechnology, and doctoral research leading to a Ph.D. degree in Biotechnology.", "answer_start": 262}]
                },
                {
                    "id": "177",
                    "question": "What is the vision of the Department of Biotechnology?",
                    "answers": [{"text": "The vision is to be recognized internationally for its strong interdisciplinary research and teaching base in Plant and Animal Biotechnology, fostering collaboration with industries and healthcare institutions.", "answer_start": 165}]
                },
                {
                    "id": "178",
                    "question": "What are some thrust areas of research in the Department of Biotechnology?",
                    "answers": [{"text": "Thrust areas include molecular biology, gene expression regulation, host-pathogen interaction, clinical virology, nanotechnology applications in diagnostics and treatment, genomics/proteomics of cancers, conservation of endangered plants, and enhancement of secondary metabolite yield in crops.", "answer_start": 637}]
                }
            ]
}
    ]
}, {
    "title": "Data item 26",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Department of Biotechnology at Jamia Hamdard stands as a beacon of excellence in the field of life sciences, with a rich history dating back to its establishment in the year 1997. Since its inception, the department has been committed to advancing knowledge and fostering innovation in the diverse realms of biotechnology. Offering a comprehensive array of academic programs, the department caters to the educational needs of aspiring biotechnologists at various levels. Students can embark on a journey of discovery through programs such as the 4-Year B.Sc. (Hons.) Biotechnology, 2-Year M.Sc. Biotechnology, 2-Year M.Tech. Biotechnology, and doctoral research leading to a Ph.D. degree in Biotechnology. This multifaceted approach ensures that learners receive a holistic education that equips them with the skills and knowledge needed to thrive in the dynamic field of biotechnology. At the heart of the departments mission lies a visionary outlook aimed at international recognition for its interdisciplinary research and teaching endeavors. With a focus on Plant and Animal Biotechnology, the department strives to foster collaboration with industries and healthcare institutions, thus driving innovation and pushing the boundaries of scientific discovery. Research within the Department of Biotechnology spans an array of thrust areas, reflecting its commitment to addressing pressing global challenges and unlocking new frontiers in biotechnological innovation. From molecular biology and gene expression regulation to clinical virology and nanotechnology applications, the departments research endeavors encompass a broad spectrum of cutting-edge topics. Additionally, the department is actively engaged in exploring genomics and proteomics of cancers, conservation of endangered plants, and enhancing secondary metabolite yield in crops, further underscoring its dedication to addressing societal needs and advancing scientific knowledge. Through its pioneering research, innovative academic programs, and visionary leadership, the Department of Biotechnology at Jamia Hamdard continues to shape the future of biotechnology and make significant contributions to the global scientific community.",
            "qas": [
                {
                    "id": "179",
                    "question": "When was the Department of Medical Elementology and Toxicology established at Jamia Hamdard?",
                    "answers": [{"text": "The Department of Medical Elementology and Toxicology was established in the year 1997.", "answer_start": 4}]
                },
                {
                    "id": "180",
                    "question": "What are some collaborative programs of the Department?",
                    "answers": [{"text": "The department has collaborative programs with institutes like Indian Institute of Toxicological Research (CSIR), Central Drug Research Institute (CSIR), and Indian Institute of Integrative Medicine (CSIR), among others.", "answer_start": 435}]
                },
                {
                    "id": "181",
                    "question": "What is the vision of the Department of Medical Elementology and Toxicology?",
                    "answers": [{"text": "The departments vision is to impart education and training in toxicology, medical elementology, and forensic toxicology, aiming to assess and mitigate risks posed by chemicals and drugs to human health and the environment.", "answer_start": 916}]
                },
                {
                    "id": "182",
                    "question": "What are some thrust areas of research in the Department?",
                    "answers": [{"text": "Thrust areas include chemoprevention of cancer, immunotoxicity, endocrine disrupting chemicals, oxidative stress, neurodegenerative disorders, trace elements in diseases, ecotoxicity, nanoparticle toxicity, arthritis models, alternative animal models, protein assembly, amyloid toxicity, drug discovery, and fabrication of amyloid-based smart materials.", "answer_start": 958}]
                }
            ]
}
    ]
}, {
    "title": "Data item 27",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The School of Unani Medical Education and Research (SUMER) holds a significant place in the history of Jamia Hamdard, tracing its roots back to its establishment in 1963. Since its inception, SUMER has been a pioneering institution in the field of Unani medicine, offering a wide array of degrees and courses. These include undergraduate degree courses (BUMS), Pre Tib bridge courses, Diploma in Unani Pharmacy, Postgraduate courses (M.D. Unani) in three disciplines, and Ph.D. in Unani Medicine. SUMER is structured into fourteen departments, each dedicated to exploring various aspects of Unani medicine, ensuring comprehensive coverage of this traditional healthcare system. Additionally, SUMER is closely associated with the Majeedia Unani Hospital, which serves as an invaluable resource for practical training, research, and providing healthcare services rooted in the principles of Unani medicine.",
            "qas": [
                {
                    "id": "183",
                    "question": "When was the School of Unani Medical Education and Research (SUMER) established?",
                    "answers": [{"text": "The School of Unani Medical Education and Research (SUMER) was established in 1963.", "answer_start": 4}]
                },
                {
                    "id": "184",
                    "question": "What degrees and courses does SUMER offer?",
                    "answers": [{"text": "SUMER offers undergraduate degree course (BUMS), Pre Tib bridge course, Diploma in Unani Pharmacy, Postgraduate courses (M.D. Unani) in three disciplines, and Ph.D. in Unani Medicine.", "answer_start": 293}]
                },
                {
                    "id": "185",
                    "question": "How many departments does SUMER consist of?",
                    "answers": [{"text": "SUMER consists of fourteen departments covering various aspects of Unani medicine.", "answer_start": 594}]
                },
                {
                    "id": "186",
                    "question": "What is the name of the hospital attached to SUMER?",
                    "answers": [{"text": "The attached hospital is named Majeedia Unani Hospital.", "answer_start": 883}]
                }
            ]
}
    ]
}, {
    "title": "Data item 28",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The School of Interdisciplinary Sciences and Technology (SIST) at Jamia Hamdard is a dynamic hub of interdisciplinary education and research, encompassing diverse fields of study. Within SIST, two departments play a pivotal role: the Department of Food Technology (DOFT) and the Department of Molecular Medicine (DMM). These departments offer a comprehensive range of degrees and courses, catering to the needs of aspiring students in various domains. SIST offers undergraduate (B.Tech.) and postgraduate (M.Tech.) programs in Food Technology, along with MSc programs in Nutrition and Dietetics, Biomedical Science, and Medical Virology. Additionally, SIST provides opportunities for doctoral research through Ph.D. programs in Food Technology and Molecular Medicine, empowering students to delve deeper into their areas of interest and contribute to cutting-edge research. One of the key strengths of SIST lies in its collaborations, fostering partnerships with esteemed institutions such as URDIP (CSIR), Pune. Through these collaborations, SIST offers unique programs like Ph.D. in Interdisciplinary Sciences and Ph.D. in Traditional Indian Medicine, providing students with a platform to explore diverse perspectives and interdisciplinary approaches to research and education. In the Department of Food Technology, research endeavors span a wide spectrum of areas, including Food Fortification, Edible films, Probiotics, Nanotechnology, Phytochemicals, Food safety and hazards, Food and Drug Interactions, Food Rheology, Powder Technology, Pharmacokinetics, Functional Foods, and Clinical Nutrition. These research areas reflect the departments commitment to addressing contemporary challenges and advancing knowledge in the field of food science and technology.",
            "qas": [
                {
                    "id": "187",
                    "question": "What departments are housed under the School of Interdisciplinary Sciences and Technology (SIST)?",
                    "answers": [{"text": "The School of Interdisciplinary Sciences and Technology (SIST) houses two departments: Department of Food Technology (DOFT) and Department of Molecular Medicine (DMM).", "answer_start": 87}]
                },
                {
                    "id": "188",
                    "question": "What degrees and courses does SIST offer?",
                    "answers": [{"text": "SIST offers B.Tech. and M.Tech. in Food Technology, MSc in Nutrition and Dietetics, Biomedical Science, and Medical Virology, as well as Ph.D. programs in Food Technology and Molecular Medicine.", "answer_start": 222}]
                },
                {
                    "id": "189",
                    "question": "What collaborations does SIST have?",
                    "answers": [{"text": "SIST has collaboration with URDIP (CSIR), Pune, and offers unique programs like Ph.D. in Interdisciplinary Sciences and Ph.D. in Traditional Indian Medicine.", "answer_start": 509}]
                },
                {
                    "id": "190",
                    "question": "What are the major research areas in the Department of Food Technology?",
                    "answers": [{"text": "The major research areas in the Department of Food Technology include Food Fortification, Edible films, Probiotics, Nanotechnology, Phytochemicals, Food safety and hazards, Food and Drug Interactions, Food Rheology, Powder Technology, Pharmacokinetics, Functional Foods, and Clinical Nutrition.", "answer_start": 971}]
                }
            ]
}
    ]
},
{
    "title": "Data item 29",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "Hamdard Institute of Legal Studies and Research (HILSR) is a prominent institution dedicated to legal education, established in 2018 with a vision to nurture future leaders in the field of law. Offering a range of degrees and programs, HILSR aims to provide comprehensive legal education and foster a culture of excellence and innovation. At HILSR, students have the opportunity to enroll in various degree programs tailored to their interests and career aspirations. These include BA.LL.B., a five-year integrated degree program designed to provide a strong foundation in legal studies from an early stage. Additionally, HILSR offers a two-year LL.M program with specializations in Constitutional Law and Commercial Law, providing students with an opportunity to delve deeper into specific areas of legal expertise. For those inclined towards research and academia, HILSR also offers a Ph.D. program, enabling scholars to contribute to the advancement of legal knowledge through rigorous research. To support the academic endeavors of its students, HILSR provides modern infrastructure equipped with state-of-the-art facilities. This includes a well-stocked library with an extensive collection of legal resources, smart classrooms equipped with modern teaching aids, a computer lab for practical learning, a Moot Court Hall to simulate courtroom experiences, Seminar Halls for academic discussions, and Students Common Rooms for relaxation and informal gatherings. Moreover, HILSR fosters a vibrant campus life by offering various clubs and committees aimed at enhancing students overall development. These include the Legal Aid Clinic, where students engage in pro bono legal assistance to the underprivileged, the Legal Literacy Club promoting legal awareness among the community, the Constitutional Club focusing on constitutional studies and debates, the ADR Board facilitating alternative dispute resolution mechanisms, the Moot Court Committee organizing moot court competitions to hone advocacy skills, the Pro Bono Club encouraging voluntary legal services, and other clubs like the Readers Club, Debate Club, and Cultural Club, providing platforms for intellectual discourse, public speaking, and cultural activities.",
            "qas": [
                {
                    "id": "191",
                    "question": "When was Hamdard Institute of Legal Studies and Research (HILSR) established?",
                    "answers": [{"text": "Hamdard Institute of Legal Studies and Research (HILSR) was established in 2018.", "answer_start": 0}]
                },
                {
                    "id": "192",
                    "question": "What degrees and programmes does HILSR offer?",
                    "answers": [{"text": "HILSR offers BA.LL.B., a five-year integrated degree programme; a two-year LL.M programme with specializations in Constitutional Law and Commercial Law; and a Ph.D. programme.", "answer_start": 140}]
                },
                {
                    "id": "193",
                    "question": "What infrastructure does HILSR provide?",
                    "answers": [{"text": "HILSR provides modern infrastructure including a well-stocked library, smart classrooms, computer lab, Moot Court Hall, Seminar Hall, and Students Common Rooms.", "answer_start": 392}]
                },
                {
                    "id": "194",
                    "question": "What clubs and committees are available at HILSR?",
                    "answers": [{"text": "HILSR has various clubs and committees such as the Legal Aid Clinic, Legal Literacy Club, Constitutional Club, ADR Board, Moot Court Committee, Pro Bono Club, Readers Club, Debate Club, and Cultural Club.", "answer_start": 610}]
                }
            ]
}
    ]
}, {
    "title": "Data item 30",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Department of Islamic Studies at Jamia Hamdard holds a significant place in the academic landscape, tracing its roots back to the visionary efforts of the late Founder-Chancellor, Janab Hakeem Abdul Hameed Saheb. Established under his guidance, the department has been a beacon of knowledge and scholarship in the field of Islamic studies. Dedicated to providing comprehensive education in Islamic studies, the department offers a range of programs catering to students academic pursuits and research interests. These include B.A. (Hons.), M.A., and Ph.D. programs, designed to equip students with a deep understanding of Islamic principles, history, and contemporary issues. One of the departments distinguishing features is its focus on diverse research areas that address socio-economic, religious, and political challenges faced by Muslims in India. Moreover, the department explores the rich contributions of Muslims to various spheres of life and delves into the profound impact of Islam on Indian society and culture. Through its rigorous academic curriculum and research endeavors, the Department of Islamic Studies at Jamia Hamdard continues to uphold its commitment to scholarly excellence and intellectual inquiry, nurturing future leaders and scholars in the field of Islamic studies.",
            "qas": [
                {
                    "id": "195",
                    "question": "Who established the Department of Islamic Studies at Jamia Hamdard?",
                    "answers": [{"text": "The Department of Islamic Studies was established by the late Founder-Chancellor, Janab Hakeem Abdul Hameed Saheb.", "answer_start": 82}]
                },
                {
                    "id": "196",
                    "question": "What programs does the Department of Islamic Studies offer?",
                    "answers": [{"text": "The Department of Islamic Studies offers B.A. (Hons.), M.A., and Ph.D. programs.", "answer_start": 130}]
                },
                {
                    "id": "197",
                    "question": "What are the main research areas of the Department of Islamic Studies?",
                    "answers": [{"text": "The main research areas include socio-economic, religious, and political problems of Muslims in India, contributions of Muslims to different walks of life, and the impact of Islam on India.", "answer_start": 625}]
                }
            ]
}
    ]
}, {
    "title": "Data item 31",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Hamdard Institute of International Studies, established in 2021, stands as a testament to Jamia Hamdards commitment to global education and fostering a nuanced understanding of international affairs. With a focus on interdisciplinary learning and research, the institute aims to equip students with the knowledge and skills necessary for thriving careers in academia, foreign services, international organizations, and think-tanks. At the heart of its mission lie the diverse areas of advanced studies and research offered by the institute. These encompass a broad spectrum of topics, including Indias Foreign Policy, Comparative Politics, Great Power Politics, History of International Politics, South Asian Studies, Globalization and Global Governance, Peace and Conflict Studies, United Nations, Regional and Security Organizations, International Law, International Security, Regional Politics including West Asia and Africa, and International Political Economy. Through rigorous academic programs and research initiatives, the institute endeavors to deepen understanding and analysis of contemporary global issues, preparing students to navigate the complexities of an interconnected world.",
            "qas": [
                {
                    "id": "198",
                    "question": "When was the Hamdard Institute of International Studies established?",
                    "answers": [{"text": "Established in 2021,", "answer_start": 0}]
                },
                {
                    "id": "199",
                    "question": "What are the aims of the Hamdard Institute of International Studies?",
                    "answers": [{"text": "The department aims to prepare students for careers in academics, foreign services, international organizations, and think-tanks.", "answer_start": 166}]
                },
                {
                    "id": "200",
                    "question": "What are some of the areas of advanced studies and research at the Hamdard Institute of International Studies?",
                    "answers": [{"text": "Some areas include Indias Foreign Policy, Comparative Politics, Great Power Politics, History of International Politics, South Asian Studies, Globalization and Global Governance, Peace and Conflict Studies, United Nations, Regional and Security Organizations, International Law, International Security, Regional Politics including West Asia and Africa, and International Political Economy.", "answer_start": 231}]
                }
            ]
}
    ]
},
{
    "title": "Data item 32",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Centre of Excellence (COE) in Unani Medicine, specifically focusing on Pharmacognosy & Pharmacology, is directed by Professor Sayeed Ahmad. The AYUSH system has significant potential to contribute to global public health, with the Unani system being a crucial part of it. However, achieving this goal requires exhaustive scientific research and training for competency development of AYUSH professionals. The CoE aligns with the vision of founder Chancellor Janab Hakeem Abdul Hameed Sahib by promoting state-of-the-art research in traditional medicine, particularly Unani, using modern concepts of drug development from pharmaceutical approaches. Supported by AYUSH under the Ayurswasthya Yojana, the CoE houses a well-equipped lab aimed at conducting research on Unani and other traditional drugs, as well as medicinal plants, for quality control, metabolomic profiling, pharmacokinetics, stability, toxicity, and biological screenings required for drug development at global standards. The objective of the CoE is to advance this challenging task by conceptualizing and executing contemporary and scientific research, along with training AYUSH professionals in modern scientific techniques.",
            "qas": [
                {
                    "id": "201",
                    "question": "Who is the director of the Centre of Excellence (COE) in Unani Medicine (Pharmacognosy & Pharmacology)?",
                    "answers": [{"text": "Professor Sayeed Ahmad", "answer_start": 17}]
                },
                {
                    "id": "202",
                    "question": "What is the objective of the COE in Unani Medicine?",
                    "answers": [{"text": "The objective of the COE is to advance this challenging task by conceptualizing and executing contemporary and scientific research, along with training AYUSH professionals in modern scientific techniques.", "answer_start": 578}]
                },
                {
                    "id": "203",
                    "question": "What kind of research does the COE focus on?",
                    "answers": [{"text": "The COE focuses on research on Unani and other traditional drugs, as well as medicinal plants, for quality control, metabolomic profiling, pharmacokinetics, stability, toxicity, and biological screenings required for drug development at global standards.", "answer_start": 301}]
                }
            ]
}
    ]
}, {
    "title": "Data item 33",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Centre for Environment and Sustainable Development (CESD) is directed by Prof. M. Z. Abdin, with Dr. M. Iqbal R. Khan serving as the Deputy Director. In the face of climate change and a growing global population, ensuring food and nutritional security requires leveraging expertise and technologies from various disciplines of basic sciences for environmental and agricultural sustainability. The vision of CESD is to generate worldwide food, nutritional, and bio-resources by nurturing environmentalists, biotechnologists, and agriculturists, enabling them to contribute to sustainable development goals (SDGs) on national and international platforms. The mission of CESD includes advancing the understanding of the environment and sustainability through multi-disciplinary education, research, and outreach programs; educating and training future generations to resolve environmental issues through innovative research; enhancing agricultural productivity and income under adverse environmental conditions; strengthening national research programs; introducing innovations and policy interventions for agricultural growth; raising awareness about the consequences of anthropogenic activities on climate systems and the environment; and integrating environment-sound perception for socio-economic concerns to enhance food and nutritional security.",
            "qas": [
                {
                    "id": "204",
                    "question": "Who is the director of the Centre for Environment and Sustainable Development (CESD)?",
                    "answers": [{"text": "Prof. M. Z. Abdin", "answer_start": 43}]
                },
                {
                    "id": "205",
                    "question": "What is the vision of CESD?",
                    "answers": [{"text": "The vision of CESD aims toward generating worldwide food, nutritional, and bio-resources by developing ambitious environmentalists, biotechnologists, and agriculturists to contribute to sustainable development goals (SDGs) on national and international platforms.", "answer_start": 198}]
                },
                {
                    "id": "206",
                    "question": "What is the mission of CESD?",
                    "answers": [{"text": "The mission of CESD includes advancing the understanding of the environment and sustainability through multi-disciplinary education, research, and outreach programs; educating and training future generations to resolve environmental issues through innovative research; enhancing agricultural productivity and income under adverse environmental conditions; strengthening national research programs; introducing innovations and policy interventions for agricultural growth; raising awareness about the consequences of anthropogenic activities on climate systems and the environment; and integrating environment-sound perception for socio-economic concerns to enhance food and nutritional security.", "answer_start": 285}]
                }
            ]
}
    ]
},
{
    "title": "Data item 34",
    "context_type": "Schools",
    "paragraphs": [
        {
            "context": "The Centre for Media and Mass Communication Studies (CMMS), led by Prof. (Dr.) Reshma Nasreen, is dedicated to providing premier education and training in journalism and mass communication. CMMS offers a B.A. (Honours with Research) in Journalism and Mass Communication and an M.A. in Journalism and Mass Communication program. In todays information and communication technology revolution, the influence and significance of Mass Media and Communication have expanded significantly. The programs at CMMS are designed to equip aspiring media professionals with industry and technical expertise essential for success in todays competitive media landscape. The curriculum trains candidates in various industry and technical skills such as audio-video editing, anchoring, content writing, photography, and media reporting, preparing them for roles in TV news channels, radio broadcasting, print media, digital journalism, and government communication setups. Graduates gain critical thinking abilities, effective communication skills, and a versatile skill set, enabling them to pursue various career opportunities in journalism, public relations, social media management, advertising, documentary filmmaking, content writing, video editing, photojournalism, and government/corporate communication roles.",
            "qas": [
                {
                    "id": "207",
                    "question": "Who is the Director (In-charge) of Centre for Media and Mass Communication Studies (CMMS)?",
                    "answers": [{"text": "Prof. (Dr.) Reshma Nasreen", "answer_start": 49}]
                },
                {
                    "id": "208",
                    "question": "What programs are offered at CMMS?",
                    "answers": [{"text": "CMMS offers a B.A. (Honours with Research) in Journalism and Mass Communication and an M.A. in Journalism and Mass Communication program.", "answer_start": 92}]
                },
                {
                    "id": "209",
                    "question": "What skills do the programs at CMMS aim to develop in students?",
                    "answers": [{"text": "The programs aim to develop strategic thinkers proficient in both practical and theoretical aspects of the field. Graduates gain critical thinking abilities, effective communication skills, and a versatile skill set.", "answer_start": 293}]
                },
                {
                    "id": "210",
                    "question": "What are some career opportunities for graduates of CMMS programs?",
                    "answers": [{"text": "Career opportunities include roles such as journalist/media reporter, public relations specialist, social media manager, media planner, advertising executive, TV anchor/radio jockey, documentary filmmaker, copy/content writer, video editor, photojournalist/photographer, and government/corporate communication roles.", "answer_start": 660}]
                }
            ]
}
    ]
}, {
    "title": "Data item 35",
    "context_type": "Campus Facilities",
    "paragraphs": [
        {
            "context": "The Centre for Training and Development (CTD), led by Prof. (Dr.) Syeedun Nisa, is an initiative of Jamia Hamdard under the School of Management and Business Studies, established in 2022. The centre aims to provide education and skill development through practical and relevant training programs to help participants build their careers, develop corporate skill sets, and keep up with the fast-changing world. Its objective is to ensure that participants are better fit in the industry by making them industry and society ready. The curriculum is based on modern-day corporate requirements and academic excellence, focusing on areas such as research, education, behavioral sense, personality development, and skill development in domains like sales & marketing, international business, HR operations, finance, strategy, and leadership. CTD conducts training programs, workshops, seminars, and webinars at local, national, and international levels, providing trainees with necessary resources, mentorship, and skill-building activities for their professional career potential. The centre collaborates with organizations such as the African – Asian Rural Development Organisation, Islamiah Womens Arts & Science College, and The Garage Productions Private Limited to drive innovation, access specialized expertise, and advance research.",
            "qas": [
                {
                    "id": "211",
                    "question": "Who is the Convener/Hony. Director of the Centre for Training and Development (CTD)?",
                    "answers": [{"text": "Prof. (Dr.) Syeedun Nisa", "answer_start": 28}]
                },
                {
                    "id": "212",
                    "question": "What is the aim of the Centre for Training and Development (CTD)?",
                    "answers": [{"text": "The centre aims to provide education and skill development through practical and relevant training programs to help participants build their careers, develop corporate skill sets, and keep up with the fast-changing world.", "answer_start": 131}]
                },
                {
                    "id": "213",
                    "question": "What is the objective of the Centre for Training and Development (CTD)?",
                    "answers": [{"text": "The objective is to ensure that participants are better fit in the industry by making them industry and society ready.", "answer_start": 255}]
                },
                {
                    "id": "214",
                    "question": "What areas does the Centre for Training and Development (CTD) focus on?",
                    "answers": [{"text": "The centre focuses on areas such as research, education, behavioral sense, personality development, and skill development in domains like sales & marketing, international business, HR operations, finance, strategy, and leadership.", "answer_start": 469}]
                },
                {
                    "id": "215",
                    "question": "With which organizations has the Centre for Training and Development (CTD) collaborated?",
                    "answers": [{"text": "The centre has collaborated with organizations such as the African – Asian Rural Development Organisation, Islamiah Womens Arts & Science College, and The Garage Productions Private Limited.", "answer_start": 819}]
                }
            ]
}
    ]
},
{
    "title": "Data item 36",
    "context_type": "Campus Facilities",
    "paragraphs": [
        {
            "context": "The Centre for Teaching Support and Innovations (CTSI) at Jamia Hamdard is led by Prof. Mymoona Akhter, a dedicated academician committed to enhancing teaching methodologies and fostering innovation in education. Established in 2022, CTSI has swiftly emerged as a cornerstone of the academic landscape, playing a pivotal role in supporting faculty members and promoting inclusive education. At its core, the mission of CTSI transcends mere compliance with UGC mandates; it aspires to embody the vision and intent of the National Education Policy (NEP). With a focus on flexibility and mobility, CTSI endeavors to break down barriers between general and vocational education, creating an environment conducive to holistic learning and growth. CTSIs significance in the academic arena lies in its multifaceted approach to supporting faculty members and advancing teaching practices. Through workshops, seminars, and collaborative platforms, CTSI facilitates the exchange of ideas and best practices among faculty members, fostering a culture of continuous improvement and innovation in teaching methodologies. By promoting academic equivalence across different domains of education, CTSI contributes to the holistic development of students and the overall enhancement of the educational experience at Jamia Hamdard.",
            "qas": [
                {
                    "id": "216",
                    "question": "Who is the Director of the Centre for Teaching Support and Innovations (CTSI)?",
                    "answers": [{"text": "Prof. Mymoona Akhter", "answer_start": 9}]
                },
                {
                    "id": "217",
                    "question": "When was the Centre for Teaching Support and Innovations (CTSI) established?",
                    "answers": [{"text": "The Centre for Teaching Support and Innovations (CTSI) was established in 2022.", "answer_start": 97}]
                },
                {
                    "id": "218",
                    "question": "What is the mission of the Centre for Teaching Support and Innovations (CTSI)?",
                    "answers": [{"text": "The mission is not only to meet UGC mandates but also to realize the vision and intent of the National Education Policy (NEP), creating a university environment that embodies flexibility and mobility, breaking down barriers between general and vocational education.", "answer_start": 261}]
                },
                {
                    "id": "219",
                    "question": "What role does the Centre for Teaching Support and Innovations (CTSI) play in the academic landscape?",
                    "answers": [{"text": "CTSI plays a crucial role in supporting faculty members and promoting innovation in teaching methodologies. It fosters an inclusive educational atmosphere and aims to establish academic equivalence between different domains of education.", "answer_start": 622}]
                },
                {
                    "id": "220",
                    "question": "What initiatives does the Centre for Teaching Support and Innovations (CTSI) undertake to support faculty members?",
                    "answers": [{"text": "Through workshops, seminars, and collaborative platforms, CTSI facilitates the exchange of ideas and best practices among faculty members.", "answer_start": 906}]
                }
            ]
}
    ]
},
{
    "title": "Data item 37",
    "context_type": "Campus Facilities",
    "paragraphs": [
        {
            "context": "The Industry Academia Partnership Cell (IAPC) at Jamia Hamdard is spearheaded by Prof. Syeedun Nisa, a distinguished academician committed to fostering collaborations between academia and industry. With a vision to produce technologically competent, morally and emotionally strong, and ethically sound professionals dedicated to the service of mankind, IAPC strives to bridge the gap between theoretical knowledge and practical application. At its core, the mission of IAPC is to develop technically and ethically strong students who can perform effectively in both personal and professional life. By nurturing a conducive environment for industry-academia collaboration, IAPC endeavors to equip students with the skills and mindset necessary to excel in a rapidly evolving global landscape. The core team of IAPC comprises senior faculties, industry representatives, assistant TPO, and faculties and student coordinators from each school. This diverse team brings together expertise from academia and industry, ensuring a holistic approach to fostering partnerships and enhancing students employability. From the School of Management and Business Studies (SMBS), Ms. Manpreet Kaur Kohli and Dr. Jwaad A Khan serve as coordinators, actively contributing to the collaborative efforts of IAPC. Their involvement underscores the commitment of SMBS to nurturing industry-ready professionals equipped with the requisite knowledge and skills to thrive in the competitive business environment.",
            "qas": [
                {
                    "id": "221",
                    "question": "Who is the Convener of the Industry Academia Partnership Cell (IAPC) at Jamia Hamdard?",
                    "answers": [{"text": "Prof. Syeedun Nisa", "answer_start": 33}]
                },
                {
                    "id": "222",
                    "question": "What is the vision of the Industry Academia Partnership Cell (IAPC)?",
                    "answers": [{"text": "The vision is to produce technologically competent, morally & emotionally strong, and ethically sound professionals dedicated to the service of mankind.", "answer_start": 367}]
                },
                {
                    "id": "223",
                    "question": "What is the mission of the Industry Academia Partnership Cell (IAPC)?",
                    "answers": [{"text": "The mission is to develop technically and ethically strong students who can perform effectively in both personal and professional life.", "answer_start": 479}]
                },
                {
                    "id": "224",
                    "question": "Who forms the core team of the Industry Academia Partnership Cell (IAPC)?",
                    "answers": [{"text": "The core team includes senior faculties, industry representatives, assistant TPO, and faculties and student coordinators from each school.", "answer_start": 772}]
                },
                {
                    "id": "225",
                    "question": "Which faculties are the coordinators from the School of Management and Business Studies (SMBS)?",
                    "answers": [{"text": "Ms. Manpreet Kaur Kohli, Dr. Jwaad A Khan", "answer_start": 890}]
                }
            ]
}
    ]
}, {
    "title": "Data item 38",
    "context_type": "Campus Facilities",
    "paragraphs": [
        {
            "context": "Recruitment processes at the Industry Academia Partnership Cell (IAPC) encompass several stages aimed at identifying and selecting suitable candidates for various job roles. Typically, companies initiate the process by conducting Pre-Placement talks to introduce students to their organization and job opportunities. This is followed by written tests to assess students knowledge and skills. If applicable, Group Discussions are conducted to evaluate candidates communication and interpersonal abilities. Subsequently, eligible and shortlisted students undergo Technical and HR interviews to further assess their suitability for the role. Students are informed about job openings through a structured process facilitated by IAPC. When an organization expresses interest in recruiting students, details of job openings and profiles are shared with relevant IAPC committee members and student coordinators. Interested students register for the recruitment process and provide their consent to participate. After the recruitment process concludes, shortlisted students undergo background verification as per the companys policy. If the eligibility criteria and background verification are satisfactory, students are extended job offers by the respective companies. The remuneration offered to students recruited through IAPC varies depending on factors such as job role, company policies, and individual performance. Students have received attractive salary packages ranging from 3.5 Lakhs Per Annum (LPA) to 31 LPA from renowned companies. Organizations frequently engage in campus hiring with each batch and consistently express satisfaction with the caliber and performance of the recruited students.",
            "qas": [
                {
                    "id": "226",
                    "question": "What are some typical rounds in the recruitment process conducted by companies at the Industry Academia Partnership Cell (IAPC)?",
                    "answers": [{"text": "Companies typically conduct Pre-Placement talks, written tests, Group Discussions (if applicable), and Technical and HR interviews for eligible and shortlisted students.", "answer_start": 167}]
                },
                {
                    "id": "227",
                    "question": "How are students informed about job openings at the Industry Academia Partnership Cell (IAPC)?",
                    "answers": [{"text": "Once an organization shows interest in recruiting students, job openings and profiles are shared with the related IAPC committee members and student coordinators. Interested students register and give consent to participate.", "answer_start": 432}]
                },
                {
                    "id": "228",
                    "question": "What happens after the recruitment process is over at the Industry Academia Partnership Cell (IAPC)?",
                    "answers": [{"text": "After the recruitment process, shortlisted students undergo background verification as per the companys policy. If eligibility and background verification are satisfactory, the student is offered the position.", "answer_start": 655}]
                },
                {
                    "id": "229",
                    "question": "What salary packages have students received from companies recruited through the Industry Academia Partnership Cell (IAPC)?",
                    "answers": [{"text": "Students have received packages ranging from 3.5LPA to 31LPA from well-known companies. Organizations frequently conduct campus hiring with each batch and express satisfaction with the students performance.", "answer_start": 992}]
                }
            ]
}
    ]
},
{
    "title": "Data item 39",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "For the academic session 2024-25, admission to undergraduate programs at Jamia Hamdard will be primarily based on merit in various entrance exams. Candidates may qualify for admission based on their performance in NEET (UG), JEE (Main-Paper-I) 2024, CUET-2024, or CLAT-2024, depending on the program they are applying for. Similarly, admission to programs other than undergraduate ones will also be conducted through a variety of criteria. This may include marks obtained in the Qualifying Examination, performance in interviews, entrance tests, or other approved modes of assessment. In cases where candidates have not qualified in NEET (UG), JEE, CLAT, or CUET, Jamia Hamdard reserves the right to consider their applications if suitable rank holders are not available in a particular discipline. However, such considerations are subject to approval by the Competent Authority. To apply for admission to programs at Jamia Hamdard for the academic session 2024-25, candidates are required to submit their applications online through the designated Online Admission Portal 2024. For MBBS/MD/MS programs at HIMSR (Hamdard Institute of Medical Sciences and Research), the admission process shall be governed by the guidelines provided by the National Medical Commission (NMC).",
            "qas": [
                {
                    "id": "230",
                    "question": "What will be the basis for admission to undergraduate programs at Jamia Hamdard for the academic session 2024-25?",
                    "answers": [{"text": "Admission to undergraduate programs at Jamia Hamdard for the academic session 2024-25 will be based on merit in NEET (UG), JEE (Main-Paper-I) 2024, CUET-2024, or CLAT-2024, as applicable.", "answer_start": 0}]
                },
                {
                    "id": "231",
                    "question": "How will admission be conducted for programs other than undergraduate ones at Jamia Hamdard for the academic session 2024-25?",
                    "answers": [{"text": "For other programs, admission may be based on marks obtained in the Qualifying Examination, Interview, Entrance Test, or other approved modes.", "answer_start": 193}]
                },
                {
                    "id": "232",
                    "question": "What does Jamia Hamdard reserve the right to do regarding applications from candidates who have not qualified in NEET (UG), JEE, CLAT, or CUET for the academic session 2024-25?",
                    "answers": [{"text": "Jamia Hamdard reserves the right to consider applications from Non-NEET (UG)/ Non-JEE / Non CLAT/CUET qualified candidates if suitable rank holders are not available in a particular discipline, subject to approval of the Competent Authority.", "answer_start": 377}]
                },
                {
                    "id": "233",
                    "question": "How can candidates apply for admission to programs at Jamia Hamdard for the academic session 2024-25?",
                    "answers": [{"text": "Applications for all programs shall be accepted online through the Online Admission Portal 2024.", "answer_start": 513}]
                },
                {
                    "id": "234",
                    "question": "What governs the admission process for MBBS/MD/MS programs at HIMSR at Jamia Hamdard for the academic session 2024-25?",
                    "answers": [{"text": "Admission to MBBS/MD/MS programs at HIMSR shall be governed by NMC Guidelines.", "answer_start": 608}]
                }
            ]
}
    ]
},
  {
    "title": "Data item 40",
    "context_type": "Admissions",
    "paragraphs": [
        {
            "context": "Applicants aspiring to join Jamia Hamdard are strongly advised to thoroughly review the contents of the Information Bulletin & Prospectus. Its crucial for them to familiarize themselves with the intricacies of the Admission/Entrance Test/GD/Interview/Counselling processes conducted by the university. This familiarity ensures that candidates are well-prepared and understand the requirements and procedures involved in seeking admission. Jamia Hamdard maintains stringent standards regarding admission criteria. Even if a candidate successfully clears all requisite eligibility conditions, such as the Entrance Test/GD/Interview merit and fulfills the criteria outlined in the offer letter, admission may be denied if the candidate fails to meet the eligibility criteria specified by the university. In such cases, Jamia Hamdard cannot be held responsible for the denial of admission. As per the instructions provided, a bachelors degree signifies that it has been obtained under either the 10+2+3 or 10+2+4 system of education, or an equivalent system. This clarification ensures that candidates understand the educational qualifications required for admission to various programs offered by Jamia Hamdard. Candidates whose qualifying examinations are not recognized by either the Association of Indian Universities (AIU) or Jamia Hamdard will not have their application forms considered for admission. Recognition by these authorities is crucial for ensuring the validity and eligibility of the candidates academic qualifications.",
            "qas": [
                {
                    "id": "235",
                    "question": "What advice is given to applicants regarding the contents of the Information Bulletin & Prospectus?",
                    "answers": [{"text": "Applicants are advised to read carefully the contents of this Information Bulletin & Prospectus and familiarize themselves with the Admission/Entrance Test/GD/Interview/Counselling of Jamia Hamdard.", "answer_start": 0}]
                },
                {
                    "id": "236",
                    "question": "For what reason might a candidate be denied admission according to Jamia Hamdard?",
                    "answers": [{"text": "Jamia Hamdard will not be responsible if a candidate is denied admission for not fulfilling the eligibility criteria even if she/he has cleared all requisite eligibility conditions such as the Entrance Test/GD/Interview merit of qualifying examination and has been issued offer letter for admission to a particular Program of study.", "answer_start": 312}]
                },
                {
                    "id": "237",
                    "question": "What does a bachelors degree signify according to the instructions?",
                    "answers": [{"text": "Bachelors degree signifies that the degree has been obtained under 10+2+3 or 10+2+4 system of education or equivalent.", "answer_start": 740}]
                },
                {
                    "id": "238",
                    "question": "What happens if a candidates qualifying examination is not recognized by AIU or Jamia Hamdard?",
                    "answers": [{"text": "Application Form of candidate whose qualifying examination is not recognized by AIU (Association of Indian Universities) or by Jamia Hamdard shall not be considered for admission.", "answer_start": 1210}]
                }
            ]}]}
]


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import Dataset, concatenate_datasets

In [ ]:

# Flatten the nested structure and transform to SQuAD format
squad_format_data = []
for example in data:
    title = example["title"]
    context_type = example["context_type"]  # New line to retrieve context type
    for paragraph in example["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            qid = qa["id"]
            question = qa["question"]
            answers = [{"text": ans["text"], "answer_start": ans["answer_start"]} for ans in qa["answers"]]
            squad_format_data.append({"title": title, "context_type": context_type, "context": context, "question": question, "answers": answers})


In [ ]:
# Separate lists for each column including context_type
titles = [item["title"] for item in squad_format_data]
context_types = [item["context_type"] for item in squad_format_data]
contexts = [item["context"] for item in squad_format_data]
questions = [item["question"] for item in squad_format_data]
answers = [item["answers"] for item in squad_format_data]

# Create a Dataset in SQuAD format including context_type
squad_dataset = Dataset.from_dict({"title": titles, "context_type": context_types, "context": contexts, "question": questions, "answers": answers})

# Print all samples to verify the format
for sample in squad_dataset:
    print(sample)


{'title': 'Data item 1', 'context_type': 'Introduction to University', 'context': 'Jamia Hamdard, nestled in Jamia Hamdard Nagar, New Delhi, stands as a beacon of educational excellence and innovation since its inception. Declared as a deemed-to-be University in the year 1989, it has consistently strived towards academic brilliance under the leadership of visionaries and educators. Inaugurated by the then Prime Minister of India, Shri Rajiv Gandhi, on August 1, 1989, Jamia Hamdard embarked on a journey dedicated to fostering knowledge and empowerment. During the inauguration, Shri Rajiv Gandhi envisioned Jamia Hamdard as not just a center of learning but as a catalyst for societal progress. He remarked,This will enable minority to go forward and help India to march forward. These words echoed the institutions commitment to inclusivity and national development. The sprawling campus of Jamia Hamdard hosts a myriad of facilities aimed at holistic growth and development. With ten Schools, 

In [ ]:
context_pool = [
    "Jamia Hamdard, nestled in Jamia Hamdard Nagar, New Delhi, stands as a beacon of educational excellence and innovation since its inception. Declared as a deemed-to-be University in the year 1989, it has consistently strived towards academic brilliance under the leadership of visionaries and educators. Inaugurated by the then Prime Minister of India, Shri Rajiv Gandhi, on August 1, 1989, Jamia Hamdard embarked on a journey dedicated to fostering knowledge and empowerment. During the inauguration, Shri Rajiv Gandhi envisioned Jamia Hamdard as not just a center of learning but as a catalyst for societal progress. He remarked,This will enable minority to go forward and help India to march forward. These words echoed the institutions commitment to inclusivity and national development. The sprawling campus of Jamia Hamdard hosts a myriad of facilities aimed at holistic growth and development. With ten Schools, including a prestigious medical college, two hospitals, a state-of-the-art library, archive center, and modern hostels for both girls and boys, it provides an enriching environment for academic pursuits. Additionally, facilities like a gymnasium, expansive playground, and residential quarters for staff underscore its dedication to the overall well-being of its community. Furthermore, its reach extends beyond Delhi, with an off-campus center in Kannur, Kerala, enriching its educational outreach. In the realm of pharmaceutical sciences, Jamia Hamdard reigns supreme, holding the coveted first position in the country according to the Government of Indias NIRF-2022 ranking. Such accolades are a testament to its commitment to excellence and innovation in academia. Financial support in the form of grants has been instrumental in advancing the institutions research endeavors. Grants from various sources including the PURSE grant, a substantial Rs. 10 Cr allocation from the AYUSH Ministry, and funding under STUITI have fueled groundbreaking research initiatives. The academic prowess of Jamia Hamdard is further evidenced by its scholarly impact, with an impressive h-index surpassing 100. This metric reflects the institutions significant contributions to the advancement of knowledge and research. Collaborations with esteemed organizations like the Defense Research and Development Organization (DRDO) have been pivotal in fostering innovation and technological advancements. The close relationship between Jamia Hamdard and DRDO has led to joint research endeavors yielding pharmaceutical products utilized by defense forces, showcasing the institutions multifaceted contributions to national development. In essence, Jamia Hamdard stands as a testament to the transformative power of education, fostering excellence, inclusivity, and innovation in the pursuit of societal progress and national advancement",
    "Hakeem Abdul Hameed, a luminary in the realm of education and healthcare, left an indelible mark as the Founder-Chancellor of Jamia Hamdard. His visionary leadership and unwavering commitment to excellence laid the foundation for the institutions journey towards academic distinction. Throughout his illustrious life, Hakeem Abdul Hameed garnered numerous accolades, reflecting his significant contributions to society. In 1965, he was honored with the Padma Shri, followed by the prestigious Padma Bhushan in 1992. Additionally, he received the esteemed Avicenna Award in 1983, among various other recognitions, underscoring his profound impact and dedication to service. Jamia Hamdard, inaugurated as a Deemed-to-be University in May 1989, stands as a testament to Hakeem Abdul Hameeds vision and foresight. His relentless pursuit of excellence in education and healthcare culminated in the establishment of this esteemed institution, which continues to uphold his legacy of innovation and social responsibility. Beyond Jamia Hamdard, Hakeem Abdul Hameeds influence extended far and wide through the establishment of several other institutions. These include the Hamdard National Foundation, Hamdard Education Society, Hamdard Study Circle, Business and Employment Bureau, Hamdard Public School, Ghalib Academy, Centre for South Asian Studies, and India Islamic Cultural Centre. Each of these institutions reflects his multifaceted approach towards societal development and his enduring commitment to holistic education and cultural enrichment. Hakeem Abdul Hameeds visionary leadership, coupled with his profound dedication to education, healthcare, and cultural preservation, continues to inspire generations. His legacy remains etched in the annals of history, serving as a guiding light for institutions and individuals alike, striving for excellence and societal advancement",
    "Hammad Ahmed has been a pivotal figure in the Hamdard National Foundation, demonstrating a profound commitment to its mission and objectives. He has been involved with the foundation for more than 50 years, dating back to its inception in 1964. His extensive experience and dedication have significantly contributed to the foundations growth and success.Currently, Hammad Ahmed holds the prestigious position of President of Hamdard National Foundation (India) - HECA, as well as Co-chairman of Hamdard National Foundation (India). His leadership roles within the foundation reflect his deep-rooted involvement and influence in guiding its strategic direction and initiatives.In addition to his work with the Hamdard National Foundation, Hammad Ahmed has played a crucial role at Jamia Hamdard. He has served on the Executive Council, Finance Committee, and various other advisory bodies, providing valuable insights and governance to the university. His contributions have been instrumental in steering Jamia Hamdard towards academic excellence and institutional integrity.Hammad Ahmeds educational background is as impressive as his professional achievements. He received his early education at Sherwood College in Nainital, a renowned institution known for its rigorous academic environment. He furthered his studies in Cornwall, UK, and New York, USA, acquiring a diverse and international educational experience that has undoubtedly enriched his professional and leadership skills.Through his longstanding service and dedication, Hammad Ahmed continues to uphold the values and vision of the Hamdard National Foundation and Jamia Hamdard, fostering growth, excellence, and societal contribution",
    "Prof. Mohammad Afshar Alam has been a distinguished figure in the academic and administrative landscape of Jamia Hamdard. He currently serves as the Vice Chancellor of Jamia Hamdard, New Delhi, a role that underscores his leadership and vision for the institutions future.Prof. Alams academic journey is marked by notable achievements. He completed his postgraduate studies (MCA) at Aligarh Muslim University, followed by a Ph.D. from Jamia Milia Islamia. His strong educational foundation has been pivotal in his ascent to various influential positions within Jamia Hamdard.Throughout his tenure at Jamia Hamdard, Prof. Alam has held several critical roles. These include serving as the Head of the Computer Centre, Dean of Management and Information Technology, Dean of Engineering Sciences and Technology, Dean of Students Welfare, Foreign Students Advisor, and Finance Officer. Each of these roles highlights his versatility and dedication to fostering a robust academic environment.Prof. Mohammad Afshar Alams expertise spans multiple domains, reflecting his diverse academic interests and research contributions. His areas of specialization include Software Re-engineering, Data Mining, Bio-Informatics, Fuzzy Databases, and Sustainable Development. This broad spectrum of knowledge underscores his ability to drive interdisciplinary research and innovation, contributing significantly to the academic communitys advancement.In his capacity as Vice Chancellor, Prof. Alam continues to lead Jamia Hamdard with a commitment to excellence, ensuring the institution remains at the forefront of higher education and research. His leadership and expertise are pivotal in navigating the university through the evolving educational landscape, fostering a culture of continuous growth and development",
    "Jamia Hamdard, a premier institution in New Delhi, is led by a distinguished team of administrators and academic leaders who ensure the smooth functioning and progressive growth of the university. At the helm of this team is Mr. Hammad Ahmad, who serves as the Chancellor of Jamia Hamdard, bringing his extensive experience and vision to the institution. Assisting him is Prof. Mohammad Afshar Alam, the Vice Chancellor, who is instrumental in steering the university towards academic excellence and innovation.The administrative framework is further strengthened by Dr. M.A. Sikandar, the Registrar, who oversees the universitys administrative operations. The financial matters are managed by Mr. Munish Malik, the Finance Officer, ensuring the financial health and stability of the institution. Mr. Syed Saud Akhtar, responsible for examinations, plays a crucial role in maintaining the academic integrity of the universitys assessment processes.The academic leadership includes several deans who oversee various schools within Jamia Hamdard. Prof. Farhan Jalees Ahmed is the Dean of the School of Pharmaceutical Education and Research, renowned for its top-ranking programs. The School of Nursing Sciences and Allied Health is led by Prof. Shakir Ali, who ensures the provision of high-quality education and training in nursing and allied health sciences. The School of Engineering Sciences & Technology is headed by Prof. Farheen Siddiqui, who drives innovation and research in engineering disciplines. Prof. Reshma Nasreen serves as the Dean of the School of Management & Business Studies, as well as the Director of the Centre for Media and Mass Communication Studies and the Dean of Students Welfare, exemplifying her versatile leadership. Prof. S. Raisuddin is the Dean of the School of Chemical & Life Sciences and also serves as the Director of the IQAC/R & D cell, playing a pivotal role in advancing research and quality assurance.Further, Prof. Asim Ali Khan holds dual roles as the Dean of the School of Unani Medical Education and Research and the Chief Medical Superintendent at Majeedia Unani Hospital, showcasing his expertise in Unani medicine. The Rufaida College of Nursing is led by Principal Prof. Veena Sharma, who upholds the highest standards in nursing education.Other key roles include Prof. M.S. Alam, the Chief Proctor; Prof. Zeenat Iqbal, the Foreign Students Advisor; Prof. Md. Shahnawaz Abdin, the Provost for Boys; and Prof. Syeedun Nisa, the Provost for Girls, each ensuring a supportive and well-regulated campus environment. The universitys library services are managed by Mr. M. Shoaib, the University Librarian, who ensures access to a vast repository of knowledge and resources.Mr. Mirza Rahil Beg serves as the Public Information Officer, handling the dissemination of information, while Mr. Md. Tauhid Alam, the Media Coordinator, manages the universitys media relations. Additionally, Prof. Mahmooduzzafar, as the Director of the Center for Distance and Online Education, facilitates flexible learning opportunities for students.Together, this team of dedicated leaders and administrators at Jamia Hamdard works tirelessly to maintain the universitys esteemed reputation and foster an environment of academic rigor, research innovation, and holistic development",
    "The School of Pharmaceutical Education & Research (SPER) at Jamia Hamdard stands as a beacon of excellence in pharmaceutical education and research. Under the adept leadership of Prof. (Dr.) Farhan Jalees Ahmed, the school has achieved remarkable milestones and accolades.The Ministry of Education, Government of India, has consistently recognized SPERs outstanding contributions to the field. The school was ranked No. 1 among national institutions in the National Institutional Ranking Framework (NIRF) for the years 2019, 2020, 2021, and 2022, and secured the No. 2 position in 2023. These rankings underscore the schools commitment to maintaining high standards in education, research, and innovation.SPER offers a comprehensive range of academic programs designed to cater to various levels of pharmaceutical education. These include undergraduate programs such as the Diploma in Pharmacy (D. Pharm.) and Bachelor of Pharmacy (B. Pharm.), postgraduate programs (M. Pharm.) in specialties like Pharmaceutics, Pharmaceutical Chemistry, Pharmacology, Pharmacognosy, Pharmaceutical Quality Assurance, Pharmaceutical Analysis, Pharmacy Practice, and Pharmaceutical Biotechnology, as well as rigorous doctoral programs. These diverse programs equip students with the knowledge and skills required to excel in the dynamic field of pharmaceuticals.The school has also established robust collaborations with prominent industries, enhancing the practical and research experience for its students and faculty. Notable industry partners include Sun Pharma, Dabur Research Foundation, Jubilant Life Sciences, Hamdard Labs India Ltd (Food Division), AIMIL Pharmaceuticals International, Fermish Clinical Technology, GBC in Beijing, China, Multani Dolphin Pharmaceuticals, Avitech Nutrition, and Tulips Lab. These collaborations facilitate cutting-edge research, internships, and job placements, thereby bridging the gap between academic learning and industry requirements.The continuous efforts and strategic initiatives led by Prof. (Dr.) Farhan Jalees Ahmed and his team have propelled the School of Pharmaceutical Education & Research at Jamia Hamdard to the forefront of pharmaceutical education, making it a preferred destination for aspiring pharmaceutical professionals",
    "The School of Pharmaceutical Education & Research (SPER) at Jamia Hamdard comprises several specialized departments, each dedicated to advancing education and research in their respective fields. These departments include the Department of Pharmaceutics, the Department of Pharmaceutical Chemistry, the Department of Pharmacology, and the Department of Pharmacognosy and Phytochemistry.The Department of Pharmaceutics offers two masters degree programs in Pharmaceutics and Pharmaceutical Quality Assurance, as well as Ph.D. programs in Pharmaceutics and Pharmaceutics with a specialization in Pharmaceutical Quality Assurance. The departments research focuses on areas such as oral bioavailability enhancement, transdermal drug delivery systems, ocular drug delivery systems, targeted nanoparticles, intranasal drug delivery systems, biodegradable targeted retention devices, herbal drug delivery systems, and transmucosal drug delivery systems.The Department of Pharmaceutical Chemistry provides academic programs including M. Pharm. in Pharmaceutical Chemistry and Pharmaceutical Analysis, along with a Ph.D. program. The departments research encompasses Drug Design and Development, Synthetic Chemistry, Analytical Pharmacy, and Natural Product Chemistry, contributing significantly to the field of pharmaceutical sciences.The Department of Pharmacology offers a masters program in Pharmacology and Pharmacy Practice, and Ph.D. programs in Pharmacology, Pharmacology with a specialization in Pharmacy Practice, and Pharmaceutical Medicine. The research conducted by this department covers Neuropharmacology, Cardiovascular Pharmacology, Endocrinology, and Clinical Pharmacology, aiming to advance understanding and treatment of various medical conditions.The Department of Pharmacognosy & Phytochemistry focuses on the discovery and development of herbal drugs, phytopharmaceuticals, and microbial products. Their academic offerings include programs that cater to the study and research of these natural compounds. The departments research areas include quality control of herbal/microbial products and botanicals, enhanced production of secondary metabolites, studies on herbal cosmetics, immunomodulatory, anti-obesity, and anti-diabetic medicinal plants, screening of new drug molecules, and the isolation, identification, and characterization of natural compounds.Additionally, the Centre of Excellence in Unani Medicine at Jamia Hamdard is dedicated to conducting scientific investigations related to Traditional Unani Medicine, contributing to the integration of traditional knowledge with modern scientific research.Each of these departments and research centers at SPER plays a crucial role in advancing pharmaceutical education and research, making Jamia Hamdard a leading institution in the field.",
    "The School of Pharmaceutical Education & Research (SPER) at Jamia Hamdard offers a range of academic programs across different levels, each with specific eligibility criteria and selection processes. Here is an overview of the programs and their requirements:The Diploma in Pharmacy (D. Pharm.) program requires candidates to have passed the Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry, and Biology or Mathematics, securing at least 45% marks in the aggregate and in these subjects. Candidates must be at least 17 years old by December 31, 2024, as per PCI norms. Admission is based on merit from the qualifying examination (10+2 PCB/PCM stream).For the Bachelor of Pharmacy (B. Pharm.) program, candidates must have passed the Senior Secondary examination with Physics, Chemistry, and Biology/Mathematics, securing at least 50% marks in aggregate. Candidates must be at least 17 years old by December 31, 2024. Selection is based on NEET 2024 or Paper-1 of JEE (Main) 2024 scores. Non-NEET/non-JEE candidates may apply but will be considered only if NEET/JEE candidates are unavailable.The B. Pharm. II Year (Lateral entry) program requires candidates to have passed the Diploma in Pharmacy examination with 50% marks in aggregate and the 10+2 examination with PCB. Admission is based on merit determined by D. Pharm. marks. Preference is given to those who have completed 500 hours of practical training over at least three months.The Master of Pharmacy (M. Pharm.) program requires candidates to have passed the B. Pharm. examination with at least 55% marks. Preference is given to GPAT qualified candidates. For the Sponsored category, candidates need two years of full-time work experience. Admission is based on GPAT merit; if insufficient GPAT candidates are available, B. Pharm. percentage is considered.The Ph.D. programs at Jamia Hamdard, including Ph.D. in Pharmaceutical Medicine, require an M. Pharm. in relevant specializations or an MBBS degree with at least 55% marks. Preference is given to candidates with MD/DNB degrees in related fields. Two seats in the Ph.D. (Pharmaceutical Medicine) program are reserved for Pharmaceutical Industry nominated candidates, one under General and one under SFS category. Notifications for Ph.D. admissions for the 2024-25 session will be released in July 2024.SPER at Jamia Hamdard also offers scholarships for GPAT qualified M. Pharm. students through UGC, supporting the academic and research pursuits of its students.This comprehensive range of programs and the robust selection criteria ensure that SPER at Jamia Hamdard continues to produce highly qualified professionals in the field of pharmaceutical education and research",
    "The School of Nursing Sciences and Allied Health (SNSAH) at Jamia Hamdard encompasses a variety of departments dedicated to healthcare education and training. These include the College of Nursing (Rufaida College of Nursing, RCN), the Department of Occupational Therapy (DOT), the Department of Physiotherapy (DPT), the Department of Paramedical Sciences (DPS), and the Rehabilitation Centre. Each of these departments offers specialized programs designed to equip students with the necessary skills and knowledge for their respective fields.Rufaida College of Nursing (RCN) offers a comprehensive range of programs including the Diploma in General Nursing and Midwifery, B.Sc. (Hons.) Nursing, M.Sc. Nursing, Post Basic B.Sc. Nursing, and Ph.D. in Nursing. The colleges motto, “Par excellence in preparing competent nurse professionals,” reflects its commitment to producing highly skilled nursing graduates. All courses offered by Rufaida College of Nursing are recognized by the Indian Nursing Council (INC) and the State Nursing Council–Delhi Nursing Council (DNC), ensuring that the education provided meets national standards.Students at Rufaida College of Nursing receive extensive clinical training, gaining hands-on experience in the parent hospital, Hakeem Abdul Hameed Centenary Hospital (HAHC), as well as other affiliated hospitals such as Safdarjung Hospital, VIMHANS, IHBAS, and various rural and urban health centers operated by M.C.D. and Jamia Hamdard. This diverse clinical exposure is designed to prepare students for real-world healthcare environments.The college is well-equipped with state-of-the-art infrastructure and learning resources. It boasts a dedicated building with an upgraded simulation lab and a well-maintained library, providing students with excellent facilities for both practical and theoretical learning. This robust educational environment supports the colleges mission to deliver high-quality nursing education and research opportunities",
    "The School of Nursing Sciences and Allied Health (SNSAH) at Jamia Hamdard offers a range of programs aimed at preparing competent healthcare professionals.Diploma in General Nursing and Midwifery (DGNM)Eligibility criteria for the DGNM program include passing the 10+2 examination with English, securing a minimum of 40% marks, and passing English individually. Candidates must also be at least 17 years old by 31st December 2024. Subjects required for admission include Physics, Chemistry, Biology, Mathematics, Commerce, Accountancy, Economics, Business Studies, Computer Science, and others.B.Sc. (Hons.) NursingFor the B.Sc. (Hons.) Nursing program, candidates must be at least 17 years old by 31st December 2024 and have passed the Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry, Biology, and English from a recognized board, securing at least 45% marks in aggregate. Selection is based on merit from NEET 2024.Post Basic B.Sc. NursingThe Post Basic B.Sc. Nursing program requires candidates to have a DGNM qualification from a recognized institution and be registered as RN (Registered Nurse) and RM (Registered Midwife) under the State Nursing Council. Candidates must be at least 17 years old by 31st December 2024. Selection may involve an entrance test conducted by Jamia Hamdard or be based on the merit of the qualifying examination.M.Sc. NursingEligibility for the M.Sc. Nursing program includes having a B.Sc. Nursing, B.Sc. (Hons.) Nursing, or Post Basic B.Sc. Nursing with at least 55% marks in aggregate from a recognized institution, being a Registered Nurse and Registered Midwife or equivalent, and having a minimum of one year of work experience in the nursing field. Candidates are selected based on the merit of the qualifying examination.Ph.D. in NursingThe Ph.D. in Nursing program requires candidates to have an M.Sc. Nursing or a postgraduate degree in Nursing recognized by INC with a minimum of 60% marks. Additionally, candidates must have three years of teaching or clinical experience after their M.Sc. Nursing. Selection is based on the merit of the entrance test and an interview conducted by Jamia Hamdard.These programs are designed to equip students with both theoretical knowledge and practical skills, preparing them for various professional roles in the healthcare sector",
    "The Department of Occupational Therapy at Jamia Hamdard has been a cornerstone of academic excellence since its inception in 2000 and 2001. Offering a diverse range of academic programs, it has played a pivotal role in shaping the future of healthcare professionals in the field of occupational therapy.With a focus on both undergraduate and postgraduate education, the Department offers comprehensive programs in Occupational Therapy. Additionally, it provides avenues for advanced research through Ph.D. programs in Rehabilitation Science and Occupational Therapy.Occupational therapy, as envisioned by the department, aims to enable individuals to participate fully in the activities of everyday life. This holistic approach is reflected in the multifaceted roles played by occupational therapists, ranging from clinical practitioners to researchers, academicians, counselors, diplomats, and rehabilitation directors.In India, occupational therapists find employment opportunities across various sectors, including government organizations, institute hospitals, projects, non-governmental organizations, and the private sector. They contribute significantly to healthcare delivery in settings such as acute care hospitals, nursing homes, and special schools.Students enrolled in the Department of Occupational Therapy receive a well-rounded education encompassing both theoretical knowledge and practical skills. Clinical exposure and internships form an integral part of their training, with opportunities available in government and private hospitals, NGOs, and special schools across Delhi and the National Capital Region (NCR).The department takes pride in its alumni, boasting a remarkable 100% placement rate. Many alumni have gone on to serve in prestigious institutions such as LN Hospital, GB Pant Hospital, Kalawati Saran Childrens Hospital, ESI Hospital, PDDUNIPPD, and Ganga Ram Hospital.Beyond national borders, the impact of the Department of Occupational Therapy extends globally, with alumni settled in countries including the USA, England, Ireland, UAE, Singapore, Republic of Congo, Australia, and Canada. Their contributions resonate across continents, showcasing the departments commitment to excellence and its enduring legacy in the field of occupational therapy.",
    "At Jamia Hamdard, the Bachelor of Occupational Therapy (BOT) program offers a gateway into the dynamic field of occupational therapy. With a commitment to excellence, Jamia Hamdard provides comprehensive education and training in occupational therapy to meet the evolving healthcare needs of society.Bachelor of Occupational Therapy Program:Available Seats: The BOT program at Jamia Hamdard has 50 seats available, offering ample opportunities for aspiring occupational therapists to pursue their education and career goals.Program Duration: The duration of the BOT program spans four years, ensuring a comprehensive academic curriculum and practical training. Additionally, students undergo a compulsory internship of six months to gain hands-on experience in real-world healthcare settings.Eligibility Criteria: To be eligible for admission to the BOT program, candidates must have passed the Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry, and Biology from a recognized board. They should secure at least 50% marks in the aggregate and also in the aggregate of Physics, Chemistry, and Biology, demonstrating their academic proficiency and readiness for the program.Selection Process: The selection process for the BOT program is based on 10+2 merit among the candidates applying through the Jamia Hamdard Online Admission Portal. This ensures a fair and transparent method of selecting candidates who exhibit academic excellence and potential for success in the field of occupational therapy.Jamia Hamdards commitment to providing quality education and training in occupational therapy sets the stage for aspiring healthcare professionals to make a meaningful impact in improving the lives of individuals through rehabilitation, therapy, and holistic care.",
    "Specializations in the Master of Occupational Therapy program:The Master of Occupational Therapy (MOT) program at Jamia Hamdard offers specializations in Musculoskeletal and Pediatrics. This enables students to focus their studies on specific areas of occupational therapy practice.Duration of the Master of Occupational Therapy program:The Master of Occupational Therapy (MOT) program spans over 2 years, providing students with a comprehensive and in-depth understanding of occupational therapy principles and practices.Eligibility Criteria for the Master of Occupational Therapy program:To be eligible for the Master of Occupational Therapy (MOT) program, candidates must have completed a Bachelor of Occupational Therapy (BOT) from an institution recognized by Jamia Hamdard, with a minimum aggregate score of 50%. Candidates who obtained their BOT degree through distance education are not eligible for admission to the MOT program.Selection Process for the Master of Occupational Therapy program:Selection for the MOT program is based on an entrance test conducted by Jamia Hamdard. The test assesses candidates understanding of basic sciences and occupational therapy subjects taught at the BOT level. This ensures that selected candidates have the necessary foundational knowledge to excel in the Masters program.",
    "Notification for Ph.D. Program in Occupational Therapy:The notification for admission to the Ph.D. program in Occupational Therapy for the session 2024-25 is scheduled to be released in July 2024. Prospective candidates interested in pursuing doctoral studies in this field should stay updated with the official announcements from Jamia Hamdard regarding the application process and important deadlines.Eligibility Criteria for Ph.D. Program in Occupational Therapy:Candidates aspiring to enroll in the Ph.D. program in Occupational Therapy must meet certain eligibility criteria. Specifically, they should hold a postgraduate degree in Occupational Therapy with a minimum of 55% marks. This degree should be obtained from either Jamia Hamdard or any other institution recognized by Jamia Hamdard. Fulfilling these criteria is essential for consideration for admission to the doctoral program.",
    "The Department of Physiotherapy at Jamia Hamdard, formerly known as the Department of Rehabilitation Sciences, has a rich history dating back to its establishment in 1998. Initially offering a Postgraduate program, it expanded its offerings to include an Undergraduate program in 2001. Since then, it has been a hub for education and research in the field of Physiotherapy, catering to a diverse range of disorders across various specialties. The scope of Physiotherapy encompasses Orthopedics, Neurology, Cardiopulmonary, Sports, Hand Rehabilitation, Community Based Rehabilitation, Obstetrics, Gynecology, Geriatrics, Pediatrics, and Womens health, reflecting the comprehensive nature of the discipline.Physiotherapists, besides their clinical practice, also play a vital role in academic and research activities. They contribute significantly to preventive healthcare initiatives,emphasizing the importance of proactive approaches to maintaining physical well-being. Thus, the Department of Physiotherapy at Jamia Hamdard serves as a cornerstone for both professional development and the advancement of knowledge in the field.",
    "Bachelor of Physiotherapy (B.P.T) Seats: 50 (Inclusive of seats for NRI/Sponsored candidates) Duration: Four years plus compulsory internship of six months Eligibility: Passed Senior Secondary (12th Standard/Intermediate) examination with Physics, Chemistry and Biology from Central Board of Secondary Education or any other examination recognized by Jamia Hamdard as equivalent thereto securing at least 50% marks in the aggregate and also in the aggregate of Physics, Chemistry and Biology. Selection process: On the basis of merit through NEET 2024, among the candidates, who apply for admission to BPT Degree Course through online Jamia Hamdard Application Form Portal. Non-NEET candidates may also apply. Although NEET candidates will be preferred, non-NEET applicants may be considered on the basis of merit of first 5 subjects in qualifying examination. Master of Physiotherapy (M.P.T) Seats: 21 (Inclusive of seats for NRI/Sponsored candidates) Specialization Cardiopulmonary 5 Musculoskeletal 6 Neurology 5 Sports 5 Duration: 2 Years Eligibility: A candidate must have: Bachelor of Physiotherapy (B.P.T.) from an institution recognized by Jamia Hamdard or University/Institute approved by the UGC, securing at least 50% marks in the aggregate. The BPT course should be recognized by Delhi Council of Physiotherapy and Occupational therapy. Candidates who have obtained degree in B.P.T. from distant mode are not eligible for Post Graduate courses in Physiotherapy. Selection process: Selection will be based on the Entrance Test conducted by Jamia Hamdard. The question paper will comprise of questions on basic sciences and physiotherapy subjects taught at BPT level. Ph.D. (Physiotherapy) The notification for admission in Ph.D. programme from session 2024-25 will be released in the month of July 2024. Eligibility: Master of Physiotherapy Programme with at least 55% marks (or equivalent CGPA) in aggregate from any University/Institution recognized by UGC. NET-JRF/Individual Fellowship holders/Project Fellows will be preferred for admissions in Ph.D. Programme.",
    "The Department of Paramedical Sciences at Jamia Hamdard was established in 2012 with a primary focus on training and providing skilled paramedical healthcare professionals. The department offers a range of diplomas, degrees, and postgraduate degrees to fulfill its mission.The vision of the Paramedical Department is to provide high-quality skilled students and competent professionals, thereby enhancing medical services and meeting international standards. Notably, 100% of successful graduate and postgraduate candidates are employed immediately after passing from the university, reflecting the departments commitment to producing job-ready professionals.For admission to diploma courses in paramedical sciences, candidates are selected based on the merit of the qualifying examination. Similarly, eligibility criteria for programs such as B.Sc. (Medical Laboratory Techniques) and M.Sc. (Medical Lab Science) require candidates to have passed the Senior Secondary examination with specific subjects and secured minimum aggregate marks.Lateral entry into B.Sc. (Medical Laboratory Techniques) is available for candidates who have completed a two-year diploma course (after 10+2) with a minimum of 50% aggregate marks or a three-year diploma after the 10th examination from a science stream.For Ph.D. (Medical Lab Science), candidates must have completed a Masters degree in Medical Lab Science or a relevant field with at least 55% marks from a recognized university or institution.The notification for admission to the Ph.D. program in Medical Lab Science for the session 2024-25 will be released in the month of July 2024, providing aspiring candidates with an opportunity to pursue advanced research and academic endeavors in the field.",
    "The Department of Computer Science & Engineering (CSE) at the School of Engineering Sciences & Technology, established in 1997, stands as a pioneering institution in the realm of computer education. It offers a diverse range of Under Graduate (UG) and Post Graduate (PG) Programmes catering to the dynamic landscape of technology and innovation. The academic spectrum includes programs in Computer Science & Engineering, Computer Application, Information Technology, Electronics & Communication Engineering, Data Science, Cyber Forensics & Information Security. Moreover, the department is a hub for cutting-edge research, with Ph.D. programs delving into various thrust areas of computer science and engineering. These include Big Data, Soft Computing, IoT, Artificial Intelligence, Data Mining, Information Security, and much more. Infrastructure The department boasts state-of-the-art infrastructure, featuring smart cluster rooms and well-equipped computer labs furnished with the latest technology. The computer labs house advanced PCs with i5 processors, ample RAM, and spacious storage, running on both Ubuntu and Windows operating systems. Additionally, theres a dedicated Research Lab for Ph.D. students, equipped with modern amenities like air conditioning and CCTV surveillance. A spacious Conference Room facilitates departmental meetings and student seminars. In the Electronics & Communication Engineering (ECE) domain, the department offers a range of labs covering Analog & Digital Electronics, Embedded Systems, FPGA Development, and more. These labs are equipped with the latest software and advanced instruments, providing students with hands-on experience in cutting-edge technologies. Internship and Training Opportunities Students at the Department of CSE benefit from numerous internship and training opportunities facilitated through strategic partnerships with industry leaders. MoUs signed with organizations like AICTEs Internshala Scheme, Co-Cubes, and RnF enable students to gain practical exposure and industry insights. Furthermore, collaborations with ETrain India Pvt. Ltd. ensure the conduct of certified seminars and workshops by Microsoft and Adobe. Industry Recruiters The department takes pride in its robust placement record, with students securing positions in renowned industry giants such as TCS, Wipro, HCL, Fiserv, RnF, and Ericsson. These partnerships not only provide valuable career opportunities but also validate the departments commitment to producing skilled professionals poised to make significant contributions to the technology landscape.",
    "The School of Engineering Sciences & Technology at Jamia Hamdard offers a diverse range of undergraduate programs designed to cater to the dynamic needs of the industry and academia. Here are some key details regarding the admission criteria and selection processes for various undergraduate programs: B.Tech. Computer Science & Engineering Program Duration: The Bachelor of Technology (B.Tech.) program in Computer Science & Engineering spans over a duration of 4 years, providing students with comprehensive knowledge and practical skills in this rapidly evolving field. Eligibility Criteria for B.Tech. Electronics and Communication Engineering: Candidates aspiring to join the B.Tech. Electronics and Communication Engineering program must have passed the Senior Secondary (12th Standard/Intermediate) examination with Mathematics and Physics as compulsory subjects, along with another subject from a specified list. A minimum aggregate of 50% marks or equivalent CGPA is required in the qualifying examination. Selection Process for Bachelor of Computer Application (BCA) Program: The selection process for admission to the Bachelor of Computer Application (BCA) program is based on the percentage of marks obtained in the qualifying examination. Selection Criteria for Bachelor of Science in Computer Science with Research Program: Admission to the Bachelor of Science (B.Sc.) program in Computer Science with Research is determined based on the merit of the Central University Entrance Test (CUET)-2024. The CUET subjects considered for evaluation include Mathematics, Computer Science, and Informatics Practices. In case of insufficient CUET-2024 qualified candidates, selection may be offered based on the merit of the qualifying examination. Eligibility Requirement for Lateral Entry to 2nd Year in B.Tech. (ECE): Candidates seeking lateral entry to the second year of the B.Tech. program in Electronics and Communication Engineering (ECE) must hold a Diploma in Engineering from a recognized institution/university, with a minimum aggregate of 50% marks or equivalent CGPA.",
    "Master of Computer Applications (MCA) Seats: 60, inclusive of seats reserved for NRI/sponsored candidates; additional seats are available for Foreign Nationals. Duration: 2 Years Eligibility:  Passed  B.E./B.Tech./BCA/B.Sc./B.Com./B.A.with  Mathematics  at  10+2  level  or  at Graduation Level and Obtained at least 50% marks in aggregate in the qualifying Examination. Selection process: Merit of the qualifying examination. M.Sc. (Computational Biology & System Biology and Bioinformatics)Seats: 10 candidates; inclusive of seats reserved for NRI/sponsored candidates; additional seats are available for Foreign Nationals.Duration: 2 years (4 Semesters)Eligibility:  A  minimum  of  55%  marks  in  Bachelors  degree  program  in  bioinformatics,  or  any branch  of  basic/applied  Sciences  or  Technology,  including  Life  Sciences,  Medicine,  Pharmacy, Engineering and Computer Science and allied disciplines.Selection process: Merit of the qualifying examination.Regular M.Tech. CoursesSeats: 30 (28 seats General and 02 seats reserved for Industry/NRI sponsored candidates)M.Tech. (Computer Science & Engineering) - M.Tech. (CSE)Duration: 2 years (4 Semesters)Eligibility:  Passed  B.Tech./B.E.  or  equivalent  degree  in  Computer  Science/Computer  Science  & Engineering/Computer Engineering/Information Technology/Software Engineering/ICT with at least 55% marks (or equivalent CGPA) in aggregate.(OR)MCA or M.Sc. in IT/Computer Science/Information Science & Technology/Electronics/Software Engineering or equivalent degree with at least 55% marks (or equivalent CGPA) in aggregate.(OR)B.Tech./B.E.  or in  Electronics  &  Communication/Electronics Engineering/Electrical Engineering with at least 55% marks (or equivalent CGPA) in aggregate.Selection process: Merit of the qualifying examination.M.Tech. (CSE) with Specialization in Data ScienceDuration: 2 years (4 Semesters)Eligibility: Passed  B.Tech./B.E.  or  equivalent  degree  in  Computer  Science/Computer  Science  and Engineering /Computer Engineering/Information Technology/Software Engineering/ICT with at least 55% marks (or equivalent CGPA) in aggregate.(OR)Passed  B.Tech./B.E.  or  equivalent  degree  in  Electronics  &  Communication/Electronics Engineering/Electrical Engineering with at least 55% marks (or equivalent CGPA) in aggregate.(OR)Passed MCA or M.Sc. (CS), M.Sc. (IT), M.Sc. (Software Engineering), M.Sc. (Electronics) with at least 55% marks (or equivalent CGPA) in aggregate.Selection process: Merit of the qualifying examination.M. Tech. (CSE) with Specialization in Cyber Forensics & Information SecurityDuration: 2 years (4 Semesters)Eligibility: Passed  B.Tech./B.E.  or  equivalent  degree  in  Computer  Science/Computer  Science  & Engineering/Computer Engineering/Information Technology/Software Engineering/ICT with at least 55% marks (or equivalent CGPA) in aggregate.(OR)MCA or  in  M.Sc. Science/Information IT/Computer  Science Technology/Electronics/Software Engineering or equivalent degree with at least 55% marks (or equivalent CGPA) in aggregate.(OR)B.Tech./B.E.  or  in  Electronics  &  Communication/Electronics Engineering/Electrical Engineering with at least 55% marks (or equivalent CGPA) in aggregate.Selection process: Merit of the qualifying examination.",
    "Management education was introduced in Jamia Hamdard in the year 1998. Since then, it has become one of the important units of the University. The School of Management & Business Studies (SMBS) has three departments namely,  Department  of  Management,  Department  of  Healthcare  & Pharmaceutical Management and Department of Hotel Management and Catering Technology. Today more than 1200 students are enrolled in the school for various PG and UG courses. Owing to the core competencies of the faculty, the school has made a name for itself, both among industry erudite and intelligentsia. The teaching pedagogy employed in SMBS is a combination of theoretical concept with practical application. The school is well endowed with intellectual capital and field experience with both senior and  young  faculty  members  who  have  plenty  of  research  experience  behind  them  and  extensive industry experience in the domain of Management specializations like Marketing, Finance, Human Resources,  Operations  Management,  International  Business  and  Entrepreneurship,  Public  Health, Healthcare  Management,  Pharmaceutical  Management,  Hotel  management,  Commerce,  Supply Chain Management etc.  Placement/Internships activities are  coordinated  by  a  placement  cell  headed  by,  Training  and Placement Officer, comprising of selected student representatives and faculty members. This includes both summer internship as well as final placement.",
    "BBA (Bachelor of Business Administration)Seats: 180Duration: 4 years/8 semestersEligibility: 10+2 from any recognized board with 50% aggregate marks.Selection Process: Candidate must appear in the CUET in the General test. If seats are vacant, the admission may be done on the basis of merit of the qualifying examination.Bachelor of Commerce (B.Com)Seats: 60Duration: 4 years/8 semestersEligibility: 10+2 from any recognized board with 50% aggregate marks.Selection  Process:  Candidate  must  appear  in  the  CUET  in  the  General  test.  If  seats  are  vacant,  the admission may be done on the basis of the qualifying examination.Master of Business Administration (MBA)The Department of Management offers MBA degree with various specialization in Management viz., Marketing,  Finance,  Human  Resource,  Operations  Management,  Information  Technology, International Business and Entrepreneurship.Seats: 120Duration: 2 years/4 semestersEligibility:i.  Bachelors degree in any discipline with minimum 50% aggregate marks/CGPA 5 on a 10-point scale. Candidates in their final year of graduation are also eligible to apply. Their final admission is subject to the availability of final mark sheet/provisional degree. Candidates from final year Graduate Programme need to  submit the aggregate marks up to  4th  semester (for 3-Year UG Programme) and should be minimum of 50% or CGPA 5 on a 10-Point Scale. * ii.  Candidates  who have appeared in any national level aptitude test like CAT/CMAT/MAT/GMAT/XAT/GPAT/ATMA and possess a valid score card. * Admission shall be cancelled if in any case candidates do not meet the eligibility criteria. Exemptions from Aptitude Test\u2022  Mid-career professionals with 2 years of work experience (after graduation).  \u2022  Candidates who have  completed undergraduate  programme of four-year or above duration (Example:  B.E./B.Tech./MBBS/B.A.  LLB,  etc.)  with  a  minimum  of  50%  aggregate marks/CGPA  of  5  on  a  10-point  scale.  Candidates  in  their  final  year  of  under  graduate programme with four-year or above duration (Example: B.E./B.Tech./MBBS/B.A. LLB, etc.) need to submit the aggregate marks up to 6th semester. * \u2022  Candidates who have obtained first division in at least two examinations out of three i.e. 10th, 12th & Undergraduate with second division in the third examination. However, no candidates will be called for GD & PI if they failed to secure minimum 50% or 5 CGPA on a 10-Point Scale in their Graduation.  Selection  Process:  On  the  basis  of  eligibility  criteria/exemptions  mentioned  above,  eligible candidates  will  be  called  for  GD/PI  at  Jamia  Hamdard  campus/online  mode  as  per  the  prevailing condition during the time of GD/PI. For  NRI/Sponsored  Seats:  CMAT/MAT/GMAT/XAT/CAT/ATMA  scores  are  not  necessary; admission will be determined solely on the basis of their performance in GD/PI.Master of Business Administration-Executive (MBA Executive)The Department of Management offers 2 years MBA Executive program for working professionals with various specialisation in Management viz., Marketing, Finance, Human Resource, Operations Management,  Information  Technology,  International  Business,  and  Entrepreneurship.  It  will  be offered in hybrid (online+offline) mode and classes will be conducted in evenings and on weekends.  Seats: 30Duration: 2 years/4 semesters.Eligibility: Educational  Qualification:  Bachelors  degree  in  any  discipline  with  minimum  50%  aggregate marks/CGPA 5 on a 10- point scale. The student should hold a degree from any of the universities incorporated  by  an  act  of  the  central  or  state  legislature  in  India,  or  other  educational  institutions established by an act of Parliament or declared in India, or other educational institutions established by an act of Parliament or declared to be deemed as a university under Section 3 of UGC Act, 1956, or possess an equivalent qualification recognized by the Ministry of Education, Government of India. Experience: Applicant must have three years of professional experience at Executive/ Managerial Level after graduation. Note: The last date of application shall be treated as the cut-off date in determining the experience of the candidate. Selection Process: The applicants will be shortlisted based on academic performance and/or work experience. Shortlisted candidates will be called for Group Discussion/Interview at Jamia Hamdard campus. Ph.D. (Management)The notification for admission in Ph.D. programme session 2024-25 will be released in the month of July 2024.Duration: 3 years minimumEligibility: Masters Degree (preferably MBA/PGDM) or a Professional degree declared equivalent to Masters degree in allied subjects with at least 55%aggregate marks (or equivalent CGPA) from any University/Institution recognized by the Association of Indian Universities (AIU/UGC/AICTE)/ corresponding statutory body. NET-JRF/Individual  Fellowship holders/Project Fellows will be preferred for admissions in Ph.D. programsPh.D. (Management) Part timeThe notification for admission in Ph.D. programme session 2024-25 will be released in the month of July, 2024.The part time Ph.D. Programme is a professional doctoral degree, designed to enhance executive and professional  practice  through  the  application  of  sound  theory  and  rigorous  research  into  real  and complex  issues  in  business  and  management.  Key  to  this  is  a  programme  to  hone  personal, consultancy and research skills amongst working professionals at different management levels. Corporate executives working with reputed organizations in India applying for admission to the part time Ph.D. Program shall be eligible as per the following conditions: Duration: 4 years minimum. The overall duration will be as per UGC Regulations for Ph.D. Eligibility: Masters Degree (preferably MBA/PGDM) or a Professional degree declared equivalent to  Masters  degree  with  at  least  55%  aggregate  marks  (or  equivalent  CGPA)  from  any  University/Institution  recognized  by  the  Association  of  Indian  Universities  (AIU/UGC/AICTE)/ corresponding statutory body. NET-JRF/Individual  Fellowship holders/Project Fellows will be preferred for admissions in Ph.D. programs Selection Process for Ph.D. (Management) and Part time Ph.D. (Management) Programmes: The admission process would be a two-tier process   \u2022  First stage would include qualifying Ph.D. entrance test, conducted by Jamia Hamdard \u2022  NET/JRF qualified candidates are  exempted from  the Ph.D. entrance test  and will directly qualify for second stage. \u2022  Second stage will include an interview organized by School  of Management  and Business Studies,  Jamia  Hamdard,  where  the  candidate  is  required  to  discuss  his/her  research interest/area  through  a  presentation  before  a  duly  constituted  Department  Research  Committee. The interview shall also consider the following aspects, viz. whether:  ●  The candidate possesses the competence for the proposed research; ●  The research work can be suitably undertaken at the Institution ●  The proposed area of research can contribute to new/additional knowledge.",
    "The School of Management and Business Studies at Jamia Hamdard offers several programs catering to the dynamic needs of the healthcare and pharmaceutical management sectors. For undergraduates interested in healthcare management, the Bachelor of Management Studies (BMS) - Healthcare Management program provides a solid foundation. Eligibility requires passing the 12th class board exams with a minimum of 50% marks, and selection is based on the merit of the qualifying exam. Moving on to postgraduate studies, the MBA program offers specializations in Healthcare and Hospital Management as well as Pharmaceutical Management. A bachelors degree in any discipline with a minimum of 50% aggregate marks is necessary for eligibility, and candidates in their final year of graduation are also eligible. Selection involves a comprehensive process including group discussions, personal interviews, and evaluation of academic performance. For those aspiring for doctoral studies, the Ph.D. programs in Health Management and Pharmaceutical Management are highly regarded. Eligibility criteria include holding an MBA, MHA, or M.Pharm. degree with at least 55% marks. The admission process comprises a two-tier system involving a qualifying Ph.D. entrance test followed by an interview where candidates present their research interests before a departmental research committee. Additionally, for working professionals seeking to pursue doctoral studies part-time, similar eligibility criteria apply. The admission process remains the same, emphasizing both academic excellence and research potential. Overall, these programs reflect Jamia Hamdards commitment to excellence in healthcare and pharmaceutical management education, providing students with the knowledge and skills needed to thrive in these critical sectors.",
    "The School of Chemical and Life Sciences at Jamia Hamdard has been a cornerstone of academic excellence since its inception. Teaching activities commenced in the academic year 1994-95, with a strong focus on postgraduate programs, marking the beginning of a journey towards advanced education and cutting-edge research in the fields of chemistry and life sciences. Comprising five departments and a Center for Translational and Clinical Research, the School offers a diverse range of academic avenues for students and researchers alike. These departments serve as hubs of innovation and exploration, fostering interdisciplinary collaboration and pushing the boundaries of scientific knowledge. In addition to its robust academic programs, the School of Chemical and Life Sciences boasts state-of-the-art research facilities designed to support groundbreaking research endeavors. The Central Instrumentation Facility (CIF) provides researchers with access to advanced equipment and instrumentation, empowering them to conduct high-impact experiments and analyses. Furthermore, the Center for Transgenic Plant Development stands as a testament to the Schools commitment to pioneering research in biotechnology and genetic engineering. Through its unwavering dedication to excellence in education and research, the School of Chemical and Life Sciences continues to inspire the next generation of scientists and contribute to the advancement of knowledge in chemical and life sciences disciplines.",
    "The Department of Biotechnology at Jamia Hamdard was established in 1997 with a vision to be recognized internationally for its strong interdisciplinary research and teaching base in Plant and Animal Biotechnology, fostering collaboration with industries and healthcare institutions. Supported by FIST (DST, Govt. of India) and SAP (UGC), the department offers a range of programs including 4-Year B.Sc. (Hons.) Biotechnology, 2-Year M.Sc. Biotechnology, 2-Year M.Tech. Biotechnology, and doctoral research leading to a Ph.D. degree in Biotechnology. Faculty members have secured numerous extramurally funded research projects from agencies like DBT, DST, CSIR, and others. Students in the M.Sc. program engage in intensive project work mentored by department faculty, covering literature survey, benchwork, data generation, thesis writing, and presentation skills.",
    "The Department of Biotechnology at Jamia Hamdard stands as a beacon of excellence in the field of life sciences, with a rich history dating back to its establishment in the year 1997. Since its inception, the department has been committed to advancing knowledge and fostering innovation in the diverse realms of biotechnology. Offering a comprehensive array of academic programs, the department caters to the educational needs of aspiring biotechnologists at various levels. Students can embark on a journey of discovery through programs such as the 4-Year B.Sc. (Hons.) Biotechnology, 2-Year M.Sc. Biotechnology, 2-Year M.Tech. Biotechnology, and doctoral research leading to a Ph.D. degree in Biotechnology. This multifaceted approach ensures that learners receive a holistic education that equips them with the skills and knowledge needed to thrive in the dynamic field of biotechnology. At the heart of the departments mission lies a visionary outlook aimed at international recognition for its interdisciplinary research and teaching endeavors. With a focus on Plant and Animal Biotechnology, the department strives to foster collaboration with industries and healthcare institutions, thus driving innovation and pushing the boundaries of scientific discovery. Research within the Department of Biotechnology spans an array of thrust areas, reflecting its commitment to addressing pressing global challenges and unlocking new frontiers in biotechnological innovation. From molecular biology and gene expression regulation to clinical virology and nanotechnology applications, the departments research endeavors encompass a broad spectrum of cutting-edge topics. Additionally, the department is actively engaged in exploring genomics and proteomics of cancers, conservation of endangered plants, and enhancing secondary metabolite yield in crops, further underscoring its dedication to addressing societal needs and advancing scientific knowledge. Through its pioneering research, innovative academic programs, and visionary leadership, the Department of Biotechnology at Jamia Hamdard continues to shape the future of biotechnology and make significant contributions to the global scientific community.",
    "The School of Unani Medical Education and Research (SUMER) holds a significant place in the history of Jamia Hamdard, tracing its roots back to its establishment in 1963. Since its inception, SUMER has been a pioneering institution in the field of Unani medicine, offering a wide array of degrees and courses. These include undergraduate degree courses (BUMS), Pre Tib bridge courses, Diploma in Unani Pharmacy, Postgraduate courses (M.D. Unani) in three disciplines, and Ph.D. in Unani Medicine. SUMER is structured into fourteen departments, each dedicated to exploring various aspects of Unani medicine, ensuring comprehensive coverage of this traditional healthcare system. Additionally, SUMER is closely associated with the Majeedia Unani Hospital, which serves as an invaluable resource for practical training, research, and providing healthcare services rooted in the principles of Unani medicine.",
    "The School of Interdisciplinary Sciences and Technology (SIST) at Jamia Hamdard is a dynamic hub of interdisciplinary education and research, encompassing diverse fields of study. Within SIST, two departments play a pivotal role: the Department of Food Technology (DOFT) and the Department of Molecular Medicine (DMM). These departments offer a comprehensive range of degrees and courses, catering to the needs of aspiring students in various domains. SIST offers undergraduate (B.Tech.) and postgraduate (M.Tech.) programs in Food Technology, along with MSc programs in Nutrition and Dietetics, Biomedical Science, and Medical Virology. Additionally, SIST provides opportunities for doctoral research through Ph.D. programs in Food Technology and Molecular Medicine, empowering students to delve deeper into their areas of interest and contribute to cutting-edge research. One of the key strengths of SIST lies in its collaborations, fostering partnerships with esteemed institutions such as URDIP (CSIR), Pune. Through these collaborations, SIST offers unique programs like Ph.D. in Interdisciplinary Sciences and Ph.D. in Traditional Indian Medicine, providing students with a platform to explore diverse perspectives and interdisciplinary approaches to research and education. In the Department of Food Technology, research endeavors span a wide spectrum of areas, including Food Fortification, Edible films, Probiotics, Nanotechnology, Phytochemicals, Food safety and hazards, Food and Drug Interactions, Food Rheology, Powder Technology, Pharmacokinetics, Functional Foods, and Clinical Nutrition. These research areas reflect the departments commitment to addressing contemporary challenges and advancing knowledge in the field of food science and technology.",
    "Hamdard Institute of Legal Studies and Research (HILSR) is a prominent institution dedicated to legal education, established in 2018 with a vision to nurture future leaders in the field of law. Offering a range of degrees and programs, HILSR aims to provide comprehensive legal education and foster a culture of excellence and innovation. At HILSR, students have the opportunity to enroll in various degree programs tailored to their interests and career aspirations. These include BA.LL.B., a five-year integrated degree program designed to provide a strong foundation in legal studies from an early stage. Additionally, HILSR offers a two-year LL.M program with specializations in Constitutional Law and Commercial Law, providing students with an opportunity to delve deeper into specific areas of legal expertise. For those inclined towards research and academia, HILSR also offers a Ph.D. program, enabling scholars to contribute to the advancement of legal knowledge through rigorous research. To support the academic endeavors of its students, HILSR provides modern infrastructure equipped with state-of-the-art facilities. This includes a well-stocked library with an extensive collection of legal resources, smart classrooms equipped with modern teaching aids, a computer lab for practical learning, a Moot Court Hall to simulate courtroom experiences, Seminar Halls for academic discussions, and Students Common Rooms for relaxation and informal gatherings. Moreover, HILSR fosters a vibrant campus life by offering various clubs and committees aimed at enhancing students overall development. These include the Legal Aid Clinic, where students engage in pro bono legal assistance to the underprivileged, the Legal Literacy Club promoting legal awareness among the community, the Constitutional Club focusing on constitutional studies and debates, the ADR Board facilitating alternative dispute resolution mechanisms, the Moot Court Committee organizing moot court competitions to hone advocacy skills, the Pro Bono Club encouraging voluntary legal services, and other clubs like the Readers Club, Debate Club, and Cultural Club, providing platforms for intellectual discourse, public speaking, and cultural activities.",
    "The Department of Islamic Studies at Jamia Hamdard holds a significant place in the academic landscape, tracing its roots back to the visionary efforts of the late Founder-Chancellor, Janab Hakeem Abdul Hameed Saheb. Established under his guidance, the department has been a beacon of knowledge and scholarship in the field of Islamic studies. Dedicated to providing comprehensive education in Islamic studies, the department offers a range of programs catering to students academic pursuits and research interests. These include B.A. (Hons.), M.A., and Ph.D. programs, designed to equip students with a deep understanding of Islamic principles, history, and contemporary issues. One of the departments distinguishing features is its focus on diverse research areas that address socio-economic, religious, and political challenges faced by Muslims in India. Moreover, the department explores the rich contributions of Muslims to various spheres of life and delves into the profound impact of Islam on Indian society and culture. Through its rigorous academic curriculum and research endeavors, the Department of Islamic Studies at Jamia Hamdard continues to uphold its commitment to scholarly excellence and intellectual inquiry, nurturing future leaders and scholars in the field of Islamic studies.",
    "The Hamdard Institute of International Studies, established in 2021, stands as a testament to Jamia Hamdards commitment to global education and fostering a nuanced understanding of international affairs. With a focus on interdisciplinary learning and research, the institute aims to equip students with the knowledge and skills necessary for thriving careers in academia, foreign services, international organizations, and think-tanks. At the heart of its mission lie the diverse areas of advanced studies and research offered by the institute. These encompass a broad spectrum of topics, including Indias Foreign Policy, Comparative Politics, Great Power Politics, History of International Politics, South Asian Studies, Globalization and Global Governance, Peace and Conflict Studies, United Nations, Regional and Security Organizations, International Law, International Security, Regional Politics including West Asia and Africa, and International Political Economy. Through rigorous academic programs and research initiatives, the institute endeavors to deepen understanding and analysis of contemporary global issues, preparing students to navigate the complexities of an interconnected world.",
    "The Centre of Excellence (COE) in Unani Medicine, specifically focusing on Pharmacognosy & Pharmacology, is directed by Professor Sayeed Ahmad. The AYUSH system has significant potential to contribute to global public health, with the Unani system being a crucial part of it. However, achieving this goal requires exhaustive scientific research and training for competency development of AYUSH professionals. The CoE aligns with the vision of founder Chancellor Janab Hakeem Abdul Hameed Sahib by promoting state-of-the-art research in traditional medicine, particularly Unani, using modern concepts of drug development from pharmaceutical approaches. Supported by AYUSH under the Ayurswasthya Yojana, the CoE houses a well-equipped lab aimed at conducting research on Unani and other traditional drugs, as well as medicinal plants, for quality control, metabolomic profiling, pharmacokinetics, stability, toxicity, and biological screenings required for drug development at global standards. The objective of the CoE is to advance this challenging task by conceptualizing and executing contemporary and scientific research, along with training AYUSH professionals in modern scientific techniques.",
    "The Centre for Environment and Sustainable Development (CESD) is directed by Prof. M. Z. Abdin, with Dr. M. Iqbal R. Khan serving as the Deputy Director. In the face of climate change and a growing global population, ensuring food and nutritional security requires leveraging expertise and technologies from various disciplines of basic sciences for environmental and agricultural sustainability. The vision of CESD is to generate worldwide food, nutritional, and bio-resources by nurturing environmentalists, biotechnologists, and agriculturists, enabling them to contribute to sustainable development goals (SDGs) on national and international platforms. The mission of CESD includes advancing the understanding of the environment and sustainability through multi-disciplinary education, research, and outreach programs; educating and training future generations to resolve environmental issues through innovative research; enhancing agricultural productivity and income under adverse environmental conditions; strengthening national research programs; introducing innovations and policy interventions for agricultural growth; raising awareness about the consequences of anthropogenic activities on climate systems and the environment; and integrating environment-sound perception for socio-economic concerns to enhance food and nutritional security.",
    "The Centre for Media and Mass Communication Studies (CMMS), led by Prof. (Dr.) Reshma Nasreen, is dedicated to providing premier education and training in journalism and mass communication. CMMS offers a B.A. (Honours with Research) in Journalism and Mass Communication and an M.A. in Journalism and Mass Communication program. In todays information and communication technology revolution, the influence and significance of Mass Media and Communication have expanded significantly. The programs at CMMS are designed to equip aspiring media professionals with industry and technical expertise essential for success in todays competitive media landscape. The curriculum trains candidates in various industry and technical skills such as audio-video editing, anchoring, content writing, photography, and media reporting, preparing them for roles in TV news channels, radio broadcasting, print media, digital journalism, and government communication setups. Graduates gain critical thinking abilities, effective communication skills, and a versatile skill set, enabling them to pursue various career opportunities in journalism, public relations, social media management, advertising, documentary filmmaking, content writing, video editing, photojournalism, and government/corporate communication roles.",
    "The Centre for Training and Development (CTD), led by Prof. (Dr.) Syeedun Nisa, is an initiative of Jamia Hamdard under the School of Management and Business Studies, established in 2022. The centre aims to provide education and skill development through practical and relevant training programs to help participants build their careers, develop corporate skill sets, and keep up with the fast-changing world. Its objective is to ensure that participants are better fit in the industry by making them industry and society ready. The curriculum is based on modern-day corporate requirements and academic excellence, focusing on areas such as research, education, behavioral sense, personality development, and skill development in domains like sales & marketing, international business, HR operations, finance, strategy, and leadership. CTD conducts training programs, workshops, seminars, and webinars at local, national, and international levels, providing trainees with necessary resources, mentorship, and skill-building activities for their professional career potential. The centre collaborates with organizations such as the African – Asian Rural Development Organisation, Islamiah Womens Arts & Science College, and The Garage Productions Private Limited to drive innovation, access specialized expertise, and advance research.",
    "The Centre for Teaching Support and Innovations (CTSI) at Jamia Hamdard is led by Prof. Mymoona Akhter, a dedicated academician committed to enhancing teaching methodologies and fostering innovation in education. Established in 2022, CTSI has swiftly emerged as a cornerstone of the academic landscape, playing a pivotal role in supporting faculty members and promoting inclusive education. At its core, the mission of CTSI transcends mere compliance with UGC mandates; it aspires to embody the vision and intent of the National Education Policy (NEP). With a focus on flexibility and mobility, CTSI endeavors to break down barriers between general and vocational education, creating an environment conducive to holistic learning and growth. CTSIs significance in the academic arena lies in its multifaceted approach to supporting faculty members and advancing teaching practices. Through workshops, seminars, and collaborative platforms, CTSI facilitates the exchange of ideas and best practices among faculty members, fostering a culture of continuous improvement and innovation in teaching methodologies. By promoting academic equivalence across different domains of education, CTSI contributes to the holistic development of students and the overall enhancement of the educational experience at Jamia Hamdard.",
    "The Industry Academia Partnership Cell (IAPC) at Jamia Hamdard is spearheaded by Prof. Syeedun Nisa, a distinguished academician committed to fostering collaborations between academia and industry. With a vision to produce technologically competent, morally and emotionally strong, and ethically sound professionals dedicated to the service of mankind, IAPC strives to bridge the gap between theoretical knowledge and practical application. At its core, the mission of IAPC is to develop technically and ethically strong students who can perform effectively in both personal and professional life. By nurturing a conducive environment for industry-academia collaboration, IAPC endeavors to equip students with the skills and mindset necessary to excel in a rapidly evolving global landscape. The core team of IAPC comprises senior faculties, industry representatives, assistant TPO, and faculties and student coordinators from each school. This diverse team brings together expertise from academia and industry, ensuring a holistic approach to fostering partnerships and enhancing students employability. From the School of Management and Business Studies (SMBS), Ms. Manpreet Kaur Kohli and Dr. Jwaad A Khan serve as coordinators, actively contributing to the collaborative efforts of IAPC. Their involvement underscores the commitment of SMBS to nurturing industry-ready professionals equipped with the requisite knowledge and skills to thrive in the competitive business environment.",
    "Recruitment processes at the Industry Academia Partnership Cell (IAPC) encompass several stages aimed at identifying and selecting suitable candidates for various job roles. Typically, companies initiate the process by conducting Pre-Placement talks to introduce students to their organization and job opportunities. This is followed by written tests to assess students knowledge and skills. If applicable, Group Discussions are conducted to evaluate candidates communication and interpersonal abilities. Subsequently, eligible and shortlisted students undergo Technical and HR interviews to further assess their suitability for the role. Students are informed about job openings through a structured process facilitated by IAPC. When an organization expresses interest in recruiting students, details of job openings and profiles are shared with relevant IAPC committee members and student coordinators. Interested students register for the recruitment process and provide their consent to participate. After the recruitment process concludes, shortlisted students undergo background verification as per the companys policy. If the eligibility criteria and background verification are satisfactory, students are extended job offers by the respective companies. The remuneration offered to students recruited through IAPC varies depending on factors such as job role, company policies, and individual performance. Students have received attractive salary packages ranging from 3.5 Lakhs Per Annum (LPA) to 31 LPA from renowned companies. Organizations frequently engage in campus hiring with each batch and consistently express satisfaction with the caliber and performance of the recruited students.",
    "For the academic session 2024-25, admission to undergraduate programs at Jamia Hamdard will be primarily based on merit in various entrance exams. Candidates may qualify for admission based on their performance in NEET (UG), JEE (Main-Paper-I) 2024, CUET-2024, or CLAT-2024, depending on the program they are applying for. Similarly, admission to programs other than undergraduate ones will also be conducted through a variety of criteria. This may include marks obtained in the Qualifying Examination, performance in interviews, entrance tests, or other approved modes of assessment. In cases where candidates have not qualified in NEET (UG), JEE, CLAT, or CUET, Jamia Hamdard reserves the right to consider their applications if suitable rank holders are not available in a particular discipline. However, such considerations are subject to approval by the Competent Authority. To apply for admission to programs at Jamia Hamdard for the academic session 2024-25, candidates are required to submit their applications online through the designated Online Admission Portal 2024. For MBBS/MD/MS programs at HIMSR (Hamdard Institute of Medical Sciences and Research), the admission process shall be governed by the guidelines provided by the National Medical Commission (NMC).",
    "Applicants aspiring to join Jamia Hamdard are strongly advised to thoroughly review the contents of the Information Bulletin & Prospectus. Its crucial for them to familiarize themselves with the intricacies of the Admission/Entrance Test/GD/Interview/Counselling processes conducted by the university. This familiarity ensures that candidates are well-prepared and understand the requirements and procedures involved in seeking admission. Jamia Hamdard maintains stringent standards regarding admission criteria. Even if a candidate successfully clears all requisite eligibility conditions, such as the Entrance Test/GD/Interview merit and fulfills the criteria outlined in the offer letter, admission may be denied if the candidate fails to meet the eligibility criteria specified by the university. In such cases, Jamia Hamdard cannot be held responsible for the denial of admission. As per the instructions provided, a bachelors degree signifies that it has been obtained under either the 10+2+3 or 10+2+4 system of education, or an equivalent system. This clarification ensures that candidates understand the educational qualifications required for admission to various programs offered by Jamia Hamdard. Candidates whose qualifying examinations are not recognized by either the Association of Indian Universities (AIU) or Jamia Hamdard will not have their application forms considered for admission. Recognition by these authorities is crucial for ensuring the validity and eligibility of the candidates academic qualifications.",

    ]

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load the model
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/Model')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Model')

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)


In [ ]:
from multiprocessing import Pool

# Function to compute relevance score between question and context
def compute_relevance_score(args):
    question, context = args
    inputs = tokenizer(question, context, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.start_logits + outputs.end_logits
    relevance_score = torch.softmax(logits, dim=1)[:, 1].item()
    return relevance_score

# Function to select the most relevant context for the question
def select_context(question, context_pool):
    max_score = -1
    selected_context = None

    # Create a multiprocessing Pool
    with Pool() as p:
        # Compute relevance scores in parallel
        scores = p.map(compute_relevance_score, [(question, context) for context in context_pool])

    for context, score in zip(context_pool, scores):
        # Additional criteria for context selection
        # 1. Length of the context
        context_length = len(context.split())

        # 2. Presence of question keywords in context
        question_keywords = set(question.lower().split())
        context_keywords = set(context.lower().split())
        common_keywords = len(question_keywords.intersection(context_keywords))

        # Combine relevance score and additional criteria
        combined_score = score * (common_keywords / len(question_keywords)) * (1 / context_length)

        # Update the selected context if the combined score is higher than the maximum score
        if combined_score > max_score:
            max_score = combined_score
            selected_context = context

    # Return the selected context
    return selected_context


In [ ]:
# Function to use the selected context to get an answer
def answer_question(question, context_pool):
    # Select the most relevant context
    selected_context = select_context(question, context_pool)

    # If no context is selected, return None
    if selected_context is None:
        return None

    # Use the selected context to answer the question
    result = qa_pipeline({'question': question, 'context': selected_context})
    return result['answer']


In [ ]:
# Initialize selected_context globally
selected_context = None

In [ ]:
import json

def save_feedback(data, file_path='/content/drive/MyDrive/Model/FeedBack.json'):
    with open(file_path, 'w') as f:
        json.dump(data, f)

# Define a function to load existing feedback
def load_feedback(file_path='/content/drive/MyDrive/Model/FeedBack.json'):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            if not data:  # If the file is empty or data is not in expected format
                return []
            if isinstance(data, dict):
                return [data]  # Wrap the single dictionary in a list
            elif isinstance(data, list):
                return data  # Return list of dictionaries as expected
            else:
                print("Unexpected data format in the file.")
                return []
    except FileNotFoundError:
        # If the file does not exist, create it with an empty list
        save_feedback([], file_path)
        return []

# Function to ask a question and get an answer
def ask_and_get_answer():
    global selected_context  # Access the global selected_context variable

    # Ask for a question input
    question = input("Ask a question (type 'exit' or 'quit' to end): ")

    # Check if the user wants to exit
    if question.lower() in ['exit', 'quit']:
        return None, None

    # Get the answer for the question
    answer = answer_question(question, context_pool)

    # Print the selected context and answer
    print("Selected Context:", selected_context)
    print("Answer:", answer)

    # Ask the user if the answer and context are correct
    feedback = input("Is the answer correct? (yes/no): ")

    # If the user is not satisfied, provide the option to select a different context
    if feedback.lower() == 'no':
        print("Available contexts:")
        for i, context in enumerate(context_pool, 1):
            print(f"{i}. {context}")

        # Ask the user to select a context
        selected_index = int(input("Enter the number corresponding to the selected context: ")) - 1
        selected_context = context_pool[selected_index]

        # Get the answer based on the selected context
        answer = answer_question(question, [selected_context])

        # Print the updated selected context and answer
        print("Updated Selected Context:", selected_context)
        print("Updated Answer:", answer)

        # Ask the user if the updated answer is correct
        feedback = input("Is the updated answer correct? (yes/no): ")

        # If the user is still not satisfied, allow them to enter the correct answer
        if feedback.lower() == 'no':
            correct_answer = input("Enter the correct answer: ")
            answer = correct_answer

    return question, answer

# Start the interaction loop
while True:
    # Ask a question and get the answer
    question, answer = ask_and_get_answer()

    # If the user wants to exit, end the loop
    if question is None:
        print("Exiting...")
        break

    # Load the original data from FeedBack.json
    feedback_data = load_feedback()

    # Print the loaded feedback data for debugging
    print("Feedback Data:", feedback_data)

    # Update the selected context and answer in the original data
for item in feedback_data:
    print("Item:", item)
    for paragraphs in item['paragraphs']:
        print("Paragraphs:", paragraphs)
        if 'qas' in paragraphs:
            for qa in paragraphs['qas']:
                print("QA:", qa)
                if qa['question'] == question:
                    qa['answers'][0]['text'] = answer  # Update the answer text
        else:
            print("No 'qas' key found in paragraphs:", paragraphs)



    # Save the updated data back to FeedBack.json
    save_feedback(feedback_data, '/content/drive/MyDrive/Model/FeedBack.json')
